<a href="https://colab.research.google.com/github/ykitaguchi77/Colab_Scripts/blob/master/yolov5_gradCAM_corneAI_revision_20250705.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**YOLOv5_GradCAM_CorneAI**

論文revision用

https://github.com/pooya-mohammadi/yolov5-gradcam

###**⭐️⭐️Area of interestの計算**

結果をcsvに保存する

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

%cd /content
!pip uninstall deep_utils -y
!pip install -U git+https://github.com/pooya-mohammadi/deep_utils.git --q
!pip install torch --q
!pip install torchvision --q
!pip install -U opencv-python --q
print("[INFO] To use new installed version of opencv, the session should be restarted!!!!")

!git clone https://github.com/pooya-mohammadi/yolov5-gradcam

import os
os.chdir('/content/yolov5-gradcam')

model_path = "/gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forcresco/weights/eye_nii_2202_onecaseoneimage2_doctorcompare_yolov5s_epoch200_batch16_89.8p/last.pt"
img_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/Ueno_Mix1039/Images"


Mounted at /gdrive
/content
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 95.8 MB/s eta 0:00:00
[INFO] To use new installed version of opencv, the session should be restarted!!!!
Cloning into 'yolov5-gradcam'...
remote: Enumerat

In [ ]:
### GPUの種類により結果が異なるため、CPUで計算したものを採用

import os
os.chdir("/content/yolov5-gradcam")

import pandas as pd
import numpy as np
import cv2
# from google.colab.patches import cv2_imshow # Colab環境でcv2_imshowを使用する場合にコメント解除
from deep_utils.utils.box_utils.boxes import Box
import os
from tqdm import tqdm
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import gc
import torchvision
import warnings
import sys

# FutureWarningを一時的に抑制（register_backward_hook使用時）
warnings.filterwarnings("ignore", category=FutureWarning,
                      message=".*Using a non-full backward hook.*")

# YOLOv5の必要な関数をインポート - attempt_loadの場所を修正
from utils.general import non_max_suppression as yolo_nms, xywh2xyxy, box_iou
from utils.datasets import letterbox
from models.experimental import attempt_load  # 修正: models.experimentalから

# YOLOv5のカスタムクラスをインポートします
from models.yolo import Model, Detect
from models.common import Conv, Bottleneck, C3, SPPF, Concat

# これまでに登場したクラスをすべてリストに追加します
from torch.nn import Sequential, Conv2d, BatchNorm2d, SiLU, MaxPool2d, Upsample, ModuleList
torch.serialization.add_safe_globals([
    Model, Detect, Sequential, ModuleList, Conv, Bottleneck, C3, SPPF, Concat,
    Conv2d, BatchNorm2d, SiLU, MaxPool2d, Upsample
])

# === 最適化: PYTORCH_CUDA_ALLOC_CONFの詳細設定 ===
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True,max_split_size_mb:512"

# === 最適化: PyTorch 2.0以上の場合、コンパイルキャッシュをリセット ===
if hasattr(torch._dynamo, 'reset'):
    torch._dynamo.reset()

# グローバル変数として定義（エラー回避のため）
global img_size
img_size = 640

# ========== 統合版：メモリ管理ユーティリティ ==========
class GPUMemoryMonitor:
    """GPU メモリの監視と管理を行うクラス"""

    def __init__(self, warning_threshold=0.8, critical_threshold=0.9):
        """
        Args:
            warning_threshold: 警告を出すメモリ使用率の閾値 (0-1)
            critical_threshold: クリティカル警告を出すメモリ使用率の閾値 (0-1)
        """
        self.warning_threshold = warning_threshold
        self.critical_threshold = critical_threshold
        self.device_name = torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU"
        self.total_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3 if torch.cuda.is_available() else 0

    def get_memory_stats(self):
        """現在のメモリ使用状況を取得"""
        if not torch.cuda.is_available():
            return {
                'allocated': 0,
                'reserved': 0,
                'free': 0,
                'total': 0,
                'usage_ratio': 0
            }

        allocated = torch.cuda.memory_allocated() / 1024**3
        reserved = torch.cuda.memory_reserved() / 1024**3
        total = self.total_memory
        free = total - reserved
        usage_ratio = reserved / total if total > 0 else 0

        return {
            'allocated': allocated,
            'reserved': reserved,
            'free': free,
            'total': total,
            'usage_ratio': usage_ratio
        }

    def display_memory_status(self, prefix=""):
        """メモリ状況を視覚的に表示"""
        stats = self.get_memory_stats()

        # 使用率に応じた色付け（ANSIエスケープコード）
        if stats['usage_ratio'] > self.critical_threshold:
            color = "\033[91m"  # 赤
            status = "⚠️  CRITICAL"
        elif stats['usage_ratio'] > self.warning_threshold:
            color = "\033[93m"  # 黄
            status = "⚠️  WARNING"
        else:
            color = "\033[92m"  # 緑
            status = "✅ OK"
        reset_color = "\033[0m"

        # プログレスバーの作成
        bar_length = 30
        filled_length = int(bar_length * stats['usage_ratio'])
        bar = '█' * filled_length + '░' * (bar_length - filled_length)

        print(f"\n{prefix}GPU Memory Status ({self.device_name}) {status}")
        print(f"├─ Usage: {color}[{bar}] {stats['usage_ratio']*100:.1f}%{reset_color}")
        print(f"├─ Allocated: {stats['allocated']:.2f} GB")
        print(f"├─ Reserved:  {stats['reserved']:.2f} GB")
        print(f"├─ Free:      {stats['free']:.2f} GB")
        print(f"└─ Total:     {stats['total']:.2f} GB")

    def check_memory_health(self):
        """メモリ使用状況をチェックし、必要に応じてクリーンアップを推奨"""
        stats = self.get_memory_stats()

        if stats['usage_ratio'] > self.critical_threshold:
            return 'critical', stats
        elif stats['usage_ratio'] > self.warning_threshold:
            return 'warning', stats
        else:
            return 'ok', stats

    def cleanup_if_needed(self, force=False):
        """必要に応じてメモリクリーンアップを実行"""
        health, stats = self.check_memory_health()

        if health == 'critical' or force:
            print(f"\n🧹 メモリクリーンアップを実行中... (使用率: {stats['usage_ratio']*100:.1f}%)")
            aggressive_memory_cleanup()  # 統合：より積極的なクリーンアップ関数を使用

            # クリーンアップ後の状態を確認
            new_stats = self.get_memory_stats()
            freed = stats['reserved'] - new_stats['reserved']
            print(f"✨ クリーンアップ完了: {freed:.2f} GB 解放されました")

            return True
        return False

# === 統合：旧バージョンの有用な関数を追加 ===
def find_cuda_tensors():
    """メモリに残っているCUDAテンソルを検出"""
    cuda_tensors = []
    for obj in gc.get_objects():
        try:
            if torch.is_tensor(obj) and obj.is_cuda:
                cuda_tensors.append((type(obj), obj.size(), obj.element_size() * obj.nelement()))
        except:
            pass
    return cuda_tensors

def log_cuda_tensors(message=""):
    """CUDAテンソルの状態をログ出力"""
    cuda_tensors = find_cuda_tensors()
    if cuda_tensors:
        print(f"{message} 検出されたCUDAテンソル数: {len(cuda_tensors)}")
        # 大きいテンソルのみ表示
        large_tensors = [t for t in cuda_tensors if t[2] > 1024*1024]  # 1MB以上
        if large_tensors:
            print(f"  大きなテンソル (>1MB): {len(large_tensors)}個")
            for i, (tensor_type, size, bytes_size) in enumerate(large_tensors[:5]):
                print(f"    {i+1}. Size: {size}, Memory: {bytes_size/1024**2:.2f}MB")

def aggressive_memory_cleanup():
    """積極的なメモリクリーンアップを実行"""
    # Pythonのガベージコレクション
    gc.collect()

    # PyTorchのキャッシュクリア
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()

        # CUDAのメモリプールをリセット（PyTorch 1.10以降）
        if hasattr(torch.cuda, 'reset_peak_memory_stats'):
            torch.cuda.reset_peak_memory_stats()
        if hasattr(torch.cuda, 'reset_accumulated_memory_stats'):
            torch.cuda.reset_accumulated_memory_stats()

    # PyTorch 2.0以上の場合、コンパイルキャッシュもクリア
    if hasattr(torch._dynamo, 'reset'):
        torch._dynamo.reset()

def set_model_gradients(model, layer_name, enable=True):
    """特定のレイヤーのみ勾配計算を有効化"""
    # まず全体の勾配を無効化
    for param in model.model.parameters():
        param.requires_grad = False

    # 特定のレイヤーのみ有効化
    if enable:
        target_layer = find_yolo_layer(model, layer_name)
        for param in target_layer.parameters():
            param.requires_grad = True

# ========== 既存のクラスは変更なし ==========
# YOLOV5TorchObjectDetectorクラス（変更なし）
class YOLOV5TorchObjectDetector(nn.Module):
    def __init__(self,
                 model_weight,
                 device,
                 img_size,
                 names=None,
                 mode="eval",
                 confidence=0.25,
                 iou_thresh=0.45,
                 agnostic_nms=False):
        super(YOLOV5TorchObjectDetector, self).__init__()
        self.device = device
        self.model = None
        self.img_size = img_size
        self.mode = mode
        self.confidence = confidence
        self.iou_thresh = iou_thresh
        self.agnostic = agnostic_nms

        self.model = attempt_load(model_weight, device=device)
        print("[INFO] Model is loaded")

        # === 最適化: デフォルトでは勾配計算を無効化 ===
        # Grad-CAM使用時のみ必要なレイヤーで有効化
        for param in self.model.parameters():
            param.requires_grad = False
        self.model.to(device)

        if self.mode == "train":
            self.model.train()
        else:
            self.model.eval()

        if names is None:
            print("[INFO] fetching names from coco file")
            self.names = ["person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat",
                          "traffic light",
                          "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat", "dog", "horse", "sheep",
                          "cow",
                          "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella", "handbag", "tie", "suitcase",
                          "frisbee",
                          "skis", "snowboard", "sports ball", "kite", "baseball bat", "baseball glove", "skateboard",
                          "surfboard",
                          "tennis racket", "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana",
                          "apple",
                          "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair",
                          "couch",
                          "potted plant", "bed", "dining table", "toilet", "tv", "laptop", "mouse", "remote",
                          "keyboard", "cell phone",
                          "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
                          "teddy bear",
                          "hair drier", "toothbrush"]
        else:
            self.names = names

    @staticmethod
    def non_max_suppression(prediction, logits, conf_thres=0.25, iou_thres=0.45, classes=None, agnostic=False,
                            multi_label=False, labels=(), max_det=300):
        nc = prediction.shape[2] - 5  # number of classes
        xc = prediction[..., 4] > conf_thres  # candidates

        assert 0 <= conf_thres <= 1, f"Invalid Confidence threshold {conf_thres}, valid values are between 0.0 and 1.0"
        assert 0 <= iou_thres <= 1, f"Invalid IoU {iou_thres}, valid values are between 0.0 and 1.0"

        min_wh, max_wh = 2, 4096  # (pixels) minimum and maximum box width and height
        max_nms = 30000  # maximum number of boxes into torchvision.ops.nms()
        time_limit = 10.0  # seconds to quit after
        redundant = True  # require redundant detections
        multi_label &= nc > 1  # multiple labels per box (adds 0.5ms/img)
        merge = False  # use merge-NMS

        t = time.time()
        output = [torch.zeros((0, 6), device=prediction.device)] * prediction.shape[0]
        logits_output = [torch.zeros((0, 80), device=logits.device)] * logits.shape[0]
        for xi, (x, log_) in enumerate(zip(prediction, logits)):  # image index, image inference
            x = x[xc[xi]]  # confidence
            log_ = log_[xc[xi]]
            if labels and len(labels[xi]):
                l = labels[xi]
                v = torch.zeros((len(l), nc + 5), device=x.device)
                v[:, :4] = l[:, 1:5]  # box
                v[:, 4] = 1.0  # conf
                v[range(len(l)), l[:, 0].long() + 5] = 1.0  # cls
                x = torch.cat((x, v), 0)

            if not x.shape[0]:
                continue

            x[:, 5:] *= x[:, 4:5]  # conf = obj_conf * cls_conf
            box = xywh2xyxy(x[:, :4])

            if multi_label:
                i, j = (x[:, 5:] > conf_thres).nonzero(as_tuple=False).T
                x = torch.cat((box[i], x[i, j + 5, None], j[:, None].float()), 1)
            else:  # best class only
                conf, j = x[:, 5:].max(1, keepdim=True)
                x = torch.cat((box, conf, j.float()), 1)[conf.view(-1) > conf_thres]
                log_ = log_[conf.view(-1) > conf_thres]
            if classes is not None:
                x = x[(x[:, 5:6] == torch.tensor(classes, device=x.device)).any(1)]

            n = x.shape[0]  # number of boxes
            if not n:
                continue
            elif n > max_nms:
                x = x[x[:, 4].argsort(descending=True)[:max_nms]]

            c = x[:, 5:6] * (0 if agnostic else max_wh)  # classes
            boxes, scores = x[:, :4] + c, x[:, 4]  # boxes (offset by class), scores
            i = torchvision.ops.nms(boxes, scores, iou_thres)  # NMS
            if i.shape[0] > max_det:
                i = i[:max_det]
            if merge and (1 < n < 3E3):
                iou = box_iou(boxes[i], boxes) > iou_thres
                weights = iou * scores[None]
                x[i, :4] = torch.mm(weights, x[:, :4]).float() / weights.sum(1, keepdim=True)
                if redundant:
                    i = i[iou.sum(1) > 1]

            output[xi] = x[i]
            logits_output[xi] = log_[i]
            assert log_[i].shape[0] == x[i].shape[0]
            if (time.time() - t) > time_limit:
                print(f"WARNING: NMS time limit {time_limit}s exceeded")
                break

        return output, logits_output

    @staticmethod
    def yolo_resize(img, new_shape=(640, 640), color=(114, 114, 114), auto=True, scaleFill=False, scaleup=True):
        return letterbox(img, new_shape=new_shape, color=color, auto=auto, scaleFill=scaleFill, scaleup=scaleup)

    def forward(self, img):
        # === 変更点: 推論部分のwith torch.no_grad()を削除 ===
        # Grad-CAMのために勾配計算を有効にする
        prediction, logits, _ = self.model(img, augment=False)
        prediction, logits = self.non_max_suppression(prediction, logits, self.confidence, self.iou_thresh,
                                                      classes=None,
                                                      agnostic=self.agnostic)

        # === 改善点: NMS後の処理は勾配不要なのでno_grad()を使用 ===
        with torch.no_grad():
            # 以下の処理はCPUで行うことでGPUメモリを節約
            self.boxes, self.class_names, self.classes, self.confidences = [[[] for _ in range(img.shape[0])] for _ in
                                                                            range(4)]
            for i, det in enumerate(prediction):  # detections per image
                if len(det):
                    # detをCPUに移動し、元のGPUテンソルは即座に削除
                    det_cpu = det.cpu()
                    del det  # GPUメモリを即座に解放

                    for *xyxy, conf, cls in det_cpu:
                        xyxy[0] = max(0, xyxy[0])
                        xyxy[1] = max(0, xyxy[1])
                        xyxy[2] = min(img_size, xyxy[2])
                        xyxy[3] = min(img_size, xyxy[3])

                        bbox = Box.box2box(xyxy,
                                           in_source=Box.BoxSource.Torch,
                                           to_source=Box.BoxSource.Numpy,
                                           return_int=True)
                        self.boxes[i].append(bbox)
                        self.confidences[i].append(round(conf.item(), 2))
                        cls = int(cls.item())
                        self.classes[i].append(cls)
                        if self.names is not None:
                            self.class_names[i].append(self.names[cls])
                        else:
                            self.class_names[i].append(cls)

            # predictionは不要になったので削除
            del prediction

        return [self.boxes, self.classes, self.class_names, self.confidences], logits

    def preprocessing(self, img):
        if len(img.shape) != 4:
            img = np.expand_dims(img, axis=0)
        im0 = img.astype(np.uint8)
        del img  # 元の画像データを削除

        # リサイズ処理を効率化
        resized_imgs = []
        for im in im0:
            resized_img = self.yolo_resize(im, new_shape=self.img_size)[0]
            resized_imgs.append(resized_img)

        img = np.array(resized_imgs)
        del resized_imgs, im0  # 中間データを削除

        img = img.transpose((0, 3, 1, 2))
        img = np.ascontiguousarray(img)
        img = torch.from_numpy(img).to(self.device)
        img = img / 255.0
        return img

# YOLOV5GradCAMクラス（変更なし）
class YOLOV5GradCAM:
    def __init__(self, model, layer_name, img_size=(640, 640), method="gradcam", debug=False):
        self.model = model
        self.gradients = dict()
        self.activations = dict()
        self.method = method
        self.cls_names = []
        self.debug = debug  # デバッグフラグ
        self.layer_name = layer_name  # デバッグ用に保存

        # === 改善点1: フックハンドルを保存 ===
        self.forward_handle = None
        self.backward_handle = None
        self.target_layer = None

        def backward_hook(module, grad_input, grad_output):
            # === 変更点: detach()してからclone()でより安全に ===
            try:
                # detach()で計算グラフから切り離してからclone()
                self.gradients["value"] = grad_output[0].detach().clone()
                if self.debug:
                    print(f"[DEBUG] Successfully detached and cloned grad_output for {layer_name}")
            except Exception as e:
                if self.debug:
                    print(f"[DEBUG] Error in backward_hook: {e}")
                raise
            return None

        def forward_hook(module, input, output):
            # === 変更点: outputもdetach().clone()してより安全に ===
            self.activations["value"] = output.detach().clone()
            return None

        self.target_layer = find_yolo_layer(self.model, layer_name)

        # === 最適化: 必要なレイヤーのみ勾配計算を有効化 ===
        set_model_gradients(self.model, layer_name, enable=True)

        # === 改善点2: フックハンドルを保存 ===
        self.forward_handle = self.target_layer.register_forward_hook(forward_hook)
        self.backward_handle = self.target_layer.register_backward_hook(backward_hook)

    def cleanup(self):
        """フックとメモリを明示的にクリーンアップする"""
        # === 改善点3: フックの削除 ===
        if self.forward_handle is not None:
            self.forward_handle.remove()
            self.forward_handle = None
            if self.debug:
                print(f"[DEBUG] Forward hook removed for {self.layer_name}")

        if self.backward_handle is not None:
            self.backward_handle.remove()
            self.backward_handle = None
            if self.debug:
                print(f"[DEBUG] Backward hook removed for {self.layer_name}")

        # === 改善点4: 辞書のクリア ===
        self.gradients.clear()
        self.activations.clear()

        # === 最適化: レイヤーの勾配計算を無効化 ===
        set_model_gradients(self.model, self.layer_name, enable=False)

        # === 改善点5: ターゲットレイヤーへの参照を削除 ===
        self.target_layer = None

        if self.debug:
            print(f"[DEBUG] GradCAM cleanup completed for {self.layer_name}")

    def __enter__(self):
        """コンテキストマネージャーのエントリーポイント"""
        if self.debug:
            print(f"[DEBUG] Entering GradCAM context for {self.layer_name}")
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        """コンテキストマネージャーの終了ポイント（自動クリーンアップ）"""
        if self.debug:
            print(f"[DEBUG] Exiting GradCAM context for {self.layer_name}")
            if exc_type is not None:
                print(f"[DEBUG] Exception occurred: {exc_type.__name__}: {exc_val}")
        self.cleanup()
        # Falseを返すことで例外を再発生させる
        return False

    def __del__(self):
        """デストラクタでも念のためクリーンアップ"""
        try:
            self.cleanup()
        except:
            pass

    def forward(self, input_img, class_idx=True):
        saliency_maps = []
        b, c, h, w = input_img.size()

        # === 改善点: 各クラスごとに順伝播を再実行 ===
        # まず、一度だけフォワードパスを実行してtop3_indicesを取得
        preds, logits = self.model(input_img)

        # topkの計算は勾配不要なのでno_grad()を適用
        with torch.no_grad():
            _, top3_indices = torch.topk(logits[0], k=3)

            if top3_indices.numel() > 0:
                preds[1][0] = top3_indices.tolist()[0]
                preds[2][0] = [names[i] for i in preds[1][0]]
                self.cls_names = preds[2][0]
            else:
                self.cls_names = []

            # top3_indicesは不要になったので削除
            del top3_indices

        if self.method == "eigencam":
            # EigenCAMの場合は一度のフォワードパスで計算
            with torch.no_grad():  # EigenCAMは勾配不要
                saliency_map = self._eigencam()
                saliency_maps.append(saliency_map)
        else:
            # GradCAMまたはGradCAM++の場合、各クラスごとに処理
            for cls, cls_name in zip(preds[1][0], preds[2][0]):
                # === 重要な変更: 各クラスごとに新しいフォワードパスを実行 ===
                # モデルの勾配をリセット
                self.model.zero_grad()

                # === 追加: インプレース操作を避けるためにinput_imgをclone ===
                input_img_clone = input_img.clone()

                # 新しいフォワードパス
                _, new_logits = self.model(input_img_clone)

                # 即座に不要な変数を削除
                del input_img_clone

                if class_idx:
                    score = new_logits[0][0][cls]
                else:
                    score = new_logits[0][0].max()

                # === 変更点: retain_graph=Trueを削除 ===
                score.backward()  # retain_graph=Trueを削除

                # === 追加: gradientsとactivationsが存在することを確認 ===
                if "value" not in self.gradients or "value" not in self.activations:
                    print(f"[WARNING] No gradients or activations for class {cls_name}")
                    del score, new_logits  # 存在しない場合はここで削除
                    continue

                gradients = self.gradients["value"]
                activations = self.activations["value"]
                b, k, u, v = gradients.size()

                if self.method == "gradcam":
                    weights = self._gradcam_weights(gradients, b, k)
                elif self.method == "gradcampp":
                    weights = self._gradcampp_weights(gradients, activations, score, b, k, u, v)

                # scoreとnew_logitsは不要になったので削除
                del score
                del new_logits

                # saliency_mapの計算は勾配不要なのでno_grad()を適用
                with torch.no_grad():
                    saliency_map = (weights * activations).sum(1, keepdim=True)
                    del weights  # weightsを即座に削除

                    saliency_map = F.relu(saliency_map)
                    saliency_map = F.interpolate(saliency_map, size=(h, w), mode="bilinear", align_corners=False)
                    saliency_map_min, saliency_map_max = saliency_map.min(), saliency_map.max()
                    # ゼロ除算を避けるためにepsilonを追加
                    saliency_map = (saliency_map - saliency_map_min).div(saliency_map_max - saliency_map_min + 1e-8).data

                    # min/maxは不要なので削除
                    del saliency_map_min, saliency_map_max

                saliency_maps.append(saliency_map)

                # === 追加: 各クラスの処理後にメモリクリーンアップ ===
                # 勾配とアクティベーションを即座に削除
                if "value" in self.gradients:
                    del self.gradients["value"]
                if "value" in self.activations:
                    del self.activations["value"]

                # 不要な中間変数を削除
                del gradients, activations

                # CUDAキャッシュのクリア（必要に応じて）
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()

        return saliency_maps, logits, preds, self.cls_names

    def _gradcam_weights(self, gradients, b, k):
        # view操作は新しいテンソルを作成しないので効率的
        alpha = gradients.view(b, k, -1).mean(2)
        weights = alpha.view(b, k, 1, 1)
        del alpha  # alphaを削除
        return weights

    def _gradcampp_weights(self, gradients, activations, score, b, k, u, v):
        alpha_num = gradients.pow(2)
        alpha_denom = gradients.pow(2).mul(2) + \
            activations.mul(gradients.pow(3)).view(b, k, u*v).sum(-1, keepdim=True).view(b, k, 1, 1)
        alpha_denom = torch.where(alpha_denom != 0.0, alpha_denom, torch.ones_like(alpha_denom))
        alpha = alpha_num.div(alpha_denom + 1e-7)
        del alpha_num, alpha_denom  # 中間変数を削除

        relu_grad = F.relu(score.exp() * gradients)
        weights = (relu_grad * alpha).view(b, k, u*v).sum(-1).view(b, k, 1, 1)
        del relu_grad, alpha  # 中間変数を削除

        return weights

    def _eigencam(self):
        activations = self.activations["value"]
        b, k, u, v = activations.size()
        activations_reshaped = activations.view(b, k, -1)

        # Compute covariance matrix
        cov = activations_reshaped @ activations_reshaped.transpose(-1, -2)
        cov /= (u * v)

        # Compute eigenvectors
        eigenvalues, eigenvectors = torch.linalg.eigh(cov)
        del cov  # 共分散行列を削除

        # Select the eigenvector corresponding to the largest eigenvalue
        leading_eigenvector = eigenvectors[..., -1].unsqueeze(-1)
        del eigenvalues, eigenvectors  # 不要な固有値・固有ベクトルを削除

        # Compute EigenCAM
        eigen_cam = (activations_reshaped.transpose(-1, -2) @ leading_eigenvector).view(b, u, v)
        del activations_reshaped, leading_eigenvector  # 中間変数を削除

        eigen_cam = F.relu(eigen_cam)

        # Normalize
        eigen_cam_min, eigen_cam_max = eigen_cam.min(), eigen_cam.max()
        eigen_cam = (eigen_cam - eigen_cam_min) / (eigen_cam_max - eigen_cam_min + 1e-8)
        del eigen_cam_min, eigen_cam_max  # min/maxを削除

        # Resize to match the input image size
        eigen_cam = F.interpolate(eigen_cam.unsqueeze(1), size=(640, 640), mode="bilinear", align_corners=False)
        return eigen_cam

    def __call__(self, input_img):
        return self.forward(input_img)

# find_yolo_layer関数
def find_yolo_layer(model, layer_name):
   hierarchy = layer_name.split("_")
   target_layer = model.model._modules[hierarchy[0]]

   for h in hierarchy[1:]:
       target_layer = target_layer._modules[h]
   return target_layer

# get_aoi関数（メモリ効率化版）
def get_aoi(bbox, masks, threshold):
    total_intersect_pixels = 0

    with torch.no_grad():  # 全体を勾配計算不要で囲む
        for mask in masks:
            # マスクをCPUで処理（GPUメモリ節約）
            mask_cpu = mask.squeeze().mul(255).add_(0.5).clamp_(0, 255).detach().cpu().numpy().astype(np.uint8)
            del mask  # 元のマスクは即座に削除

            mask_cpu = np.nan_to_num(mask_cpu, nan=0.0).astype(np.uint8)

            # 閾値処理（より効率的な方法）
            binary_mask = mask_cpu >= (threshold * 255)
            del mask_cpu  # 処理済みのマスクは削除

            x1, y1, x2, y2 = bbox
            mask_bbox = binary_mask[y1:y2, x1:x2]

            intersect_pixels = np.sum(mask_bbox)
            total_intersect_pixels += intersect_pixels

            del mask_bbox, binary_mask  # 不要な変数を削除

        mask_bbox_area = (y2 - y1) * (x2 - x1)
        AOI = total_intersect_pixels / mask_bbox_area if mask_bbox_area > 0 else 0

    return AOI

# ========== 統合版：calculate_aoi関数（視覚的メモリモニタリング機能付き） ==========
def calculate_aoi(folder_path, csv_path, threshold, model, target_layers,
                  start_index=None, end_index=None, skip_processed=True, debug_mode=False):
    df = pd.read_csv(csv_path)
    df = df.loc[:, ~df.columns.str.contains("^Unnamed")]

    # メモリモニターの初期化
    memory_monitor = GPUMemoryMonitor(warning_threshold=0.75, critical_threshold=0.85)

    # 開始時のメモリ状況を表示
    memory_monitor.display_memory_status("🚀 処理開始時")

    # === システム情報の表示 ===
    print("\n=== システム情報 ===")
    print(f"Python version: {sys.version}")
    print(f"PyTorch version: {torch.__version__}")
    print(f"CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"CUDA version: {torch.version.cuda}")
        print(f"GPU count: {torch.cuda.device_count()}")
        for i in range(torch.cuda.device_count()):
            print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")
            props = torch.cuda.get_device_properties(i)
            print(f"    Total memory: {props.total_memory / 1024**3:.2f} GB")
    print("="*50 + "\n")

    # AOIカラムの定義
    all_layers_cols = [
        ("model_17_cv3_conv", f"AOI_{threshold}_layermodel_17_cv3_conv"),
        ("model_20_cv3_conv", f"AOI_{threshold}_layermodel_20_cv3_conv"),
        ("model_23_cv3_conv", f"AOI_{threshold}_layermodel_23_cv3_conv"),
        ("model_24_m_0", f"AOI_{threshold}_layer24_m_0"),
        ("model_24_m_1", f"AOI_{threshold}_layer24_m_1"),
        ("model_24_m_2", f"AOI_{threshold}_layer24_m_2")
    ]

    # カラムが存在しない場合は追加
    for layer_name, col_name in all_layers_cols:
        if col_name not in df.columns:
            df[col_name] = None

    # 処理済み画像の判定関数
    def is_processed(row):
        """すべてのAOIカラムに値が入っているかチェック"""
        for _, col_name in all_layers_cols:
            if pd.isna(row[col_name]) or row[col_name] is None:
                return False
        return True

    # 処理対象のインデックスを決定
    if skip_processed and start_index is None and end_index is None:
        # 未処理の画像のみを対象とする
        unprocessed_indices = []
        for idx, row in df.iterrows():
            if not is_processed(row):
                unprocessed_indices.append(idx)

        if not unprocessed_indices:
            print("すべての画像が処理済みです。")
            return

        print(f"未処理画像数: {len(unprocessed_indices)}/{len(df)}")
        print(f"最初の未処理画像インデックス: {unprocessed_indices[0]}")

        # 処理対象インデックスのリスト
        target_indices = unprocessed_indices

    else:
        # 手動でインデックス範囲を指定した場合
        if start_index is None:
            start_index = 0
        if end_index is None:
            end_index = len(df)

        target_indices = list(range(start_index, end_index))

        # skip_processedがTrueの場合、範囲内でも処理済みはスキップ
        if skip_processed:
            target_indices = [idx for idx in target_indices
                            if idx < len(df) and not is_processed(df.iloc[idx])]

    print(f"処理対象画像数: {len(target_indices)}")

    # 進捗状況の表示
    processed_count = 0
    skipped_count = len(df) - len(target_indices) if skip_processed else 0
    not_found_count = 0
    error_count = 0
    cuda_error_occurred = False
    memory_cleanup_count = 0

    # エラー詳細の記録用
    error_details = {}
    # 成功したレイヤー数のカウント
    successful_layers = 0

    # 定期的な保存のカウンター
    save_interval = 5  # 5画像ごとに保存（より頻繁に）
    memory_check_interval = 3  # 3画像ごとにメモリチェック

    # メインの処理ループ
    for i, index in enumerate(tqdm(target_indices, desc="Processing images")):
        # 定期的なメモリチェック
        if i % memory_check_interval == 0 and i > 0:
            health, stats = memory_monitor.check_memory_health()

            if health == 'critical':
                print(f"\n⚠️  メモリ使用率がクリティカル ({stats['usage_ratio']*100:.1f}%) です！")
                memory_monitor.display_memory_status("📊 現在")

                # 強制クリーンアップ
                if memory_monitor.cleanup_if_needed(force=True):
                    memory_cleanup_count += 1
                    time.sleep(2)  # GPUのクールダウン

                    # クリーンアップ後の状態を表示
                    memory_monitor.display_memory_status("🔄 クリーンアップ後")

                    # デバッグモードの場合、詳細なテンソル情報も表示
                    if debug_mode:
                        log_cuda_tensors("クリーンアップ後:")

            elif health == 'warning':
                if i % (memory_check_interval * 3) == 0:  # 警告時は頻度を下げて表示
                    memory_monitor.display_memory_status("⚠️  メモリ警告")

                # 自動クリーンアップの実行
                if memory_monitor.cleanup_if_needed():
                    memory_cleanup_count += 1

        row = df.iloc[index]
        img_basename = row["image_basename"]

        if pd.isna(img_basename):
            print(f"Skipping row {index} due to NaN image_basename")
            continue

        img_found = False
        img_path = None

        # 画像ファイルの検索
        for ext in [".jpg", ".jpeg", ".png", ".JPG", ".JPEG", ".PNG", ""]:
            test_path = os.path.join(folder_path, f"{img_basename}{ext}")
            if os.path.exists(test_path):
                img_path = test_path
                img_found = True
                break

        if not img_found:
            test_path = os.path.join(folder_path, img_basename)
            if os.path.exists(test_path):
                img_path = test_path
                img_found = True

        if img_found:
            img = cv2.imread(img_path)

            if img is None:
                print(f"Failed to read image: {img_path}")
                continue

            try:
                # === 改善点: 前処理後、元の画像は即座に削除 ===
                torch_img = model.preprocessing(img[..., ::-1])
                del img  # 元の画像データを削除してメモリ解放

                for layer_name, col_name in all_layers_cols:
                    try:
                        # デバッグモードの場合はdebug=Trueを渡す
                        debug_flag = debug_mode and processed_count < 5

                        # === 改善点: コンテキストマネージャーを使用 ===
                        with YOLOV5GradCAM(model=model, layer_name=layer_name,
                                          img_size=input_size, method="gradcampp",
                                          debug=debug_flag) as saliency_method:

                            masks, logits, [boxes, _, _, _], cls_names = saliency_method(torch_img)

                            if len(masks) > 0 and len(boxes) > 0:
                                mask = masks[0][0]
                                bbox = boxes[0][0]

                                aoi = get_aoi(bbox, [mask], threshold)
                                df.at[index, col_name] = aoi
                                successful_layers += 1

                                # デバッグ用：AOI値が正しく計算されているか確認
                                if processed_count < 5:  # 最初の5画像のみログ出力
                                    print(f"  Layer {layer_name}: AOI = {aoi:.4f}")

                                # === 改善点: マスクとボックスを即座に削除 ===
                                del mask, bbox
                            else:
                                # デバッグ用：マスクまたはボックスが空の場合
                                if processed_count < 5:
                                    print(f"  Layer {layer_name}: No masks or boxes detected")

                            # === 改善点: 結果を即座に削除 ===
                            del masks, logits, boxes, cls_names

                        # with文を抜けた時点で自動的にcleanup()が呼ばれる
                        # 追加のメモリクリーンアップ（各レイヤー処理後）
                        aggressive_memory_cleanup()

                    except Exception as e:
                        # CUDA out of memoryエラーのチェック
                        if "CUDA out of memory" in str(e):
                            print(f"\n\nCUDA out of memory エラーが発生しました！")
                            print(f"エラー詳細: {str(e)}")
                            print(f"現在の画像: {img_basename} (インデックス: {index})")
                            print(f"エラー発生レイヤー: {layer_name}")
                            memory_monitor.display_memory_status("💥 エラー発生時")
                            if debug_mode:
                                log_cuda_tensors("エラー発生時:")
                            cuda_error_occurred = True
                            break
                        else:
                            error_count += 1
                            # エラー詳細を記録
                            error_key = f"{img_basename}_{layer_name}"
                            if error_key not in error_details:
                                error_details[error_key] = str(e)

                            if error_count <= 5:
                                print(f"Error processing image {img_basename} at layer {layer_name}: {str(e)}")

                # CUDA out of memoryエラーが発生したらループを抜ける
                if cuda_error_occurred:
                    break

                # === 改善点: 全レイヤー処理後にtorch_imgを削除 ===
                del torch_img

                # === 最適化: 各画像処理後に完全なメモリクリーンアップ ===
                aggressive_memory_cleanup()

                processed_count += 1

                # 進捗表示の改善（メモリ使用率も表示）
                if processed_count % 10 == 0:
                    stats = memory_monitor.get_memory_stats()
                    print(f"\n📈 進捗: {processed_count}/{len(target_indices)} 画像処理済み "
                          f"(GPU使用率: {stats['usage_ratio']*100:.1f}%)")

                # 定期的な中間保存
                if (i + 1) % save_interval == 0:
                    # デバッグ用：保存前のデータ確認
                    saved_count = sum(1 for idx in target_indices[:i+1]
                                    if idx < len(df) and is_processed(df.iloc[idx]))
                    print(f"\n💾 中間保存: {saved_count}個の画像が完全に処理済み")

                    # メモリ使用量のログ
                    memory_monitor.display_memory_status("💾 保存時")

                    df.to_csv(csv_path, index=False)
                    print(f"✅ 中間保存完了: {i + 1}/{len(target_indices)} 画像処理済み")

                    # === 最適化: より積極的なメモリクリーンアップ ===
                    aggressive_memory_cleanup()

                    # クリーンアップ後のメモリ使用量
                    memory_monitor.display_memory_status("クリーンアップ後:")
                    if debug_mode:
                        log_cuda_tensors("クリーンアップ後:")

            except Exception as e:
                # CUDA out of memoryエラーのチェック
                if "CUDA out of memory" in str(e):
                    print(f"\n\n💥 CUDA out of memory エラーが発生しました！")
                    print(f"エラー詳細: {str(e)}")
                    print(f"現在の画像: {img_basename} (インデックス: {index})")
                    memory_monitor.display_memory_status("💥 エラー発生時")
                    cuda_error_occurred = True
                    break
                else:
                    error_count += 1
                    if error_count <= 5:
                        print(f"Error preprocessing image {img_basename}: {str(e)}")
        else:
            not_found_count += 1
            if not_found_count <= 5:
                print(f"Image not found: {img_basename}")

        # CUDA out of memoryエラーが発生したらループを抜ける
        if cuda_error_occurred:
            break

    # 最終保存（エラーが発生した場合も含む）
    df.to_csv(csv_path, index=False)

    # 最終的なメモリ使用量
    memory_monitor.display_memory_status("🏁 処理終了時")
    if debug_mode:
        log_cuda_tensors("最終状態:")

    if cuda_error_occurred:
        print(f"\n\n========== CUDA OUT OF MEMORY エラーで処理を中止しました ==========")
        print(f"最後に処理しようとした画像: {img_basename} (インデックス: {index})")
        print(f"処理済み画像数: {processed_count}")
        print(f"成功したレイヤー処理: {successful_layers}個")
        print(f"メモリクリーンアップ実行回数: {memory_cleanup_count}回")
        print(f"残り未処理画像数: {len(target_indices) - i}")
        print(f"結果を保存しました: {csv_path}")
        print(f"\n次回実行時は自動的に続きから処理されます。")
    else:
        print(f"\n✨ 処理完了サマリー:")
        print(f"├─ 処理した画像: {processed_count}個")
        print(f"├─ 成功したレイヤー処理: {successful_layers}個")
        print(f"├─ スキップした画像: {skipped_count}個")
        print(f"├─ 見つからなかった画像: {not_found_count}個")
        print(f"├─ エラーが発生した画像: {error_count}個")
        print(f"└─ メモリクリーンアップ実行回数: {memory_cleanup_count}回")

        # エラーの詳細を表示
        if error_details:
            print(f"\n⚠️  エラーの詳細（最初の10件）:")
            for i, (key, error_msg) in enumerate(list(error_details.items())[:10]):
                print(f"  {i+1}. {key}: {error_msg[:100]}...")

        # 最終的な完全処理済み画像数を確認
        fully_processed = sum(1 for idx, row in df.iterrows() if is_processed(row))
        print(f"\n📊 完全に処理済みの画像数: {fully_processed}/{len(df)}")

        print(f"\n💾 結果を保存しました: {csv_path}")

# ========== 実行モード設定 ==========
# 以下の変数を変更することで実行モードを切り替えできます

# MODE = "auto"：未処理画像を自動検出して処理
# MODE = "manual"：start_indexとend_indexで範囲を指定（処理済みはスキップ）
# MODE = "force"：start_indexとend_indexで範囲を指定（処理済みも再処理）
# MODE = "debug"：最初の未処理画像1枚のみ処理（デバッグ用）
MODE = "auto"  # "auto", "manual", "force", "debug" のいずれかを指定

# manualモードまたはforceモードの場合の範囲指定
MANUAL_START_INDEX = 0
MANUAL_END_INDEX = 10

# ========== メイン実行部分 ==========
# デバッグ関数：CSVファイルの状態を確認
def check_csv_status(csv_path, threshold=0.5):
    """CSVファイルの処理状況を確認する関数"""
    df = pd.read_csv(csv_path)

    all_layers_cols = [
        f"AOI_{threshold}_layermodel_17_cv3_conv",
        f"AOI_{threshold}_layermodel_20_cv3_conv",
        f"AOI_{threshold}_layermodel_23_cv3_conv",
        f"AOI_{threshold}_layer24_m_0",
        f"AOI_{threshold}_layer24_m_1",
        f"AOI_{threshold}_layer24_m_2"
    ]

    print(f"CSVファイル: {csv_path}")
    print(f"総行数: {len(df)}")

    # 各カラムの状態を確認
    for col in all_layers_cols:
        if col in df.columns:
            non_null_count = df[col].notna().sum()
            print(f"  {col}: {non_null_count}/{len(df)} ({non_null_count/len(df)*100:.1f}%)")
        else:
            print(f"  {col}: カラムが存在しません")

    # 完全に処理済みの行数
    fully_processed = 0
    for idx, row in df.iterrows():
        if all(pd.notna(row[col]) and row[col] is not None for col in all_layers_cols if col in df.columns):
            fully_processed += 1

    print(f"\n完全に処理済みの画像: {fully_processed}/{len(df)} ({fully_processed/len(df)*100:.1f}%)")

    # 最初の5行のデータを表示
    print("\n最初の5行のAOI値:")
    for idx in range(min(5, len(df))):
        print(f"  行{idx}: ", end="")
        for col in all_layers_cols:
            if col in df.columns:
                val = df.iloc[idx][col]
                if pd.notna(val):
                    print(f"{val:.4f} ", end="")
                else:
                    print("NaN ", end="")
        print()

folder_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/Ueno_Mix1039/Images"
csv_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/Ueno_Mix1039/Ueno_Mix1039_gradcam++.csv"

threshold = 0.5

model_path = "/gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forcresco/weights/eye_nii_2202_onecaseoneimage2_doctorcompare_yolov5s_epoch200_batch16_89.8p/last.pt"
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"使用デバイス: {device}")
if device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")

input_size = (640, 640)
names = ["infection","normal","non-infection","scar","tumor","deposit","APAC","lens opacity","bullous"]

model = YOLOV5TorchObjectDetector(model_path, device, img_size=input_size, names=names)

target_layers = [
    "model_17_cv3_conv",
    "model_20_cv3_conv",
    "model_23_cv3_conv",
    "model_24_m_0",
    "model_24_m_1",
    "model_24_m_2"
]

# バッチ処理モードの設定
USE_BATCH_MODE = False
BATCH_SIZE = 20

if USE_BATCH_MODE:
    print("=== バッチ処理モードで実行します ===")

    # 処理前のCSV状態を確認
    print("\n=== 処理前のCSV状態 ===")
    check_csv_status(csv_path, threshold)
    print("="*50 + "\n")

    # is_processed関数を定義（calculate_aoi関数外でも使えるように）
    def is_processed(row):
        """すべてのAOIカラムに値が入っているかチェック"""
        all_layers_cols = [
            ("model_17_cv3_conv", f"AOI_{threshold}_layermodel_17_cv3_conv"),
            ("model_20_cv3_conv", f"AOI_{threshold}_layermodel_20_cv3_conv"),
            ("model_23_cv3_conv", f"AOI_{threshold}_layermodel_23_cv3_conv"),
            ("model_24_m_0", f"AOI_{threshold}_layer24_m_0"),
            ("model_24_m_1", f"AOI_{threshold}_layer24_m_1"),
            ("model_24_m_2", f"AOI_{threshold}_layer24_m_2")
        ]
        for _, col_name in all_layers_cols:
            if pd.isna(row[col_name]) or row[col_name] is None:
                return False
        return True

    # 未処理画像のインデックスを取得
    df_check = pd.read_csv(csv_path)
    unprocessed_indices = []

    for idx, row in df_check.iterrows():
        if not is_processed(row):
            unprocessed_indices.append(idx)

    if not unprocessed_indices:
        print("すべての画像が処理済みです。")
    else:
        print(f"未処理画像数: {len(unprocessed_indices)}")

        # バッチごとに処理
        for i in range(0, len(unprocessed_indices), BATCH_SIZE):
            batch_indices = unprocessed_indices[i:i+BATCH_SIZE]
            batch_start = batch_indices[0] if batch_indices else 0
            batch_end = batch_indices[-1] + 1 if batch_indices else 0

            print(f"\nバッチ {i//BATCH_SIZE + 1}/{(len(unprocessed_indices)-1)//BATCH_SIZE + 1}: "
                  f"インデックス {batch_indices}")

            # バッチ内の画像をまとめて処理
            calculate_aoi(folder_path, csv_path, threshold, model, target_layers,
                         start_index=None, end_index=None, skip_processed=True, debug_mode=False)

            # 処理完了後のクリーンアップ
            aggressive_memory_cleanup()

            print(f"バッチ {i//BATCH_SIZE + 1} 完了")

            # バッチが完了したら、再度未処理画像を確認（中断された場合の対策）
            df_check = pd.read_csv(csv_path)
            remaining_unprocessed = sum(1 for idx, row in df_check.iterrows() if not is_processed(row))
            print(f"残り未処理画像数: {remaining_unprocessed}")

            if remaining_unprocessed == 0:
                break

    print("\n=== 全バッチ処理完了 ===")

    # 処理後のCSV状態を確認
    print("\n=== 処理後のCSV状態 ===")
    check_csv_status(csv_path, threshold)

else:
    print("=== 通常モードで実行します ===")
    print(f"実行モード: {MODE}")

    # 処理前のCSV状態を確認
    print("\n=== 処理前のCSV状態 ===")
    check_csv_status(csv_path, threshold)
    print("="*50 + "\n")

    if MODE == "auto":
        # 自動モード（未処理画像を検出）
        print("\n未処理画像を自動検出して処理します...")
        calculate_aoi(folder_path, csv_path, threshold, model, target_layers,
                     skip_processed=True, debug_mode=False)

    elif MODE == "manual":
        # 手動モード（範囲指定、処理済みはスキップ）
        print(f"\nインデックス {MANUAL_START_INDEX} から {MANUAL_END_INDEX-1} まで処理します（処理済みはスキップ）...")
        calculate_aoi(folder_path, csv_path, threshold, model, target_layers,
                     start_index=MANUAL_START_INDEX, end_index=MANUAL_END_INDEX,
                     skip_processed=True, debug_mode=False)

    elif MODE == "force":
        # 強制再処理モード（範囲指定、処理済みも再処理）
        print(f"\nインデックス {MANUAL_START_INDEX} から {MANUAL_END_INDEX-1} まで強制的に再処理します...")
        calculate_aoi(folder_path, csv_path, threshold, model, target_layers,
                     start_index=MANUAL_START_INDEX, end_index=MANUAL_END_INDEX,
                     skip_processed=False, debug_mode=False)

    elif MODE == "debug":
        # デバッグモード（最初の未処理画像1枚のみ処理）
        print("\nデバッグモード: 最初の未処理画像1枚のみ処理します...")
        # 未処理画像を探す
        df_debug = pd.read_csv(csv_path)
        debug_idx = None
        for idx, row in df_debug.iterrows():
            all_layers_cols = [
                f"AOI_{threshold}_layermodel_17_cv3_conv",
                f"AOI_{threshold}_layermodel_20_cv3_conv",
                f"AOI_{threshold}_layermodel_23_cv3_conv",
                f"AOI_{threshold}_layer24_m_0",
                f"AOI_{threshold}_layer24_m_1",
                f"AOI_{threshold}_layer24_m_2"
            ]
            if any(pd.isna(row[col]) or row[col] is None for col in all_layers_cols if col in df_debug.columns):
                debug_idx = idx
                break

        if debug_idx is not None:
            print(f"デバッグ対象: インデックス {debug_idx}, 画像: {df_debug.iloc[debug_idx]['image_basename']}")
            calculate_aoi(folder_path, csv_path, threshold, model, target_layers,
                         start_index=debug_idx, end_index=debug_idx+1,
                         skip_processed=False, debug_mode=True)
        else:
            print("未処理画像が見つかりませんでした。")

    else:
        print(f"無効なモード: {MODE}")
        print("MODE は 'auto', 'manual', 'force', 'debug' のいずれかを指定してください。")

    # 処理後のCSV状態を確認
    print("\n=== 処理後のCSV状態 ===")
    check_csv_status(csv_path, threshold)

# === 最適化: 最終的なメモリクリーンアップ ===
print("\n=== 最終クリーンアップ ===")
if device == "cuda":
    torch.cuda.synchronize()
    torch.cuda.empty_cache()

    # 最終メモリ状態の表示
    memory_monitor = GPUMemoryMonitor()
    memory_monitor.display_memory_status("最終クリーンアップ後:")

    if MODE == "debug":
        log_cuda_tensors("最終クリーンアップ後:")

# モデルの削除
del model
gc.collect()

print("\n処理が完了しました。")

### VersionによるAOIの差を確認 (結論：変わりなし)

In [ ]:
import os
import pandas as pd
import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import gc
import torchvision
import warnings
import matplotlib.pyplot as plt
from tqdm import tqdm
from deep_utils.utils.box_utils.boxes import Box

# 警告の抑制
warnings.filterwarnings("ignore", category=FutureWarning,
                      message=".*Using a non-full backward hook.*")

# YOLOv5の必要な関数をインポート
from utils.general import non_max_suppression as yolo_nms, xywh2xyxy, box_iou
from utils.datasets import letterbox
from models.experimental import attempt_load

# YOLOv5のカスタムクラスをインポート
from models.yolo import Model, Detect
from models.common import Conv, Bottleneck, C3, SPPF, Concat

# シリアライゼーションの設定
from torch.nn import Sequential, Conv2d, BatchNorm2d, SiLU, MaxPool2d, Upsample, ModuleList
torch.serialization.add_safe_globals([
    Model, Detect, Sequential, ModuleList, Conv, Bottleneck, C3, SPPF, Concat,
    Conv2d, BatchNorm2d, SiLU, MaxPool2d, Upsample
])

# グローバル変数
global img_size
img_size = 640

# ========== ヘルパー関数 ==========
def find_yolo_layer(model, layer_name):
    """YOLOモデルから特定のレイヤーを見つける"""
    hierarchy = layer_name.split("_")
    target_layer = model.model._modules[hierarchy[0]]

    for h in hierarchy[1:]:
        target_layer = target_layer._modules[h]
    return target_layer

def set_model_gradients(model, layer_name, enable=True):
    """特定のレイヤーのみ勾配計算を有効化"""
    # まず全体の勾配を無効化
    for param in model.model.parameters():
        param.requires_grad = False

    # 特定のレイヤーのみ有効化
    if enable:
        target_layer = find_yolo_layer(model, layer_name)
        for param in target_layer.parameters():
            param.requires_grad = True

def get_aoi(bbox, masks, threshold):
    """Area of Interest (AOI)を計算"""
    total_intersect_pixels = 0

    with torch.no_grad():
        for mask in masks:
            # マスクをCPUで処理
            mask_cpu = mask.squeeze().mul(255).add_(0.5).clamp_(0, 255).detach().cpu().numpy().astype(np.uint8)
            del mask

            mask_cpu = np.nan_to_num(mask_cpu, nan=0.0).astype(np.uint8)

            # 閾値処理
            binary_mask = mask_cpu >= (threshold * 255)
            del mask_cpu

            x1, y1, x2, y2 = bbox
            mask_bbox = binary_mask[y1:y2, x1:x2]

            intersect_pixels = np.sum(mask_bbox)
            total_intersect_pixels += intersect_pixels

            del mask_bbox, binary_mask

    mask_bbox_area = (y2 - y1) * (x2 - x1)
    AOI = total_intersect_pixels / mask_bbox_area if mask_bbox_area > 0 else 0

    return AOI

# ========== YOLOv5物体検出器クラス ==========
class YOLOV5TorchObjectDetector(nn.Module):
    def __init__(self,
                 model_weight,
                 device,
                 img_size,
                 names=None,
                 mode="eval",
                 confidence=0.25,
                 iou_thresh=0.45,
                 agnostic_nms=False):
        super(YOLOV5TorchObjectDetector, self).__init__()
        self.device = device
        self.model = None
        self.img_size = img_size
        self.mode = mode
        self.confidence = confidence
        self.iou_thresh = iou_thresh
        self.agnostic = agnostic_nms

        self.model = attempt_load(model_weight, device=device)
        print("[INFO] Model is loaded")

        # デフォルトでは勾配計算を無効化
        for param in self.model.parameters():
            param.requires_grad = False
        self.model.to(device)

        if self.mode == "train":
            self.model.train()
        else:
            self.model.eval()

        if names is None:
            self.names = ["person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat",
                          "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
                          "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack",
                          "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball",
                          "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket",
                          "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple",
                          "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake",
                          "chair", "couch", "potted plant", "bed", "dining table", "toilet", "tv", "laptop",
                          "mouse", "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink",
                          "refrigerator", "book", "clock", "vase", "scissors", "teddy bear", "hair drier",
                          "toothbrush"]
        else:
            self.names = names

    @staticmethod
    def non_max_suppression(prediction, logits, conf_thres=0.25, iou_thres=0.45, classes=None, agnostic=False,
                            multi_label=False, labels=(), max_det=300):
        nc = prediction.shape[2] - 5
        xc = prediction[..., 4] > conf_thres

        assert 0 <= conf_thres <= 1, f"Invalid Confidence threshold {conf_thres}"
        assert 0 <= iou_thres <= 1, f"Invalid IoU {iou_thres}"

        min_wh, max_wh = 2, 4096
        max_nms = 30000
        time_limit = 10.0
        redundant = True
        multi_label &= nc > 1
        merge = False

        import time
        t = time.time()
        output = [torch.zeros((0, 6), device=prediction.device)] * prediction.shape[0]
        logits_output = [torch.zeros((0, 80), device=logits.device)] * logits.shape[0]

        for xi, (x, log_) in enumerate(zip(prediction, logits)):
            x = x[xc[xi]]
            log_ = log_[xc[xi]]

            if labels and len(labels[xi]):
                l = labels[xi]
                v = torch.zeros((len(l), nc + 5), device=x.device)
                v[:, :4] = l[:, 1:5]
                v[:, 4] = 1.0
                v[range(len(l)), l[:, 0].long() + 5] = 1.0
                x = torch.cat((x, v), 0)

            if not x.shape[0]:
                continue

            x[:, 5:] *= x[:, 4:5]
            box = xywh2xyxy(x[:, :4])

            if multi_label:
                i, j = (x[:, 5:] > conf_thres).nonzero(as_tuple=False).T
                x = torch.cat((box[i], x[i, j + 5, None], j[:, None].float()), 1)
            else:
                conf, j = x[:, 5:].max(1, keepdim=True)
                x = torch.cat((box, conf, j.float()), 1)[conf.view(-1) > conf_thres]
                log_ = log_[conf.view(-1) > conf_thres]

            if classes is not None:
                x = x[(x[:, 5:6] == torch.tensor(classes, device=x.device)).any(1)]

            n = x.shape[0]
            if not n:
                continue
            elif n > max_nms:
                x = x[x[:, 4].argsort(descending=True)[:max_nms]]

            c = x[:, 5:6] * (0 if agnostic else max_wh)
            boxes, scores = x[:, :4] + c, x[:, 4]
            i = torchvision.ops.nms(boxes, scores, iou_thres)

            if i.shape[0] > max_det:
                i = i[:max_det]

            if merge and (1 < n < 3E3):
                iou = box_iou(boxes[i], boxes) > iou_thres
                weights = iou * scores[None]
                x[i, :4] = torch.mm(weights, x[:, :4]).float() / weights.sum(1, keepdim=True)
                if redundant:
                    i = i[iou.sum(1) > 1]

            output[xi] = x[i]
            logits_output[xi] = log_[i]
            assert log_[i].shape[0] == x[i].shape[0]

            if (time.time() - t) > time_limit:
                print(f"WARNING: NMS time limit {time_limit}s exceeded")
                break

        return output, logits_output

    @staticmethod
    def yolo_resize(img, new_shape=(640, 640), color=(114, 114, 114), auto=True, scaleFill=False, scaleup=True):
        return letterbox(img, new_shape=new_shape, color=color, auto=auto, scaleFill=scaleFill, scaleup=scaleup)

    def forward(self, img):
        prediction, logits, _ = self.model(img, augment=False)
        prediction, logits = self.non_max_suppression(prediction, logits, self.confidence, self.iou_thresh,
                                                      classes=None, agnostic=self.agnostic)

        with torch.no_grad():
            self.boxes, self.class_names, self.classes, self.confidences = [[[] for _ in range(img.shape[0])] for _ in range(4)]

            for i, det in enumerate(prediction):
                if len(det):
                    det_cpu = det.cpu()
                    del det

                    for *xyxy, conf, cls in det_cpu:
                        xyxy[0] = max(0, xyxy[0])
                        xyxy[1] = max(0, xyxy[1])
                        xyxy[2] = min(img_size, xyxy[2])
                        xyxy[3] = min(img_size, xyxy[3])

                        bbox = Box.box2box(xyxy,
                                           in_source=Box.BoxSource.Torch,
                                           to_source=Box.BoxSource.Numpy,
                                           return_int=True)
                        self.boxes[i].append(bbox)
                        self.confidences[i].append(round(conf.item(), 2))
                        cls = int(cls.item())
                        self.classes[i].append(cls)
                        if self.names is not None:
                            self.class_names[i].append(self.names[cls])
                        else:
                            self.class_names[i].append(cls)

            del prediction

        return [self.boxes, self.classes, self.class_names, self.confidences], logits

    def preprocessing(self, img):
        if len(img.shape) != 4:
            img = np.expand_dims(img, axis=0)
        im0 = img.astype(np.uint8)
        del img

        resized_imgs = []
        for im in im0:
            resized_img = self.yolo_resize(im, new_shape=self.img_size)[0]
            resized_imgs.append(resized_img)

        img = np.array(resized_imgs)
        del resized_imgs, im0

        img = img.transpose((0, 3, 1, 2))
        img = np.ascontiguousarray(img)
        img = torch.from_numpy(img).to(self.device)
        img = img / 255.0
        return img

# ========== バージョン1: 完全初期版（全体勾配 + 1回フォワード + retain_graph=True） ==========
class YOLOV5GradCAM_V1_Original:
    """完全初期版：全体勾配有効化 + 1回のフォワードパス + retain_graph=True"""

    def __init__(self, model, layer_name, img_size=(640, 640), method="gradcam"):
        self.model = model
        self.gradients = dict()
        self.activations = dict()
        self.method = method
        self.cls_names = []
        self.layer_name = layer_name

        self.forward_handle = None
        self.backward_handle = None
        self.target_layer = None

        def backward_hook(module, grad_input, grad_output):
            self.gradients["value"] = grad_output[0].detach().clone()
            return None

        def forward_hook(module, input, output):
            self.activations["value"] = output.detach().clone()
            return None

        self.target_layer = find_yolo_layer(self.model, layer_name)

        # 全体の勾配を有効化
        for param in self.model.model.parameters():
            param.requires_grad = True

        self.forward_handle = self.target_layer.register_forward_hook(forward_hook)
        self.backward_handle = self.target_layer.register_backward_hook(backward_hook)

    def cleanup(self):
        if self.forward_handle is not None:
            self.forward_handle.remove()
            self.forward_handle = None

        if self.backward_handle is not None:
            self.backward_handle.remove()
            self.backward_handle = None

        self.gradients.clear()
        self.activations.clear()
        self.target_layer = None

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.cleanup()
        return False

    def forward(self, input_img, class_idx=True):
        saliency_maps = []
        b, c, h, w = input_img.size()

        # 1回のフォワードパス
        preds, logits = self.model(input_img)

        with torch.no_grad():
            _, top3_indices = torch.topk(logits[0], k=3)

            if top3_indices.numel() > 0:
                preds[1][0] = top3_indices.tolist()[0]
                preds[2][0] = [self.model.names[i] for i in preds[1][0]]
                self.cls_names = preds[2][0]
            else:
                self.cls_names = []

            del top3_indices

        # 各クラスに対して同じlogitsを使用
        for i, (cls, cls_name) in enumerate(zip(preds[1][0], preds[2][0])):
            self.model.zero_grad()

            if class_idx:
                score = logits[0][0][cls]
            else:
                score = logits[0][0].max()

            # retain_graph=Trueを使用（最後のクラス以外）
            if i < len(preds[1][0]) - 1:
                score.backward(retain_graph=True)
            else:
                score.backward(retain_graph=False)

            if "value" not in self.gradients or "value" not in self.activations:
                print(f"[WARNING] No gradients or activations for class {cls_name}")
                continue

            gradients = self.gradients["value"]
            activations = self.activations["value"]
            b, k, u, v = gradients.size()

            if self.method == "gradcam":
                weights = self._gradcam_weights(gradients, b, k)
            elif self.method == "gradcampp":
                weights = self._gradcampp_weights(gradients, activations, score, b, k, u, v)

            with torch.no_grad():
                saliency_map = (weights * activations).sum(1, keepdim=True)
                del weights

                saliency_map = F.relu(saliency_map)
                saliency_map = F.interpolate(saliency_map, size=(h, w), mode="bilinear", align_corners=False)
                saliency_map_min, saliency_map_max = saliency_map.min(), saliency_map.max()
                saliency_map = (saliency_map - saliency_map_min).div(saliency_map_max - saliency_map_min + 1e-8).data

                del saliency_map_min, saliency_map_max

            saliency_maps.append(saliency_map)

        return saliency_maps, logits, preds, self.cls_names

    def _gradcam_weights(self, gradients, b, k):
        alpha = gradients.view(b, k, -1).mean(2)
        weights = alpha.view(b, k, 1, 1)
        del alpha
        return weights

    def _gradcampp_weights(self, gradients, activations, score, b, k, u, v):
        alpha_num = gradients.pow(2)
        alpha_denom = gradients.pow(2).mul(2) + \
            activations.mul(gradients.pow(3)).view(b, k, u*v).sum(-1, keepdim=True).view(b, k, 1, 1)
        alpha_denom = torch.where(alpha_denom != 0.0, alpha_denom, torch.ones_like(alpha_denom))
        alpha = alpha_num.div(alpha_denom + 1e-7)
        del alpha_num, alpha_denom

        relu_grad = F.relu(score.exp() * gradients)
        weights = (relu_grad * alpha).view(b, k, u*v).sum(-1).view(b, k, 1, 1)
        del relu_grad, alpha

        return weights

    def __call__(self, input_img):
        return self.forward(input_img)

# ========== バージョン2: 全体勾配 + 各クラスフォワード + retain_graph=False ==========
class YOLOV5GradCAM_V2_AllGrad:
    """全体勾配有効化 + 各クラスごとに新しいフォワードパス + retain_graph=False"""

    def __init__(self, model, layer_name, img_size=(640, 640), method="gradcam"):
        self.model = model
        self.gradients = dict()
        self.activations = dict()
        self.method = method
        self.cls_names = []
        self.layer_name = layer_name

        self.forward_handle = None
        self.backward_handle = None
        self.target_layer = None

        def backward_hook(module, grad_input, grad_output):
            self.gradients["value"] = grad_output[0].detach().clone()
            return None

        def forward_hook(module, input, output):
            self.activations["value"] = output.detach().clone()
            return None

        self.target_layer = find_yolo_layer(self.model, layer_name)

        # 全体の勾配を有効化
        for param in self.model.model.parameters():
            param.requires_grad = True

        self.forward_handle = self.target_layer.register_forward_hook(forward_hook)
        self.backward_handle = self.target_layer.register_backward_hook(backward_hook)

    def cleanup(self):
        if self.forward_handle is not None:
            self.forward_handle.remove()
            self.forward_handle = None

        if self.backward_handle is not None:
            self.backward_handle.remove()
            self.backward_handle = None

        self.gradients.clear()
        self.activations.clear()
        self.target_layer = None

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.cleanup()
        return False

    def forward(self, input_img, class_idx=True):
        saliency_maps = []
        b, c, h, w = input_img.size()

        # 最初のフォワードパスでtop3を取得
        preds, logits = self.model(input_img)

        with torch.no_grad():
            _, top3_indices = torch.topk(logits[0], k=3)

            if top3_indices.numel() > 0:
                preds[1][0] = top3_indices.tolist()[0]
                preds[2][0] = [self.model.names[i] for i in preds[1][0]]
                self.cls_names = preds[2][0]
            else:
                self.cls_names = []

            del top3_indices

        # 各クラスごとに新しいフォワードパス
        for cls, cls_name in zip(preds[1][0], preds[2][0]):
            self.model.zero_grad()

            input_img_clone = input_img.clone()
            _, new_logits = self.model(input_img_clone)
            del input_img_clone

            if class_idx:
                score = new_logits[0][0][cls]
            else:
                score = new_logits[0][0].max()

            # retain_graph=False
            score.backward()

            if "value" not in self.gradients or "value" not in self.activations:
                print(f"[WARNING] No gradients or activations for class {cls_name}")
                del score, new_logits
                continue

            gradients = self.gradients["value"]
            activations = self.activations["value"]
            b, k, u, v = gradients.size()

            if self.method == "gradcam":
                weights = self._gradcam_weights(gradients, b, k)
            elif self.method == "gradcampp":
                weights = self._gradcampp_weights(gradients, activations, score, b, k, u, v)

            del score, new_logits

            with torch.no_grad():
                saliency_map = (weights * activations).sum(1, keepdim=True)
                del weights

                saliency_map = F.relu(saliency_map)
                saliency_map = F.interpolate(saliency_map, size=(h, w), mode="bilinear", align_corners=False)
                saliency_map_min, saliency_map_max = saliency_map.min(), saliency_map.max()
                saliency_map = (saliency_map - saliency_map_min).div(saliency_map_max - saliency_map_min + 1e-8).data

                del saliency_map_min, saliency_map_max

            saliency_maps.append(saliency_map)

            # メモリクリーンアップ
            if "value" in self.gradients:
                del self.gradients["value"]
            if "value" in self.activations:
                del self.activations["value"]

            del gradients, activations

            if torch.cuda.is_available():
                torch.cuda.empty_cache()

        return saliency_maps, logits, preds, self.cls_names

    def _gradcam_weights(self, gradients, b, k):
        alpha = gradients.view(b, k, -1).mean(2)
        weights = alpha.view(b, k, 1, 1)
        del alpha
        return weights

    def _gradcampp_weights(self, gradients, activations, score, b, k, u, v):
        alpha_num = gradients.pow(2)
        alpha_denom = gradients.pow(2).mul(2) + \
            activations.mul(gradients.pow(3)).view(b, k, u*v).sum(-1, keepdim=True).view(b, k, 1, 1)
        alpha_denom = torch.where(alpha_denom != 0.0, alpha_denom, torch.ones_like(alpha_denom))
        alpha = alpha_num.div(alpha_denom + 1e-7)
        del alpha_num, alpha_denom

        relu_grad = F.relu(score.exp() * gradients)
        weights = (relu_grad * alpha).view(b, k, u*v).sum(-1).view(b, k, 1, 1)
        del relu_grad, alpha

        return weights

    def __call__(self, input_img):
        return self.forward(input_img)

# ========== バージョン3: 選択的勾配 + 各クラスフォワード + retain_graph=False ==========
class YOLOV5GradCAM_V3_Optimized:
    """選択的勾配有効化 + 各クラスごとに新しいフォワードパス + retain_graph=False"""

    def __init__(self, model, layer_name, img_size=(640, 640), method="gradcam"):
        self.model = model
        self.gradients = dict()
        self.activations = dict()
        self.method = method
        self.cls_names = []
        self.layer_name = layer_name

        self.forward_handle = None
        self.backward_handle = None
        self.target_layer = None

        def backward_hook(module, grad_input, grad_output):
            self.gradients["value"] = grad_output[0].detach().clone()
            return None

        def forward_hook(module, input, output):
            self.activations["value"] = output.detach().clone()
            return None

        self.target_layer = find_yolo_layer(self.model, layer_name)

        # 選択的勾配有効化
        set_model_gradients(self.model, layer_name, enable=True)

        self.forward_handle = self.target_layer.register_forward_hook(forward_hook)
        self.backward_handle = self.target_layer.register_backward_hook(backward_hook)

    def cleanup(self):
        if self.forward_handle is not None:
            self.forward_handle.remove()
            self.forward_handle = None

        if self.backward_handle is not None:
            self.backward_handle.remove()
            self.backward_handle = None

        self.gradients.clear()
        self.activations.clear()

        # レイヤーの勾配計算を無効化
        set_model_gradients(self.model, self.layer_name, enable=False)

        self.target_layer = None

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.cleanup()
        return False

    def forward(self, input_img, class_idx=True):
        saliency_maps = []
        b, c, h, w = input_img.size()

        # 最初のフォワードパスでtop3を取得
        preds, logits = self.model(input_img)

        with torch.no_grad():
            _, top3_indices = torch.topk(logits[0], k=3)

            if top3_indices.numel() > 0:
                preds[1][0] = top3_indices.tolist()[0]
                preds[2][0] = [self.model.names[i] for i in preds[1][0]]
                self.cls_names = preds[2][0]
            else:
                self.cls_names = []

            del top3_indices

        # 各クラスごとに新しいフォワードパス
        for cls, cls_name in zip(preds[1][0], preds[2][0]):
            self.model.zero_grad()

            input_img_clone = input_img.clone()
            _, new_logits = self.model(input_img_clone)
            del input_img_clone

            if class_idx:
                score = new_logits[0][0][cls]
            else:
                score = new_logits[0][0].max()

            # retain_graph=False
            score.backward()

            if "value" not in self.gradients or "value" not in self.activations:
                print(f"[WARNING] No gradients or activations for class {cls_name}")
                del score, new_logits
                continue

            gradients = self.gradients["value"]
            activations = self.activations["value"]
            b, k, u, v = gradients.size()

            if self.method == "gradcam":
                weights = self._gradcam_weights(gradients, b, k)
            elif self.method == "gradcampp":
                weights = self._gradcampp_weights(gradients, activations, score, b, k, u, v)

            del score, new_logits

            with torch.no_grad():
                saliency_map = (weights * activations).sum(1, keepdim=True)
                del weights

                saliency_map = F.relu(saliency_map)
                saliency_map = F.interpolate(saliency_map, size=(h, w), mode="bilinear", align_corners=False)
                saliency_map_min, saliency_map_max = saliency_map.min(), saliency_map.max()
                saliency_map = (saliency_map - saliency_map_min).div(saliency_map_max - saliency_map_min + 1e-8).data

                del saliency_map_min, saliency_map_max

            saliency_maps.append(saliency_map)

            # メモリクリーンアップ
            if "value" in self.gradients:
                del self.gradients["value"]
            if "value" in self.activations:
                del self.activations["value"]

            del gradients, activations

            if torch.cuda.is_available():
                torch.cuda.empty_cache()

        return saliency_maps, logits, preds, self.cls_names

    def _gradcam_weights(self, gradients, b, k):
        alpha = gradients.view(b, k, -1).mean(2)
        weights = alpha.view(b, k, 1, 1)
        del alpha
        return weights

    def _gradcampp_weights(self, gradients, activations, score, b, k, u, v):
        alpha_num = gradients.pow(2)
        alpha_denom = gradients.pow(2).mul(2) + \
            activations.mul(gradients.pow(3)).view(b, k, u*v).sum(-1, keepdim=True).view(b, k, 1, 1)
        alpha_denom = torch.where(alpha_denom != 0.0, alpha_denom, torch.ones_like(alpha_denom))
        alpha = alpha_num.div(alpha_denom + 1e-7)
        del alpha_num, alpha_denom

        relu_grad = F.relu(score.exp() * gradients)
        weights = (relu_grad * alpha).view(b, k, u*v).sum(-1).view(b, k, 1, 1)
        del relu_grad, alpha

        return weights

    def __call__(self, input_img):
        return self.forward(input_img)

# ========== 3つのバージョンを比較する関数 ==========
def compare_all_versions(image_path, model, layer_names, threshold=0.5):
    """3つのバージョンを全て比較"""

    print(f"\n画像を読み込み中: {image_path}")
    img = cv2.imread(image_path)
    if img is None:
        print(f"Failed to read image: {image_path}")
        return None

    print(f"画像サイズ: {img.shape}")

    # 前処理
    torch_img = model.preprocessing(img[..., ::-1])
    print(f"前処理後のテンソルサイズ: {torch_img.shape}")

    # まず物体検出を確認
    with torch.no_grad():
        [boxes_test, _, _, _], _ = model(torch_img.clone())
        if len(boxes_test[0]) == 0:
            print("物体が検出されませんでした。")
            return None
        print(f"検出された物体数: {len(boxes_test[0])}")
        print(f"最初の物体のバウンディングボックス: {boxes_test[0][0]}")

    # 結果を格納する辞書
    results = {
        'layer_name': [],
        'aoi_v1_original': [],  # 完全初期版
        'aoi_v2_allgrad': [],   # 全体勾配 + 各クラスフォワード
        'aoi_v3_optimized': [], # 選択的勾配 + 各クラスフォワード
        'diff_v1_v2': [],       # V1とV2の差
        'diff_v2_v3': [],       # V2とV3の差
        'diff_v1_v3': []        # V1とV3の差
    }

    print(f"\n=== 画像: {os.path.basename(image_path)} の比較 ===")
    print("-" * 120)
    print(f"{'レイヤー名':<25} {'V1:完全初期版':>15} {'V2:全体勾配':>15} {'V3:選択的勾配':>15} {'V1-V2差':>12} {'V2-V3差':>12} {'V1-V3差':>12}")
    print("-" * 120)

    for layer_name in layer_names:
        print(f"\n処理中のレイヤー: {layer_name}")

        # === V1: 完全初期版 ===
        print("  V1: 完全初期版（全体勾配 + 1回フォワード + retain_graph=True）...")
        try:
            with YOLOV5GradCAM_V1_Original(model=model, layer_name=layer_name,
                                          img_size=(640, 640), method="gradcampp") as gradcam_v1:
                masks_v1, _, preds_v1, _ = gradcam_v1(torch_img.clone())
                boxes_v1 = preds_v1[0]

                if len(masks_v1) > 0 and len(boxes_v1) > 0 and len(boxes_v1[0]) > 0:
                    mask_v1 = masks_v1[0]
                    bbox_v1 = boxes_v1[0][0]
                    aoi_v1 = get_aoi(bbox_v1, [mask_v1], threshold)
                    print(f"    AOI値: {aoi_v1}")
                else:
                    aoi_v1 = None
                    print("    検出なし")
        except Exception as e:
            print(f"    エラー: {str(e)}")
            aoi_v1 = None

        # === V2: 全体勾配 + 各クラスフォワード ===
        print("  V2: 全体勾配 + 各クラスフォワード + retain_graph=False...")
        try:
            with YOLOV5GradCAM_V2_AllGrad(model=model, layer_name=layer_name,
                                          img_size=(640, 640), method="gradcampp") as gradcam_v2:
                masks_v2, _, preds_v2, _ = gradcam_v2(torch_img.clone())
                boxes_v2 = preds_v2[0]

                if len(masks_v2) > 0 and len(boxes_v2) > 0 and len(boxes_v2[0]) > 0:
                    mask_v2 = masks_v2[0]
                    bbox_v2 = boxes_v2[0][0]
                    aoi_v2 = get_aoi(bbox_v2, [mask_v2], threshold)
                    print(f"    AOI値: {aoi_v2}")
                else:
                    aoi_v2 = None
                    print("    検出なし")
        except Exception as e:
            print(f"    エラー: {str(e)}")
            aoi_v2 = None

        # === V3: 選択的勾配 + 各クラスフォワード ===
        print("  V3: 選択的勾配 + 各クラスフォワード + retain_graph=False...")
        try:
            with YOLOV5GradCAM_V3_Optimized(model=model, layer_name=layer_name,
                                            img_size=(640, 640), method="gradcampp") as gradcam_v3:
                masks_v3, _, preds_v3, _ = gradcam_v3(torch_img.clone())
                boxes_v3 = preds_v3[0]

                if len(masks_v3) > 0 and len(boxes_v3) > 0 and len(boxes_v3[0]) > 0:
                    mask_v3 = masks_v3[0]
                    bbox_v3 = boxes_v3[0][0]
                    aoi_v3 = get_aoi(bbox_v3, [mask_v3], threshold)
                    print(f"    AOI値: {aoi_v3}")
                else:
                    aoi_v3 = None
                    print("    検出なし")
        except Exception as e:
            print(f"    エラー: {str(e)}")
            aoi_v3 = None

        # 結果の記録と表示
        if aoi_v1 is not None and aoi_v2 is not None and aoi_v3 is not None:
            diff_v1_v2 = aoi_v2 - aoi_v1
            diff_v2_v3 = aoi_v3 - aoi_v2
            diff_v1_v3 = aoi_v3 - aoi_v1

            results['layer_name'].append(layer_name)
            results['aoi_v1_original'].append(aoi_v1)
            results['aoi_v2_allgrad'].append(aoi_v2)
            results['aoi_v3_optimized'].append(aoi_v3)
            results['diff_v1_v2'].append(diff_v1_v2)
            results['diff_v2_v3'].append(diff_v2_v3)
            results['diff_v1_v3'].append(diff_v1_v3)

            print(f"{layer_name:<25} {aoi_v1:>15.6f} {aoi_v2:>15.6f} {aoi_v3:>15.6f} "
                  f"{diff_v1_v2:>+12.6f} {diff_v2_v3:>+12.6f} {diff_v1_v3:>+12.6f}")
        else:
            v1_str = f"{aoi_v1:.6f}" if aoi_v1 is not None else "検出なし"
            v2_str = f"{aoi_v2:.6f}" if aoi_v2 is not None else "検出なし"
            v3_str = f"{aoi_v3:.6f}" if aoi_v3 is not None else "検出なし"
            print(f"{layer_name:<25} {v1_str:>15} {v2_str:>15} {v3_str:>15} {'N/A':>12} {'N/A':>12} {'N/A':>12}")

        # メモリクリーンアップ
        torch.cuda.empty_cache()
        gc.collect()

    print("-" * 120)

    # 統計情報の表示
    if results['diff_v1_v2']:
        print(f"\n統計情報:")
        print(f"  V1→V2の平均差分: {np.mean(results['diff_v1_v2']):+.6f}")
        print(f"  V2→V3の平均差分: {np.mean(results['diff_v2_v3']):+.6f}")
        print(f"  V1→V3の平均差分: {np.mean(results['diff_v1_v3']):+.6f}")

        # 視覚化
        if len(results['layer_name']) > 0:
            plt.figure(figsize=(15, 10))

            # AOI値の比較
            plt.subplot(2, 2, 1)
            x = np.arange(len(results['layer_name']))
            width = 0.25

            plt.bar(x - width, results['aoi_v1_original'], width, label='V1:完全初期版', alpha=0.8)
            plt.bar(x, results['aoi_v2_allgrad'], width, label='V2:全体勾配', alpha=0.8)
            plt.bar(x + width, results['aoi_v3_optimized'], width, label='V3:選択的勾配', alpha=0.8)

            plt.xlabel('レイヤー')
            plt.ylabel('AOI値')
            plt.title('各バージョンのAOI値比較')
            plt.xticks(x, [name.replace('model_', '') for name in results['layer_name']], rotation=45)
            plt.legend()
            plt.grid(True, alpha=0.3)

            # 差分の比較
            plt.subplot(2, 2, 2)
            plt.plot(x, results['diff_v1_v2'], 'o-', label='V1→V2', alpha=0.8)
            plt.plot(x, results['diff_v2_v3'], 's-', label='V2→V3', alpha=0.8)
            plt.plot(x, results['diff_v1_v3'], '^-', label='V1→V3', alpha=0.8)

            plt.xlabel('レイヤー')
            plt.ylabel('差分')
            plt.title('バージョン間の差分')
            plt.xticks(x, [name.replace('model_', '') for name in results['layer_name']], rotation=45)
            plt.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
            plt.legend()
            plt.grid(True, alpha=0.3)

            # 相対的な差分（%）
            plt.subplot(2, 2, 3)
            rel_diff_v1_v2 = [(d/v1)*100 if v1 != 0 else 0 for d, v1 in zip(results['diff_v1_v2'], results['aoi_v1_original'])]
            rel_diff_v2_v3 = [(d/v2)*100 if v2 != 0 else 0 for d, v2 in zip(results['diff_v2_v3'], results['aoi_v2_allgrad'])]
            rel_diff_v1_v3 = [(d/v1)*100 if v1 != 0 else 0 for d, v1 in zip(results['diff_v1_v3'], results['aoi_v1_original'])]

            plt.bar(x - width/2, rel_diff_v1_v2, width, label='V1→V2', alpha=0.8)
            plt.bar(x + width/2, rel_diff_v1_v3, width, label='V1→V3', alpha=0.8)

            plt.xlabel('レイヤー')
            plt.ylabel('相対差分 (%)')
            plt.title('V1からの相対的な変化')
            plt.xticks(x, [name.replace('model_', '') for name in results['layer_name']], rotation=45)
            plt.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
            plt.legend()
            plt.grid(True, alpha=0.3)

            # ヒートマップ
            plt.subplot(2, 2, 4)
            data = np.array([results['aoi_v1_original'],
                            results['aoi_v2_allgrad'],
                            results['aoi_v3_optimized']])
            im = plt.imshow(data, aspect='auto', cmap='viridis')
            plt.colorbar(im, label='AOI値')
            plt.yticks([0, 1, 2], ['V1:完全初期版', 'V2:全体勾配', 'V3:選択的勾配'])
            plt.xticks(range(len(results['layer_name'])),
                      [name.replace('model_', '') for name in results['layer_name']], rotation=45)
            plt.title('AOI値のヒートマップ')

            plt.tight_layout()
            plt.savefig('gradcam_version_comparison.png', dpi=300, bbox_inches='tight')
            print("\n比較グラフを 'gradcam_version_comparison.png' として保存しました。")
    else:
        print("\n有効な結果が得られませんでした。")

    # 結果をDataFrameとして返す
    return pd.DataFrame(results)

# ========== メイン実行部分 ==========
if __name__ == "__main__":
    # 作業ディレクトリの設定
    os.chdir("/content/yolov5-gradcam")

    # パスの設定
    folder_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/Ueno_Mix1039/Images"
    csv_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/Ueno_Mix1039/Ueno_Mix1039_gradcam++.csv"
    model_path = "/gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forcresco/weights/eye_nii_2202_onecaseoneimage2_doctorcompare_yolov5s_epoch200_batch16_89.8p/last.pt"

    # 設定
    threshold = 0.5
    device = "cuda" if torch.cuda.is_available() else "cpu"
    input_size = (640, 640)
    names = ["infection","normal","non-infection","scar","tumor","deposit","APAC","lens opacity","bullous"]

    print(f"使用デバイス: {device}")
    if device == "cuda":
        print(f"GPU: {torch.cuda.get_device_name(0)}")

    # モデルのロード
    print("\nモデルをロード中...")
    model = YOLOV5TorchObjectDetector(model_path, device, img_size=input_size, names=names)

    # 比較するレイヤー
    target_layers = [
        "model_17_cv3_conv",
        "model_20_cv3_conv",
        "model_23_cv3_conv",
        "model_24_m_0",
        "model_24_m_1",
        "model_24_m_2"
    ]

    # CSVから最初の画像を自動選択
    print("\nCSVファイルから画像を選択中...")
    df = pd.read_csv(csv_path)
    test_image_path = None

    for idx in range(min(10, len(df))):
        row = df.iloc[idx]
        img_basename = row["image_basename"]

        if pd.isna(img_basename):
            continue

        for ext in [".jpg", ".jpeg", ".png", ".JPG", ".JPEG", ".PNG", ""]:
            test_path = os.path.join(folder_path, f"{img_basename}{ext}")
            if os.path.exists(test_path):
                test_image_path = test_path
                break

        if test_image_path:
            break

    if test_image_path:
        print(f"テスト画像: {test_image_path}")

        # 3つのバージョンの比較実行
        results_df = compare_all_versions(test_image_path, model, target_layers, threshold)

        if results_df is not None and not results_df.empty:
            results_df.to_csv('gradcam_version_comparison_results.csv', index=False)
            print("\n詳細な結果を 'gradcam_version_comparison_results.csv' として保存しました。")

            # 結果の要約を表示
            print("\n=== 結果の要約 ===")
            print(results_df)

            # 各要素の影響を分析
            print("\n=== 各要素の影響分析 ===")
            print("1. retain_graph=True → False の影響（V1→V2）:")
            print(f"   平均差分: {results_df['diff_v1_v2'].mean():+.6f}")
            print("\n2. 全体勾配 → 選択的勾配 の影響（V2→V3）:")
            print(f"   平均差分: {results_df['diff_v2_v3'].mean():+.6f}")
            print("\n3. 全体的な最適化の影響（V1→V3）:")
            print(f"   平均差分: {results_df['diff_v1_v3'].mean():+.6f}")
        else:
            print("\n比較結果が得られませんでした。")
    else:
        print("テスト画像が見つかりませんでした。")

    # クリーンアップ
    del model
    torch.cuda.empty_cache()
    gc.collect()

    print("\n処理が完了しました。")

###**上記スクリプトのための保存用csvファイル作成**

In [ ]:
import pandas as pd

###これを押すとcsvが更新されてしまうので注意！！！
import pandas as pd
import os
from pathlib import Path

# ディレクトリとファイルパス
image_dir = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/Ueno_Mix1039/Images"
output_file_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/Ueno_Mix1039/Ueno_Mix1039_gradcam++.csv"

# CSVファイルが既に存在するかチェック
if os.path.exists(output_file_path):
    print(f"警告: CSVファイルが既に存在します: {output_file_path}")
    response = input("上書きしますか？ (y/n): ").lower()

    if response != 'y':
        print("処理を中止しました。")
        exit()
    else:
        print("既存のファイルを上書きします。")

# 画像ディレクトリが存在するかチェック
if not os.path.exists(image_dir):
    print(f"エラー: 画像ディレクトリが見つかりません: {image_dir}")
    exit()

# 画像ディレクトリ内のファイルを取得（一般的な画像拡張子をフィルタリング）
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.tif']
image_files = []

# ディレクトリ内のすべてのファイルをチェック
try:
    for file in os.listdir(image_dir):
        if any(file.lower().endswith(ext) for ext in image_extensions):
            # basenameを取得（拡張子なし）
            basename = os.path.splitext(file)[0]
            image_files.append(basename)
except Exception as e:
    print(f"エラー: 画像ディレクトリの読み込みに失敗しました: {e}")
    exit()

# 画像ファイルが見つからない場合の処理
if not image_files:
    print(f"警告: 画像ファイルが見つかりませんでした。")
    print(f"対象ディレクトリ: {image_dir}")
    print(f"対象拡張子: {', '.join(image_extensions)}")
    exit()

# ソート（必要に応じて）
image_files.sort()

# DataFrameを作成（basenameをインデックスとして）
df = pd.DataFrame(index=image_files)
df.index.name = 'image_basename'  # インデックス名を設定

# 例としてthresholdの値を設定
threshold = 0.5

# 指定されたレイヤー (originalはconvでなくact??)
layers = ['model_17_cv3_conv', 'model_20_cv3_conv', 'model_23_cv3_conv', '24_m_0', '24_m_1', '24_m_2']

# 各レイヤーに対して新しい列を作成
for layer in layers:
    df[f'AOI_{threshold}_layer{layer}'] = None  # 初期値をNoneに設定

# DataFrameの最初の数行を表示
print("\n作成されたDataFrame:")
print(df.head())
print(f"\n総画像数: {len(df)}")

# CSVファイルとして保存
try:
    df.to_csv(output_file_path, index=True)
    print(f"\nCSVファイルが正常に保存されました: {output_file_path}")
except Exception as e:
    print(f"エラー: CSVファイルの保存に失敗しました: {e}")

警告: CSVファイルが既に存在します: /gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/Ueno_Mix1039/Ueno_Mix1039_gradcam++.csv
上書きしますか？ (y/n): n
処理を中止しました。

作成されたDataFrame:
                     AOI_0.5_layermodel_17_cv3_conv  \
image_basename                                        
[apac]FKS_130_R_slit                           None   
[apac]FKS_238_L_slit                           None   
[apac]FKS_382_L_slit                           None   
[apac]TKB_001_R_slit                           None   
[apac]fko0074                                  None   

                     AOI_0.5_layermodel_20_cv3_conv  \
image_basename                                        
[apac]FKS_130_R_slit                           None   
[apac]FKS_238_L_slit                           None   
[apac]FKS_382_L_slit                           None   
[apac]TKB_001_R_slit                           None   
[apac]fko0074                                  None   

                     AOI_0.5_layermodel_23_cv3_conv AOI_0.5_layer24_m_0  \
im

#**Expert's annotation**

In [ ]:
from google.colab import drive
drive.mount('/gdrive')


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
# prompt: /gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/Ueno_Mix1039/Imagesと/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/Ueno_Mix1039/Masksの中の画像ファイルのbasename(拡張子は違うことあり）を比較＞

import os

# ディレクトリのパス
image_dir = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/Ueno_Mix1039/Images"
mask_dir = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/Ueno_Mix1039/Masks"

# 画像ファイルのリスト（拡張子なしのbasename）
image_files = [os.path.splitext(f)[0] for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]

# マスクファイルのリスト（拡張子なしのbasename）
mask_files = [os.path.splitext(f)[0] for f in os.listdir(mask_dir) if os.path.isfile(os.path.join(mask_dir, f))]

# 両方のリストに存在するbasenameを特定
common_files = sorted(list(set(image_files) & set(mask_files)))

# どちらか一方にしか存在しないbasenameを特定
image_only = sorted(list(set(image_files) - set(mask_files)))
mask_only = sorted(list(set(mask_files) - set(image_files)))

print(f"画像の総数: {len(image_files)}")
print(f"マスクの総数: {len(mask_files)}")
print(f"両方に存在するファイル数: {len(common_files)}")
print(f"画像にのみ存在するファイル数: {len(image_only)}")
print(f"マスクにのみ存在するファイル数: {len(mask_only)}")

# 結果を表示（オプション）
print("\n両方に存在するファイル (先頭10件):")
for f in common_files[:10]:
    print(f)

print("\n画像にのみ存在するファイル:")
for f in image_only[0:]:
    print(f)

print("\nマスクにのみ存在するファイル:")
for f in mask_only[0:]:
    print(f)

画像の総数: 1039
マスクの総数: 1007
両方に存在するファイル数: 996
画像にのみ存在するファイル数: 43
マスクにのみ存在するファイル数: 10

両方に存在するファイル (先頭10件):
[apac]FKS_130_R_slit
[apac]FKS_238_L_slit
[apac]FKS_382_L_slit
[apac]TKB_001_R_slit
[apac]fko0074
[apac]kei0903_1
[apac]kei0934
[apac]ttr_0366
[apac]ttr_0369_01
[apac]ttr_0378

画像にのみ存在するファイル:
[bullous]FKS_251_R_slit
[bullous]FKS_302_R_slit
[deposit]FKS_272_R_slit
[immun]FKS_374_R_slit
[immun]HRS_391_R_slit
[immun]TDC_269_R_slit
[immun]モーレン_wky2005_01
[infection]FKS_269_R_20240118_slit
[infection]細菌_KNZ5048
[infection]細菌_KNZ5052
[normal]FKS_339_R_slit
[normal]FKS_350_R_slit
[normal]TDC_263_R_slit
[scar]FKS_279_R_20240521_slit
[scar]FKS_325_R_slit
[scar]HRS_336_R_slit
[scar]HRS_409_R_slit
[scar]TDC_137_R_slit
[scar]myt 0024
[scar]myt 0035
[scar]myt 0036
[scar]myt 0040
[scar]myt 0049
[scar]myt 0052
[scar]myt 0073
[tumor]HRS_307_L_slit
[tumor]HRS_307_R_slit
[tumor]HRS_309_L_slit
[tumor]HRS_312_L_slit
[tumor]HRS_321_L_slit
[tumor]HRS_321_R_slit
[tumor]HRS_322_R_slit
[tumor]HRS_328_L_slit


In [ ]:
import os
import re
from collections import defaultdict

# ===== 設定 =====
image_dir = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/Ueno_Mix1039/Images"
mask_dir  = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/Ueno_Mix1039/Masks"
dry_run   = True        # True: 確認だけ, False: 実際にリネーム
# ===============

def normalize(name: str) -> str:
    """比較用キーを生成（大小区別せず, 空白/アンダースコア/ハイフンを除外, 末尾の -1/-2 も除外）"""
    # 末尾の -数字, _数字 を削除
    name = re.sub(r'[-_]\d+$', '', name)
    # 空白, アンダースコア, ハイフン を消して小文字化
    return re.sub(r'[\s_-]+', '', name).lower()

# Images 側: 正規化 → オリジナル名
image_key2name = {normalize(os.path.splitext(f)[0]): os.path.splitext(f)[0]
                  for f in os.listdir(image_dir)
                  if os.path.isfile(os.path.join(image_dir, f))}

# Masks 側: 同じく正規化 → [ (basename, 拡張子) , ... ]
mask_key2names = defaultdict(list)
for f in os.listdir(mask_dir):
    if os.path.isfile(os.path.join(mask_dir, f)):
        base, ext = os.path.splitext(f)
        mask_key2names[normalize(base)].append((base, ext))

# ① 1:1 で対応が確定するものだけ自動リネーム候補に
rename_plan = []
ambiguous   = []      # 同じキーで image と mask が複数ある等
no_match    = []      # mask 側に対応する image が無い

for key, mask_list in mask_key2names.items():
    if key in image_key2name and len(mask_list) == 1:
        img_base = image_key2name[key]
        mask_base, ext = mask_list[0]
        if img_base != mask_base:  # すでに一致していなければ
            src = os.path.join(mask_dir, mask_base + ext)
            dst = os.path.join(mask_dir, img_base + ext)
            rename_plan.append((src, dst))
    elif key in image_key2name:
        ambiguous.append(mask_list)   # 同キーで mask ファイルが複数
    else:
        no_match.extend(mask_list)    # そもそも image 側にキー無し

# ---- 結果を確認 ----
print(f"自動リネーム候補: {len(rename_plan)} 件")
for src, dst in rename_plan[:20]:
    print(f"  {os.path.basename(src)}  →  {os.path.basename(dst)}")

print(f"\nあいまい (手動確認推奨): {len(ambiguous)} 件")
print(f"マッチしない mask:      {len(no_match)} 件")

# ---- 実行フェーズ ----
if not dry_run:
    for src, dst in rename_plan:
        os.rename(src, dst)
    print(f"\n{len(rename_plan)} 件リネーム完了")
else:
    print("\n※ dry_run=True のためリネームせず終了。チェック後 dry_run=False にして再実行してください。")


自動リネーム候補: 0 件

あいまい (手動確認推奨): 5 件
マッチしない mask:      9 件

※ dry_run=True のためリネームせず終了。チェック後 dry_run=False にして再実行してください。


In [ ]:
import os
import re
from collections import defaultdict

# ====== 設定 ======
image_dir = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/Ueno_Mix1039/Images"
mask_dir  = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/Ueno_Mix1039/Masks"
dry_run   = True        # 確認だけなら True、実際にリネームするなら False
# ===================

def normalize(name: str) -> str:
    """比較キーを生成
       ・末尾の -1/_2 など 1–2 桁番号だけ削除
       ・空白・アンダーバー・ハイフン除去
       ・小文字化
    """
    name = re.sub(r'[-_]\d{1,2}$', '', name)
    return re.sub(r'[\s_-]+', '', name).lower()

# ── 画像側 ──
img_bases = [os.path.splitext(f)[0] for f in os.listdir(image_dir)
             if os.path.isfile(os.path.join(image_dir, f))]
img_key2base = {normalize(b): b for b in img_bases}

# ── マスク側 ──
mask_files = [f for f in os.listdir(mask_dir)
              if os.path.isfile(os.path.join(mask_dir, f))]
mask_key2info = defaultdict(list)  # key → [(base, ext, full_path), …]
for f in mask_files:
    base, ext = os.path.splitext(f)
    key = normalize(base)
    mask_key2info[key].append((base, ext, os.path.join(mask_dir, f)))

# ── リネーム計画 ──
rename_plan, ambiguous, no_match = [], [], []
for key, masks in mask_key2info.items():
    if key in img_key2base and len(masks) == 1:
        img_base = img_key2base[key]
        m_base, ext, src = masks[0]
        if m_base != img_base:
            dst = os.path.join(mask_dir, img_base + ext)
            rename_plan.append((src, dst))
    elif key in img_key2base:
        ambiguous.extend(masks)
    else:
        no_match.extend(masks)

# ── dry-run 出力 ──
print(f"リネーム予定: {len(rename_plan)} 件")
for s, d in rename_plan[:20]:
    print("  ", os.path.basename(s), "→", os.path.basename(d))

print(f"\n曖昧 (手動確認): {len(ambiguous)} 件")
print(f"対応する画像なし: {len(no_match)} 件")

# ── 実行 ──
if not dry_run:
    for src, dst in rename_plan:
        try:
            os.rename(src, dst)
        except FileExistsError:
            print("  * 既に存在: ", os.path.basename(dst))
    print("\nリネーム完了")

# ── リネーム後の完全一致チェック ──
# （dry_run でも現状を確認できる）
img_keys  = set(img_key2base)
mask_keys = set(normalize(os.path.splitext(f)[0]) for f in
                os.listdir(mask_dir) if os.path.isfile(os.path.join(mask_dir, f)))

unmatched_imgs  = sorted(img_key2base[k] for k in img_keys - mask_keys)
unmatched_masks = sorted(os.path.splitext(f)[0] for f in mask_files
                         if normalize(os.path.splitext(f)[0]) not in img_keys)

print("\n====== 最終サマリ ======")
print("画像にのみ存在:", len(unmatched_imgs))
print("マスクにのみ存在:", len(unmatched_masks))
if unmatched_imgs:
    print("\n▼画像のみ")
    for b in unmatched_imgs:
        print(b)
if unmatched_masks:
    print("\n▼マスクのみ")
    for b in unmatched_masks:
        print(b)


リネーム予定: 7 件
   [scar]myt_0073.png → [scar]myt 0073.png
   [scar]myt_0035.png → [scar]myt 0035.png
   [scar]myt_0036.png → [scar]myt 0036.png
   [scar]myt_0040.png → [scar]myt 0040.png
   [scar]myt_0049.png → [scar]myt 0049.png
   [scar]myt_0052.png → [scar]myt 0052.png
   [scar]myt_0024.png → [scar]myt 0024.png

曖昧 (手動確認): 4 件
対応する画像なし: 2 件

====== 最終サマリ ======
画像にのみ存在: 36
マスクにのみ存在: 2

▼画像のみ
[bullous]FKS_251_R_slit
[bullous]FKS_302_R_slit
[deposit]FKS_272_R_slit
[immun]FKS_374_R_slit
[immun]HRS_391_R_slit
[immun]TDC_269_R_slit
[immun]モーレン_wky2005_01
[infection]FKS_269_R_20240118_slit
[infection]細菌_KNZ5048
[infection]細菌_KNZ5052
[normal]FKS_339_R_slit
[normal]FKS_350_R_slit
[normal]TDC_263_R_slit
[scar]FKS_279_R_20240521_slit
[scar]FKS_325_R_slit
[scar]HRS_336_R_slit
[scar]HRS_409_R_slit
[scar]TDC_137_R_slit
[tumor]HRS_307_L_slit
[tumor]HRS_307_R_slit
[tumor]HRS_309_L_slit
[tumor]HRS_312_L_slit
[tumor]HRS_321_L_slit
[tumor]HRS_321_R_slit
[tumor]HRS_322_R_slit
[tumor]HRS_328_L_slit
[tumor

In [ ]:
import os
import re

# ===== ディレクトリ設定 =====
image_dir = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/Ueno_Mix1039/Images"
mask_dir  = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/Ueno_Mix1039/Masks"
# ==========================

def normalize(name: str) -> str:
    """比較用キーを生成"""
    name = re.sub(r'[-_]\d+$', '', name)       # 末尾 -1/-2/_1 などを削除
    return re.sub(r'[\s_-]+', '', name).lower()  # 空白・アンダースコアを除去 → 小文字

# ── 画像側 ──
image_bases = [os.path.splitext(f)[0] for f in os.listdir(image_dir)
               if os.path.isfile(os.path.join(image_dir, f))]
image_norm2orig = {normalize(b): b for b in image_bases}

# ── マスク側 ──
mask_bases = [os.path.splitext(f)[0] for f in os.listdir(mask_dir)
              if os.path.isfile(os.path.join(mask_dir, f))]
mask_norm2orig = {normalize(b): b for b in mask_bases}

# ── 一致判定 ──
image_keys = set(image_norm2orig)
mask_keys  = set(mask_norm2orig)

# まだ一致しないもの
image_only_keys = image_keys - mask_keys
mask_only_keys  = mask_keys  - image_keys

unmatched_images = sorted([image_norm2orig[k] for k in image_only_keys])
unmatched_masks  = sorted([mask_norm2orig[k]  for k in mask_only_keys])

# ── 結果表示 ──
print(f"完全一致しない画像   : {len(unmatched_images)} 件")
print(f"完全一致しないマスク : {len(unmatched_masks)} 件")

print("\n▼ 画像にのみ存在する basename")
for b in unmatched_images:
    print(b)

print("\n▼ マスクにのみ存在する basename")
for b in unmatched_masks:
    print(b)


完全一致しない画像   : 43 件
完全一致しないマスク : 2 件

▼ 画像にのみ存在する basename
[bullous]FKS_251_R_slit
[bullous]FKS_302_R_slit
[deposit]FKS_272_R_slit
[immun]FKS_374_R_slit
[immun]HRS_391_R_slit
[immun]TDC_269_R_slit
[immun]モーレン_wky2005_01
[infection]FKS_269_R_20240118_slit
[infection]細菌_KNZ5048
[infection]細菌_KNZ5052
[normal]FKS_339_R_slit
[normal]FKS_350_R_slit
[normal]TDC_263_R_slit
[scar]FKS_279_R_20240521_slit
[scar]FKS_325_R_slit
[scar]HRS_336_R_slit
[scar]HRS_409_R_slit
[scar]TDC_137_R_slit
[scar]myt 0024
[scar]myt 0035
[scar]myt 0036
[scar]myt 0040
[scar]myt 0049
[scar]myt 0052
[scar]myt 0073
[tumor]HRS_307_L_slit
[tumor]HRS_307_R_slit
[tumor]HRS_309_L_slit
[tumor]HRS_312_L_slit
[tumor]HRS_321_L_slit
[tumor]HRS_321_R_slit
[tumor]HRS_322_R_slit
[tumor]HRS_328_L_slit
[tumor]HRS_339_L_slit
[tumor]HRS_353_L_slit
[tumor]HRS_354_R_slit
[tumor]HRS_356_L_slit
[tumor]HRS_356_R_slit
[tumor]HRS_370_R_slit
[tumor]HRS_371_L_slit
[tumor]HRS_384_R_slit
[tumor]HRS_404_R_slit
[tumor]翼状片_wky7047

▼ マスクにのみ存在する basenam

In [ ]:
import os
os.chdir("/content/yolov5-gradcam")

import pandas as pd
import numpy as np
import cv2
# from google.colab.patches import cv2_imshow # Colab環境でcv2_imshowを使用する場合にコメント解除
from deep_utils.utils.box_utils.boxes import Box
import os
from tqdm import tqdm
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import gc
import torchvision

# YOLOv5の必要な関数をインポート - attempt_loadの場所を修正
from utils.general import non_max_suppression as yolo_nms, xywh2xyxy, box_iou
from utils.datasets import letterbox
from models.experimental import attempt_load  # 修正: models.experimentalから

# YOLOv5のカスタムクラスをインポートします
from models.yolo import Model, Detect
from models.common import Conv, Bottleneck, C3, SPPF, Concat

# これまでに登場したクラスをすべてリストに追加します
from torch.nn import Sequential, Conv2d, BatchNorm2d, SiLU, MaxPool2d, Upsample, ModuleList
torch.serialization.add_safe_globals([
    Model, Detect, Sequential, ModuleList, Conv, Bottleneck, C3, SPPF, Concat,
    Conv2d, BatchNorm2d, SiLU, MaxPool2d, Upsample
])

# PYTORCH_CUDA_ALLOC_CONFを設定
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# グローバル変数として定義（エラー回避のため）
global img_size
img_size = 640

# PYTORCH_CUDA_ALLOC_CONFを設定
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# グローバル変数として定義（エラー回避のため）
global img_size
img_size = 640

# ユーザー提供のYOLOV5TorchObjectDetectorクラス
class YOLOV5TorchObjectDetector(nn.Module):
    def __init__(self,
                 model_weight,
                 device,
                 img_size,
                 names=None,
                 mode="eval",
                 confidence=0.25,
                 iou_thresh=0.45,
                 agnostic_nms=False):
        super(YOLOV5TorchObjectDetector, self).__init__()
        self.device = device
        self.model = None
        self.img_size = img_size
        self.mode = mode
        self.confidence = confidence
        self.iou_thresh = iou_thresh
        self.agnostic = agnostic_nms

        self.model = attempt_load(model_weight, device=device)
        print("[INFO] Model is loaded")

        # === 変更点: モデル全体のrequires_grad_(True)を明示的に設定 ===
        # Grad-CAMのためにモデル全体の勾配計算を有効にする
        for param in self.model.parameters():
            param.requires_grad = True
        self.model.to(device)

        if self.mode == "train":
            self.model.train()
        else:
            self.model.eval()

        if names is None:
            print("[INFO] fetching names from coco file")
            self.names = ["person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat",
                          "traffic light",
                          "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat", "dog", "horse", "sheep",
                          "cow",
                          "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella", "handbag", "tie", "suitcase",
                          "frisbee",
                          "skis", "snowboard", "sports ball", "kite", "baseball bat", "baseball glove", "skateboard",
                          "surfboard",
                          "tennis racket", "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana",
                          "apple",
                          "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair",
                          "couch",
                          "potted plant", "bed", "dining table", "toilet", "tv", "laptop", "mouse", "remote",
                          "keyboard", "cell phone",
                          "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
                          "teddy bear",
                          "hair drier", "toothbrush"]
        else:
            self.names = names

        # preventing cold start
        # ダミーのフォワードパスはメモリを消費するため、コメントアウト
        # img = torch.zeros((1, 3, *self.img_size), device=device)
        # self.model(img)

    @staticmethod
    def non_max_suppression(prediction, logits, conf_thres=0.25, iou_thres=0.45, classes=None, agnostic=False,
                            multi_label=False, labels=(), max_det=300):
        nc = prediction.shape[2] - 5  # number of classes
        xc = prediction[..., 4] > conf_thres  # candidates

        assert 0 <= conf_thres <= 1, f"Invalid Confidence threshold {conf_thres}, valid values are between 0.0 and 1.0"
        assert 0 <= iou_thres <= 1, f"Invalid IoU {iou_thres}, valid values are between 0.0 and 1.0"

        min_wh, max_wh = 2, 4096  # (pixels) minimum and maximum box width and height
        max_nms = 30000  # maximum number of boxes into torchvision.ops.nms()
        time_limit = 10.0  # seconds to quit after
        redundant = True  # require redundant detections
        multi_label &= nc > 1  # multiple labels per box (adds 0.5ms/img)
        merge = False  # use merge-NMS

        t = time.time()
        output = [torch.zeros((0, 6), device=prediction.device)] * prediction.shape[0]
        logits_output = [torch.zeros((0, 80), device=logits.device)] * logits.shape[0]
        for xi, (x, log_) in enumerate(zip(prediction, logits)):  # image index, image inference
            x = x[xc[xi]]  # confidence
            log_ = log_[xc[xi]]
            if labels and len(labels[xi]):
                l = labels[xi]
                v = torch.zeros((len(l), nc + 5), device=x.device)
                v[:, :4] = l[:, 1:5]  # box
                v[:, 4] = 1.0  # conf
                v[range(len(l)), l[:, 0].long() + 5] = 1.0  # cls
                x = torch.cat((x, v), 0)

            if not x.shape[0]:
                continue

            x[:, 5:] *= x[:, 4:5]  # conf = obj_conf * cls_conf
            box = xywh2xyxy(x[:, :4])

            if multi_label:
                i, j = (x[:, 5:] > conf_thres).nonzero(as_tuple=False).T
                x = torch.cat((box[i], x[i, j + 5, None], j[:, None].float()), 1)
            else:  # best class only
                conf, j = x[:, 5:].max(1, keepdim=True)
                x = torch.cat((box, conf, j.float()), 1)[conf.view(-1) > conf_thres]
                log_ = log_[conf.view(-1) > conf_thres]
            if classes is not None:
                x = x[(x[:, 5:6] == torch.tensor(classes, device=x.device)).any(1)]

            n = x.shape[0]  # number of boxes
            if not n:
                continue
            elif n > max_nms:
                x = x[x[:, 4].argsort(descending=True)[:max_nms]]

            c = x[:, 5:6] * (0 if agnostic else max_wh)  # classes
            boxes, scores = x[:, :4] + c, x[:, 4]  # boxes (offset by class), scores
            i = torchvision.ops.nms(boxes, scores, iou_thres)  # NMS
            if i.shape[0] > max_det:
                i = i[:max_det]
            if merge and (1 < n < 3E3):
                iou = box_iou(boxes[i], boxes) > iou_thres
                weights = iou * scores[None]
                x[i, :4] = torch.mm(weights, x[:, :4]).float() / weights.sum(1, keepdim=True)
                if redundant:
                    i = i[iou.sum(1) > 1]

            output[xi] = x[i]
            logits_output[xi] = log_[i]
            assert log_[i].shape[0] == x[i].shape[0]
            if (time.time() - t) > time_limit:
                print(f"WARNING: NMS time limit {time_limit}s exceeded")
                break

        return output, logits_output

    @staticmethod
    def yolo_resize(img, new_shape=(640, 640), color=(114, 114, 114), auto=True, scaleFill=False, scaleup=True):
        return letterbox(img, new_shape=new_shape, color=color, auto=auto, scaleFill=scaleFill, scaleup=scaleup)

    def forward(self, img):
        # === 変更点: 推論部分のwith torch.no_grad()を削除 ===
        # Grad-CAMのために勾配計算を有効にする
        prediction, logits, _ = self.model(img, augment=False)
        prediction, logits = self.non_max_suppression(prediction, logits, self.confidence, self.iou_thresh,
                                                      classes=None,
                                                      agnostic=self.agnostic)

        # 以下の処理はCPUで行うことでGPUメモリを節約
        self.boxes, self.class_names, self.classes, self.confidences = [[[] for _ in range(img.shape[0])] for _ in
                                                                        range(4)]
        for i, det in enumerate(prediction):  # detections per image
            if len(det):
                # detをCPUに移動
                det_cpu = det.cpu()
                for *xyxy, conf, cls in det_cpu:
                    xyxy[0] = max(0, xyxy[0])
                    xyxy[1] = max(0, xyxy[1])
                    xyxy[2] = min(img_size, xyxy[2])
                    xyxy[3] = min(img_size, xyxy[3])

                    bbox = Box.box2box(xyxy,
                                       in_source=Box.BoxSource.Torch,
                                       to_source=Box.BoxSource.Numpy,
                                       return_int=True)
                    self.boxes[i].append(bbox)
                    self.confidences[i].append(round(conf.item(), 2))
                    cls = int(cls.item())
                    self.classes[i].append(cls)
                    if self.names is not None:
                        self.class_names[i].append(self.names[cls])
                    else:
                        self.class_names[i].append(cls)
        return [self.boxes, self.classes, self.class_names, self.confidences], logits

    def preprocessing(self, img):
        if len(img.shape) != 4:
            img = np.expand_dims(img, axis=0)
        im0 = img.astype(np.uint8)
        img = np.array([self.yolo_resize(im, new_shape=self.img_size)[0] for im in im0])
        img = img.transpose((0, 3, 1, 2))
        img = np.ascontiguousarray(img)
        img = torch.from_numpy(img).to(self.device)
        img = img / 255.0
        return img

# YOLOV5GradCAMクラス
class YOLOV5GradCAM:
    def __init__(self, model, layer_name, img_size=(640, 640), method="gradcam"):
        self.model = model
        self.gradients = dict()
        self.activations = dict()
        self.method = method
        self.cls_names = []

        def backward_hook(module, grad_input, grad_output):
            self.gradients["value"] = grad_output[0]
            return None

        def forward_hook(module, input, output):
            self.activations["value"] = output
            return None

        target_layer = find_yolo_layer(self.model, layer_name)
        # === 変更点: target_layerのrequires_gradをTrueに設定 ===
        # モデル全体のrequires_grad_(True)を削除したため、ここでGrad-CAMに必要なレイヤーのみTrueにする
        for param in target_layer.parameters():
            param.requires_grad = True
        target_layer.register_forward_hook(forward_hook)
        target_layer.register_backward_hook(backward_hook)

        # 初期化時のメモリ消費を抑えるため、ダミーのフォワードパスを削除
        # device = "cuda" if next(self.model.model.parameters()).is_cuda else "cpu"
        # self.model(torch.zeros(1, 3, *img_size, device=device))

    def forward(self, input_img, class_idx=True):
        saliency_maps = []
        b, c, h, w = input_img.size()

        # モデルのフォワードパスは勾配計算が必要なため、no_grad()を適用しない
        preds, logits = self.model(input_img)

        # topkの計算は勾配不要なのでno_grad()を適用
        with torch.no_grad():
            _, top3_indices = torch.topk(logits[0], k=3)

        if top3_indices.numel() > 0:
            preds[1][0] = top3_indices.tolist()[0]
            preds[2][0] = [names[i] for i in preds[1][0]]
            self.cls_names = preds[2][0]
        else:
            self.cls_names = []

        if self.method == "eigencam":
            saliency_map = self._eigencam()
            saliency_maps.append(saliency_map)
        else:
            for cls, cls_name in zip(preds[1][0], preds[2][0]):
                if class_idx:
                    score = logits[0][0][cls]
                else:
                    score = logits[0][0].max()

                self.model.zero_grad()
                score.backward(retain_graph=True)
                gradients = self.gradients["value"]
                activations = self.activations["value"]
                b, k, u, v = gradients.size()

                if self.method == "gradcam":
                    weights = self._gradcam_weights(gradients, b, k)
                elif self.method == "gradcampp":
                    weights = self._gradcampp_weights(gradients, activations, score, b, k, u, v)

                # saliency_mapの計算は勾配不要なのでno_grad()を適用
                with torch.no_grad():
                    saliency_map = (weights * activations).sum(1, keepdim=True)
                    saliency_map = F.relu(saliency_map)
                    saliency_map = F.interpolate(saliency_map, size=(h, w), mode="bilinear", align_corners=False)
                    saliency_map_min, saliency_map_max = saliency_map.min(), saliency_map.max()
                    # ゼロ除算を避けるためにepsilonを追加
                    saliency_map = (saliency_map - saliency_map_min).div(saliency_map_max - saliency_map_min + 1e-8).data
                saliency_maps.append(saliency_map)

        return saliency_maps, logits, preds, self.cls_names

    def _gradcam_weights(self, gradients, b, k):
        alpha = gradients.view(b, k, -1).mean(2)
        weights = alpha.view(b, k, 1, 1)
        return weights

    def _gradcampp_weights(self, gradients, activations, score, b, k, u, v):
        alpha_num = gradients.pow(2)
        alpha_denom = gradients.pow(2).mul(2) + \
            activations.mul(gradients.pow(3)).view(b, k, u*v).sum(-1, keepdim=True).view(b, k, 1, 1)
        alpha_denom = torch.where(alpha_denom != 0.0, alpha_denom, torch.ones_like(alpha_denom))
        alpha = alpha_num.div(alpha_denom + 1e-7)

        relu_grad = F.relu(score.exp() * gradients)
        weights = (relu_grad * alpha).view(b, k, u*v).sum(-1).view(b, k, 1, 1)

        return weights

    def _eigencam(self):
        activations = self.activations["value"]
        b, k, u, v = activations.size()
        activations_reshaped = activations.view(b, k, -1)
        # Compute covariance matrix
        cov = activations_reshaped @ activations_reshaped.transpose(-1, -2)
        cov /= (u * v)
        # Compute eigenvectors
        eigenvalues, eigenvectors = torch.linalg.eigh(cov)
        # Select the eigenvector corresponding to the largest eigenvalue
        leading_eigenvector = eigenvectors[..., -1].unsqueeze(-1)
        # Compute EigenCAM
        eigen_cam = (activations_reshaped.transpose(-1, -2) @ leading_eigenvector).view(b, u, v)
        eigen_cam = F.relu(eigen_cam)
        # Normalize
        eigen_cam_min, eigen_cam_max = eigen_cam.min(), eigen_cam.max()
        eigen_cam = (eigen_cam - eigen_cam_min) / (eigen_cam_max - eigen_cam_min + 1e-8)
        # Resize to match the input image size
        eigen_cam = F.interpolate(eigen_cam.unsqueeze(1), size=(640, 640), mode="bilinear", align_corners=False)
        return eigen_cam

    def __call__(self, input_img):
        return self.forward(input_img)

# find_yolo_layer関数
def find_yolo_layer(model, layer_name):
   hierarchy = layer_name.split("_")
   target_layer = model.model._modules[hierarchy[0]]

   for h in hierarchy[1:]:
       target_layer = target_layer._modules[h]
   return target_layer

# get_aoi関数
def get_aoi(bbox, masks, threshold):
   total_intersect_pixels = 0

   for mask in masks:
       mask = mask.squeeze().mul(255).add_(0.5).clamp_(0, 255).detach().cpu().numpy().astype(np.uint8)
       mask = np.nan_to_num(mask, nan=0.0).astype(np.uint8)

       mask[mask < threshold * 255] = 0
       mask[mask >= threshold * 255] = 255
       binary_mask = mask >= threshold * 255

       x1, y1, x2, y2 = bbox
       mask_bbox = binary_mask[y1:y2, x1:x2]

       intersect_pixels = np.sum(mask_bbox)
       total_intersect_pixels += intersect_pixels

       mask_bbox_area = (y2 - y1) * (x2 - x1)

       AOI = total_intersect_pixels / mask_bbox_area if mask_bbox_area > 0 else 0

   return AOI

# calculate_aoi関数（チェックポイント機能付き）
def calculate_aoi(folder_path, csv_path, threshold, model, target_layers,
                  start_index=None, end_index=None, skip_processed=True):
    df = pd.read_csv(csv_path)
    df = df.loc[:, ~df.columns.str.contains("^Unnamed")]

    # AOIカラムの定義
    all_layers_cols = [
        ("model_17_cv3_conv", f"AOI_{threshold}_layermodel_17_cv3_conv"),
        ("model_20_cv3_conv", f"AOI_{threshold}_layermodel_20_cv3_conv"),
        ("model_23_cv3_conv", f"AOI_{threshold}_layermodel_23_cv3_conv"),
        ("model_24_m_0", f"AOI_{threshold}_layer24_m_0"),
        ("model_24_m_1", f"AOI_{threshold}_layer24_m_1"),
        ("model_24_m_2", f"AOI_{threshold}_layer24_m_2")
    ]

    # カラムが存在しない場合は追加
    for layer_name, col_name in all_layers_cols:
        if col_name not in df.columns:
            df[col_name] = None

    # 処理済み画像の判定関数
    def is_processed(row):
        """すべてのAOIカラムに値が入っているかチェック"""
        for _, col_name in all_layers_cols:
            if pd.isna(row[col_name]) or row[col_name] is None:
                return False
        return True

    # 処理対象のインデックスを決定
    if skip_processed and start_index is None and end_index is None:
        # 未処理の画像のみを対象とする
        unprocessed_indices = []
        for idx, row in df.iterrows():
            if not is_processed(row):
                unprocessed_indices.append(idx)

        if not unprocessed_indices:
            print("すべての画像が処理済みです。")
            return

        print(f"未処理画像数: {len(unprocessed_indices)}/{len(df)}")
        print(f"最初の未処理画像インデックス: {unprocessed_indices[0]}")

        # 処理対象インデックスのリスト
        target_indices = unprocessed_indices

    else:
        # 手動でインデックス範囲を指定した場合
        if start_index is None:
            start_index = 0
        if end_index is None:
            end_index = len(df)

        target_indices = list(range(start_index, end_index))

        # skip_processedがTrueの場合、範囲内でも処理済みはスキップ
        if skip_processed:
            target_indices = [idx for idx in target_indices
                            if idx < len(df) and not is_processed(df.iloc[idx])]

    print(f"処理対象画像数: {len(target_indices)}")

    # 進捗状況の表示
    processed_count = 0
    skipped_count = len(df) - len(target_indices) if skip_processed else 0
    not_found_count = 0
    error_count = 0

    # 定期的な保存のカウンター
    save_interval = 10  # 10画像ごとに保存

    # メインの処理ループ
    for i, index in enumerate(tqdm(target_indices, desc="Processing images")):
        row = df.iloc[index]
        img_basename = row["image_basename"]

        if pd.isna(img_basename):
            print(f"Skipping row {index} due to NaN image_basename")
            continue

        img_found = False
        img_path = None

        # 画像ファイルの検索
        for ext in [".jpg", ".jpeg", ".png", ".JPG", ".JPEG", ".PNG", ""]:
            test_path = os.path.join(folder_path, f"{img_basename}{ext}")
            if os.path.exists(test_path):
                img_path = test_path
                img_found = True
                break

        if not img_found:
            test_path = os.path.join(folder_path, img_basename)
            if os.path.exists(test_path):
                img_path = test_path
                img_found = True

        if img_found:
            img = cv2.imread(img_path)

            if img is None:
                print(f"Failed to read image: {img_path}")
                continue

            try:
                torch_img = model.preprocessing(img[..., ::-1])

                for layer_name, col_name in all_layers_cols:
                    try:
                        saliency_method = YOLOV5GradCAM(model=model, layer_name=layer_name, img_size=input_size, method="gradcampp")
                        masks, logits, [boxes, _, _, _], cls_names = saliency_method(torch_img)

                        if len(masks) > 0 and len(boxes) > 0:
                            mask = masks[0][0]
                            bbox = boxes[0][0]

                            aoi = get_aoi(bbox, [mask], threshold)
                            df.at[index, col_name] = aoi

                        del saliency_method
                        if device == "cuda":
                            torch.cuda.empty_cache()
                        gc.collect()

                    except Exception as e:
                        error_count += 1
                        if error_count <= 5:
                            print(f"Error processing image {img_basename} at layer {layer_name}: {str(e)}")

                processed_count += 1
                if processed_count % 50 == 0:
                    print(f"Processed {processed_count} images...")

                # 定期的な中間保存
                if (i + 1) % save_interval == 0:
                    df.to_csv(csv_path, index=False)
                    print(f"中間保存完了: {i + 1}/{len(target_indices)} 画像処理済み")

                    # メモリクリーンアップ
                    if device == "cuda":
                        torch.cuda.empty_cache()
                    gc.collect()

            except Exception as e:
                error_count += 1
                if error_count <= 5:
                    print(f"Error preprocessing image {img_basename}: {str(e)}")
        else:
            not_found_count += 1
            if not_found_count <= 5:
                print(f"Image not found: {img_basename}")

    # 最終保存
    df.to_csv(csv_path, index=False)

    print(f"\n処理完了サマリー:")
    print(f"- 処理した画像: {processed_count}個")
    print(f"- スキップした画像: {skipped_count}個")
    print(f"- 見つからなかった画像: {not_found_count}個")
    print(f"- エラーが発生した画像: {error_count}個")
    print(f"結果を保存しました: {csv_path}")

# メイン実行部分
folder_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/Ueno_Mix1039/Images"
csv_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/Ueno_Mix1039/Ueno_Mix1039_gradcam++.csv"

threshold = 0.5

model_path = "/gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forcresco/weights/eye_nii_2202_onecaseoneimage2_doctorcompare_yolov5s_epoch200_batch16_89.8p/last.pt"
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"使用デバイス: {device}")
if device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")

input_size = (640, 640)
names = ["infection","normal","non-infection","scar","tumor","deposit","APAC","lens opacity","bullous"]

model = YOLOV5TorchObjectDetector(model_path, device, img_size=input_size, names=names)

target_layers = [
    "model_17_cv3_conv",
    "model_20_cv3_conv",
    "model_23_cv3_conv",
    "model_24_m_0",
    "model_24_m_1",
    "model_24_m_2"
]

# バッチ処理モードの設定
USE_BATCH_MODE = False
BATCH_SIZE = 20

if USE_BATCH_MODE:
    print("=== バッチ処理モードで実行します ===")

    # is_processed関数を定義（calculate_aoi関数外でも使えるように）
    def is_processed(row):
        """すべてのAOIカラムに値が入っているかチェック"""
        all_layers_cols = [
            ("model_17_cv3_conv", f"AOI_{threshold}_layermodel_17_cv3_conv"),
            ("model_20_cv3_conv", f"AOI_{threshold}_layermodel_20_cv3_conv"),
            ("model_23_cv3_conv", f"AOI_{threshold}_layermodel_23_cv3_conv"),
            ("model_24_m_0", f"AOI_{threshold}_layer24_m_0"),
            ("model_24_m_1", f"AOI_{threshold}_layer24_m_1"),
            ("model_24_m_2", f"AOI_{threshold}_layer24_m_2")
        ]
        for _, col_name in all_layers_cols:
            if pd.isna(row[col_name]) or row[col_name] is None:
                return False
        return True

    # 未処理画像のインデックスを取得
    df_check = pd.read_csv(csv_path)
    unprocessed_indices = []

    for idx, row in df_check.iterrows():
        if not is_processed(row):
            unprocessed_indices.append(idx)

    if not unprocessed_indices:
        print("すべての画像が処理済みです。")
    else:
        print(f"未処理画像数: {len(unprocessed_indices)}")

        # バッチごとに処理
        for i in range(0, len(unprocessed_indices), BATCH_SIZE):
            batch_indices = unprocessed_indices[i:i+BATCH_SIZE]
            batch_start = batch_indices[0] if batch_indices else 0
            batch_end = batch_indices[-1] + 1 if batch_indices else 0

            print(f"\nバッチ {i//BATCH_SIZE + 1}/{(len(unprocessed_indices)-1)//BATCH_SIZE + 1}: "
                  f"インデックス {batch_indices}")

            # バッチ内の画像をまとめて処理
            calculate_aoi(folder_path, csv_path, threshold, model, target_layers,
                         start_index=None, end_index=None, skip_processed=True)

            # 処理完了後のクリーンアップ
            if device == "cuda":
                torch.cuda.empty_cache()
            gc.collect()

            print(f"バッチ {i//BATCH_SIZE + 1} 完了")

            # バッチが完了したら、再度未処理画像を確認（中断された場合の対策）
            df_check = pd.read_csv(csv_path)
            remaining_unprocessed = sum(1 for idx, row in df_check.iterrows() if not is_processed(row))
            print(f"残り未処理画像数: {remaining_unprocessed}")

            if remaining_unprocessed == 0:
                break

    print("\n=== 全バッチ処理完了 ===")

else:
    print("=== 通常モードで実行します ===")

    # モード選択
    mode = input("実行モードを選択してください:\n1: 自動（未処理画像を検出）\n2: 手動（範囲を指定）\n3: 強制再処理（範囲を指定、処理済みも再処理）\n選択 (1/2/3): ")

    if mode == "1":
        # 自動モード
        print("\n未処理画像を自動検出して処理します...")
        calculate_aoi(folder_path, csv_path, threshold, model, target_layers,
                     skip_processed=True)

    elif mode == "2":
        # 手動モード（処理済みはスキップ）
        start_index = int(input("開始インデックスを入力してください: "))
        end_index = int(input("終了インデックスを入力してください: "))
        print(f"\nインデックス {start_index} から {end_index-1} まで処理します（処理済みはスキップ）...")
        calculate_aoi(folder_path, csv_path, threshold, model, target_layers,
                     start_index=start_index, end_index=end_index, skip_processed=True)

    elif mode == "3":
        # 強制再処理モード
        start_index = int(input("開始インデックスを入力してください: "))
        end_index = int(input("終了インデックスを入力してください: "))
        print(f"\nインデックス {start_index} から {end_index-1} まで強制的に再処理します...")
        calculate_aoi(folder_path, csv_path, threshold, model, target_layers,
                     start_index=start_index, end_index=end_index, skip_processed=False)

    else:
        print("無効な選択です。プログラムを終了します。")

In [ ]:
#################
#################
os.chdir('/content/yolov5-gradcam')

import numpy as np
from deep_utils.utils.box_utils.boxes import Box
import torch
from models.experimental import attempt_load
from utils.general import xywh2xyxy
from utils.datasets import letterbox
import cv2
import time
import torchvision
import torch.nn as nn
from utils.metrics import box_iou

from torch.nn import Sequential, Conv2d, BatchNorm2d, SiLU, MaxPool2d, Upsample, ModuleList

# YOLOv5のカスタムクラスをインポートします
from models.yolo import Model, Detect
from models.common import Conv, Bottleneck, C3, SPPF, Concat

# これまでに登場したクラスをすべてリストに追加します
torch.serialization.add_safe_globals([
    Model, Detect, Sequential, ModuleList, Conv, Bottleneck, C3, SPPF, Concat,
    Conv2d, BatchNorm2d, SiLU, MaxPool2d, Upsample
])



class YOLOV5TorchObjectDetector(nn.Module):
    def __init__(self,
                 model_weight,
                 device,
                 img_size,
                 names=None,
                 mode='eval',
                 confidence=0.25,
                 iou_thresh=0.45,
                 agnostic_nms=False):
        super(YOLOV5TorchObjectDetector, self).__init__()
        self.device = device
        self.model = None
        self.img_size = img_size
        self.mode = mode
        self.confidence = confidence
        self.iou_thresh = iou_thresh
        self.agnostic = agnostic_nms
        self.model = attempt_load(model_weight, device=device)
        print("[INFO] Model is loaded")
        self.model.requires_grad_(True)
        self.model.to(device)
        if self.mode == 'train':
            self.model.train()
        else:
            self.model.eval()
        # fetch the names
        if names is None:
            print('[INFO] fetching names from coco file')
            self.names = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat',
                          'traffic light',
                          'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep',
                          'cow',
                          'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase',
                          'frisbee',
                          'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard',
                          'surfboard',
                          'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana',
                          'apple',
                          'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair',
                          'couch',
                          'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
                          'keyboard', 'cell phone',
                          'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
                          'teddy bear',
                          'hair drier', 'toothbrush']
        else:
            self.names = names

        # preventing cold start
        img = torch.zeros((1, 3, *self.img_size), device=device)
        self.model(img)

    @staticmethod
    def non_max_suppression(prediction, logits, conf_thres=0.25, iou_thres=0.45, classes=None, agnostic=False,
                            multi_label=False, labels=(), max_det=300):
        """Runs Non-Maximum Suppression (NMS) on inference and logits results

        Returns:
             list of detections, on (n,6) tensor per image [xyxy, conf, cls] and pruned input logits (n, number-classes)
        """

        nc = prediction.shape[2] - 5  # number of classes
        xc = prediction[..., 4] > conf_thres  # candidates

        # Checks
        assert 0 <= conf_thres <= 1, f'Invalid Confidence threshold {conf_thres}, valid values are between 0.0 and 1.0'
        assert 0 <= iou_thres <= 1, f'Invalid IoU {iou_thres}, valid values are between 0.0 and 1.0'

        # Settings
        min_wh, max_wh = 2, 4096  # (pixels) minimum and maximum box width and height
        max_nms = 30000  # maximum number of boxes into torchvision.ops.nms()
        time_limit = 10.0  # seconds to quit after
        redundant = True  # require redundant detections
        multi_label &= nc > 1  # multiple labels per box (adds 0.5ms/img)
        merge = False  # use merge-NMS

        t = time.time()
        output = [torch.zeros((0, 6), device=prediction.device)] * prediction.shape[0]
        logits_output = [torch.zeros((0, 80), device=logits.device)] * logits.shape[0]
        for xi, (x, log_) in enumerate(zip(prediction, logits)):  # image index, image inference
            # Apply constraints
            # x[((x[..., 2:4] < min_wh) | (x[..., 2:4] > max_wh)).any(1), 4] = 0  # width-height
            x = x[xc[xi]]  # confidence
            log_ = log_[xc[xi]]
            # Cat apriori labels if autolabelling
            if labels and len(labels[xi]):
                l = labels[xi]
                v = torch.zeros((len(l), nc + 5), device=x.device)
                v[:, :4] = l[:, 1:5]  # box
                v[:, 4] = 1.0  # conf
                v[range(len(l)), l[:, 0].long() + 5] = 1.0  # cls
                x = torch.cat((x, v), 0)

            # If none remain process next image
            if not x.shape[0]:
                continue

            # Compute conf
            x[:, 5:] *= x[:, 4:5]  # conf = obj_conf * cls_conf
            # log_ *= x[:, 4:5]
            # Box (center x, center y, width, height) to (x1, y1, x2, y2)
            box = xywh2xyxy(x[:, :4])

            # Detections matrix nx6 (xyxy, conf, cls)
            if multi_label:
                i, j = (x[:, 5:] > conf_thres).nonzero(as_tuple=False).T
                x = torch.cat((box[i], x[i, j + 5, None], j[:, None].float()), 1)
            else:  # best class only
                conf, j = x[:, 5:].max(1, keepdim=True)
                # log_ = x[:, 5:]
                x = torch.cat((box, conf, j.float()), 1)[conf.view(-1) > conf_thres]
                log_ = log_[conf.view(-1) > conf_thres]
            # Filter by class
            if classes is not None:
                x = x[(x[:, 5:6] == torch.tensor(classes, device=x.device)).any(1)]

            # Check shape
            n = x.shape[0]  # number of boxes
            if not n:  # no boxes
                continue
            elif n > max_nms:  # excess boxes
                x = x[x[:, 4].argsort(descending=True)[:max_nms]]  # sort by confidence

            # Batched NMS
            c = x[:, 5:6] * (0 if agnostic else max_wh)  # classes
            boxes, scores = x[:, :4] + c, x[:, 4]  # boxes (offset by class), scores
            i = torchvision.ops.nms(boxes, scores, iou_thres)  # NMS
            if i.shape[0] > max_det:  # limit detections
                i = i[:max_det]
            if merge and (1 < n < 3E3):  # Merge NMS (boxes merged using weighted mean)
                # update boxes as boxes(i,4) = weights(i,n) * boxes(n,4)
                iou = box_iou(boxes[i], boxes) > iou_thres  # iou matrix
                weights = iou * scores[None]  # box weights
                x[i, :4] = torch.mm(weights, x[:, :4]).float() / weights.sum(1, keepdim=True)  # merged boxes
                if redundant:
                    i = i[iou.sum(1) > 1]  # require redundancy

            output[xi] = x[i]
            logits_output[xi] = log_[i]
            assert log_[i].shape[0] == x[i].shape[0]
            if (time.time() - t) > time_limit:
                print(f'WARNING: NMS time limit {time_limit}s exceeded')
                break  # time limit exceeded

        return output, logits_output

    @staticmethod
    def yolo_resize(img, new_shape=(640, 640), color=(114, 114, 114), auto=True, scaleFill=False, scaleup=True):

        return letterbox(img, new_shape=new_shape, color=color, auto=auto, scaleFill=scaleFill, scaleup=scaleup)

    def forward(self, img):
        prediction, logits, _ = self.model(img, augment=False)
        prediction, logits = self.non_max_suppression(prediction, logits, self.confidence, self.iou_thresh,
                                                      classes=None,
                                                      agnostic=self.agnostic)
        self.boxes, self.class_names, self.classes, self.confidences = [[[] for _ in range(img.shape[0])] for _ in
                                                                        range(4)]
        for i, det in enumerate(prediction):  # detections per image
            if len(det):
                for *xyxy, conf, cls in det:
                    # xyxyの値を修正
                    xyxy[0] = max(0, xyxy[0])
                    xyxy[1] = max(0, xyxy[1])
                    xyxy[2] = min(img_size, xyxy[2])
                    xyxy[3] = min(img_size, xyxy[3])

                    bbox = Box.box2box(xyxy,
                                       in_source=Box.BoxSource.Torch,
                                       to_source=Box.BoxSource.Numpy,
                                       return_int=True)
                    self.boxes[i].append(bbox)
                    self.confidences[i].append(round(conf.item(), 2))
                    cls = int(cls.item())
                    self.classes[i].append(cls)
                    if self.names is not None:
                        self.class_names[i].append(self.names[cls])
                    else:
                        self.class_names[i].append(cls)
        return [self.boxes, self.classes, self.class_names, self.confidences], logits

    def preprocessing(self, img):
        if len(img.shape) != 4:
            img = np.expand_dims(img, axis=0)
        im0 = img.astype(np.uint8)
        img = np.array([self.yolo_resize(im, new_shape=self.img_size)[0] for im in im0])
        img = img.transpose((0, 3, 1, 2))
        img = np.ascontiguousarray(img)
        img = torch.from_numpy(img).to(self.device)
        img = img / 255.0
        return img

In [ ]:
import gc
gc.collect()

In [ ]:
##################
##################
## 6-layerで解析##
##################
##################

import pandas as pd
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
from deep_utils import Box, split_extension
import os
from tqdm import tqdm
#from models.yolo_v5_object_detector import YOLOV5TorchObjectDetector
# from models.gradcam import YOLOV5GradCAM
import time
import torch
import torch.nn.functional as F
import gc

# グローバル変数として定義（エラー回避のため）
global img_size
img_size = 640

class YOLOV5GradCAM:
    def __init__(self, model, layer_name, img_size=(640, 640), method='gradcam'):
        self.model = model
        self.gradients = dict()
        self.activations = dict()
        self.method = method
        self.cls_names = []

        def backward_hook(module, grad_input, grad_output):
            self.gradients['value'] = grad_output[0]
            return None

        def forward_hook(module, input, output):
            self.activations['value'] = output
            return None

        target_layer = find_yolo_layer(self.model, layer_name)
        target_layer.register_forward_hook(forward_hook)
        target_layer.register_backward_hook(backward_hook)

        device = 'cuda' if next(self.model.model.parameters()).is_cuda else 'cpu'
        self.model(torch.zeros(1, 3, *img_size, device=device))

    def forward(self, input_img, class_idx=True):
        saliency_maps = []
        b, c, h, w = input_img.size()
        preds, logits = self.model(input_img)

        _, top3_indices = torch.topk(logits[0], k=3)

        if top3_indices.numel() > 0:
            preds[1][0] = top3_indices.tolist()[0]
            preds[2][0] = [names[i] for i in preds[1][0]]
            self.cls_names = preds[2][0]
        else:
            self.cls_names = []

        if self.method == 'eigencam':
            saliency_map = self._eigencam()
            saliency_maps.append(saliency_map)
        else:
            for cls, cls_name in zip(preds[1][0], preds[2][0]):
                if class_idx:
                    score = logits[0][0][cls]
                else:
                    score = logits[0][0].max()

                self.model.zero_grad()
                score.backward(retain_graph=True)
                gradients = self.gradients['value']
                activations = self.activations['value']
                b, k, u, v = gradients.size()

                if self.method == 'gradcam':
                    weights = self._gradcam_weights(gradients, b, k)
                elif self.method == 'gradcampp':
                    weights = self._gradcampp_weights(gradients, activations, score, b, k, u, v)

                saliency_map = (weights * activations).sum(1, keepdim=True)
                saliency_map = F.relu(saliency_map)
                saliency_map = F.interpolate(saliency_map, size=(h, w), mode='bilinear', align_corners=False)
                saliency_map_min, saliency_map_max = saliency_map.min(), saliency_map.max()
                saliency_map = (saliency_map - saliency_map_min).div(saliency_map_max - saliency_map_min).data
                saliency_maps.append(saliency_map)

        return saliency_maps, logits, preds, self.cls_names

    def _gradcam_weights(self, gradients, b, k):
        alpha = gradients.view(b, k, -1).mean(2)
        weights = alpha.view(b, k, 1, 1)
        return weights

    def _gradcampp_weights(self, gradients, activations, score, b, k, u, v):
        alpha_num = gradients.pow(2)
        alpha_denom = gradients.pow(2).mul(2) + \
            activations.mul(gradients.pow(3)).view(b, k, u*v).sum(-1, keepdim=True).view(b, k, 1, 1)
        alpha_denom = torch.where(alpha_denom != 0.0, alpha_denom, torch.ones_like(alpha_denom))
        alpha = alpha_num.div(alpha_denom + 1e-7)

        relu_grad = F.relu(score.exp() * gradients)
        weights = (relu_grad * alpha).view(b, k, u*v).sum(-1).view(b, k, 1, 1)

        return weights

    def _eigencam(self):
        activations = self.activations['value']
        b, k, u, v = activations.size()
        activations_reshaped = activations.view(b, k, -1)
        # Compute covariance matrix
        cov = activations_reshaped @ activations_reshaped.transpose(-1, -2)
        cov /= (u * v)
        # Compute eigenvectors
        eigenvalues, eigenvectors = torch.linalg.eigh(cov)
        # Select the eigenvector corresponding to the largest eigenvalue
        leading_eigenvector = eigenvectors[..., -1].unsqueeze(-1)
        # Compute EigenCAM
        eigen_cam = (activations_reshaped.transpose(-1, -2) @ leading_eigenvector).view(b, u, v)
        eigen_cam = F.relu(eigen_cam)
        # Normalize
        eigen_cam_min, eigen_cam_max = eigen_cam.min(), eigen_cam.max()
        eigen_cam = (eigen_cam - eigen_cam_min) / (eigen_cam_max - eigen_cam_min + 1e-8)
        # Resize to match the input image size
        eigen_cam = F.interpolate(eigen_cam.unsqueeze(1), size=(640, 640), mode='bilinear', align_corners=False)
        return eigen_cam

    def __call__(self, input_img):
        return self.forward(input_img)

def find_yolo_layer(model, layer_name):
   """Find yolov5 layer to calculate GradCAM and GradCAM++

   Args:
       model: yolov5 model.
       layer_name (str): the name of layer with its hierarchical information.

   Return:
       target_layer: found layer
   """
   hierarchy = layer_name.split('_')
   target_layer = model.model._modules[hierarchy[0]]

   for h in hierarchy[1:]:
       target_layer = target_layer._modules[h]
   return target_layer

def get_aoi(bbox, masks, threshold):
   total_intersect_pixels = 0  # 共通部分のピクセル数のカウンタを初期化

   for mask in masks:
       # マスクの前処理
       mask = mask.squeeze().mul(255).add_(0.5).clamp_(0, 255).detach().cpu().numpy().astype(np.uint8)
       mask[mask < threshold * 255] = 0
       mask[mask >= threshold * 255] = 255
       binary_mask = mask >= threshold * 255  # 閾値を超える部分を二値化

       # bboxの範囲内のマスク部分を取得
       x1, y1, x2, y2 = bbox
       mask_bbox = binary_mask[y1:y2, x1:x2]

       # 閾値を超える共通部分のピクセル数をカウント
       intersect_pixels = np.sum(mask_bbox)
       total_intersect_pixels += intersect_pixels

       # mask_bbox のピクセル数を取得
       mask_bbox_area = (y2 - y1) * (x2 - x1)

       # AOI を計算
       AOI = total_intersect_pixels / mask_bbox_area if mask_bbox_area > 0 else 0

   return AOI

def calculate_aoi(folder_path, csv_path, threshold, model, saliency_methods, start_index=0, end_index=None):
    df = pd.read_csv(csv_path)
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

    # 全レイヤーの設定
    all_layers = [
        ('model_17_cv3_act', f'AOI_{threshold}_layermodel_17_cv3_act'),
        ('model_20_cv3_act', f'AOI_{threshold}_layermodel_20_cv3_act'),
        ('model_23_cv3_act', f'AOI_{threshold}_layermodel_23_cv3_act'),
        ('model_24_m_0', f'AOI_{threshold}_layer24_m_0'),
        ('model_24_m_1', f'AOI_{threshold}_layer24_m_1'),
        ('model_24_m_2', f'AOI_{threshold}_layer24_m_2')
    ]

    # 列が存在しない場合は作成
    for layer_name, col_name in all_layers:
        if col_name not in df.columns:
            df[col_name] = None

    if end_index is None:
        end_index = len(df)

    # 実際に処理された画像の数をカウント
    processed_count = 0
    not_found_count = 0
    error_count = 0

    for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing images"):
        if index < start_index or index >= end_index:
            continue

        img_basename = row['image_basename']

        if pd.isna(img_basename):
            print(f"Skipping row {index} due to NaN image_basename")
            continue

        # 拡張子を試す
        img_found = False
        img_path = None

        for ext in ['.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG', '']:
            test_path = os.path.join(folder_path, f"{img_basename}{ext}")
            if os.path.exists(test_path):
                img_path = test_path
                img_found = True
                break

        if not img_found:
            test_path = os.path.join(folder_path, img_basename)
            if os.path.exists(test_path):
                img_path = test_path
                img_found = True

        if img_found:
            img = cv2.imread(img_path)

            if img is None:
                print(f"Failed to read image: {img_path}")
                continue

            try:
                torch_img = model.preprocessing(img[..., ::-1])

                # 全レイヤーで処理
                for i, (layer_name, col_name) in enumerate(all_layers):
                    if i < len(saliency_methods):
                        try:
                            masks, logits, [boxes, _, _, _], cls_names = saliency_methods[i](torch_img)

                            if len(masks) > 0 and len(boxes) > 0:
                                mask = masks[0][0]
                                bbox = boxes[0][0]

                                aoi = get_aoi(bbox, [mask], threshold)
                                df.at[index, col_name] = aoi

                        except Exception as e:
                            error_count += 1
                            if error_count <= 5:
                                print(f"Error processing image {img_basename} at layer {layer_name}: {str(e)}")

                processed_count += 1
                # 進捗表示
                if processed_count % 50 == 0:
                    print(f"Processed {processed_count} images...")

            except Exception as e:
                error_count += 1
                if error_count <= 5:
                    print(f"Error preprocessing image {img_basename}: {str(e)}")
        else:
            not_found_count += 1
            if not_found_count <= 5:
                print(f"Image not found: {img_basename}")

    print(f"\n処理完了: {processed_count}個の画像を処理")
    print(f"見つからなかった画像: {not_found_count}個")
    print(f"エラーが発生した画像: {error_count}個")

    df.to_csv(csv_path, index=False)
    print(f"結果を保存しました: {csv_path}")

# メイン実行部分
folder_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/Ueno_Mix1039/Images"
csv_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/Ueno_Mix1039/Ueno_Mix1039_gradcam++.csv"

threshold = 0.5

# モデルとsaliency_methodの定義
model_path = "/gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forcresco/weights/eye_nii_2202_onecaseoneimage2_doctorcompare_yolov5s_epoch200_batch16_89.8p/last.pt"
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"使用デバイス: {device}")
if device == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")

input_size = (640, 640)
names = ["infection","normal","non-infection","scar","tumor","deposit","APAC","lens opacity","bullous"]

model = YOLOV5TorchObjectDetector(model_path, device, img_size=input_size, names=names)

# 全レイヤーのtarget_layers（convに修正が必要な場合）
target_layers = [
    'model_17_cv3_conv',  # actからconvに変更
    'model_20_cv3_conv',  # actからconvに変更
    'model_23_cv3_conv',  # actからconvに変更
    'model_24_m_0',
    'model_24_m_1',
    'model_24_m_2'
]

# 実行モード選択
USE_BATCH_MODE = False  # バッチ処理モードを使用するかどうか
BATCH_SIZE = 20       # バッチサイズ

if USE_BATCH_MODE:
    print("=== バッチ処理モードで実行します ===")

    # CSVファイルの総行数を確認
    df_check = pd.read_csv(csv_path)
    total_images = len(df_check)
    print(f"総画像数: {total_images}")

    # 全レイヤーのsaliency_methods を作成
    print("\nGradCAMメソッドを初期化中...")
    saliency_methods = []
    for i, layer in enumerate(target_layers):
        print(f"  Layer {i+1}/{len(target_layers)}: {layer}")
        try:
            saliency_methods.append(
                YOLOV5GradCAM(model=model, layer_name=layer, img_size=input_size, method="gradcampp")
            )
        except Exception as e:
            print(f"  エラー: {layer} - {str(e)}")

    print(f"\n初期化完了: {len(saliency_methods)}個のレイヤー")

    # バッチ処理実行
    for batch_start in range(0, total_images, BATCH_SIZE):
        batch_end = min(batch_start + BATCH_SIZE, total_images)
        print(f"\n=== バッチ {batch_start//BATCH_SIZE + 1}/{(total_images-1)//BATCH_SIZE + 1}: "
              f"画像 {batch_start+1}-{batch_end}/{total_images} ===")

        calculate_aoi(folder_path, csv_path, threshold, model, saliency_methods,
                     start_index=batch_start, end_index=batch_end)

        # メモリ解放
        if device == 'cuda':
            torch.cuda.empty_cache()
        gc.collect()

        print(f"バッチ {batch_start//BATCH_SIZE + 1} 完了")

    print("\n=== 全バッチ処理完了 ===")

else:
    print("=== 通常モードで実行します ===")

    # 処理範囲の設定
    start_index = 0
    end_index = 5  # Noneで全データ処理

    # 全レイヤーのsaliency_methods を作成
    print("GradCAMメソッドを初期化中...")
    saliency_methods = []
    for i, layer in enumerate(target_layers):
        print(f"  Layer {i+1}/{len(target_layers)}: {layer}")
        try:
            saliency_methods.append(
                YOLOV5GradCAM(model=model, layer_name=layer, img_size=input_size, method="gradcampp")
            )
        except Exception as e:
            print(f"  エラー: {layer} - {str(e)}")

    print(f"\n初期化完了: {len(saliency_methods)}個のレイヤー")
    print("\n解析を開始します...")

    calculate_aoi(folder_path, csv_path, threshold, model, saliency_methods, start_index, end_index)

# レイヤーごとの個別処理（エラーが発生する場合の代替案）
"""
print("=== レイヤーごとの個別処理モード ===")
for layer_idx, layer_name in enumerate(target_layers):
    print(f"\n--- レイヤー {layer_idx+1}/6: {layer_name} ---")

    try:
        # 単一レイヤーのGradCAMを作成
        saliency_method = YOLOV5GradCAM(model=model, layer_name=layer_name,
                                       img_size=input_size, method="gradcampp")

        # このレイヤーのみで全データを処理
        calculate_aoi(folder_path, csv_path, threshold, model,
                     [saliency_method], start_index=0, end_index=None)

        # メモリ解放
        del saliency_method
        if device == 'cuda':
            torch.cuda.empty_cache()
        gc.collect()

    except Exception as e:
        print(f"  レイヤー {layer_name} でエラー: {str(e)}")
        continue
"""

In [ ]:
### メモリ節約バージョン ###

import pandas as pd
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
from deep_utils import Box, split_extension
import os
from tqdm import tqdm
#from models.yolo_v5_object_detector import YOLOV5TorchObjectDetector
# from models.gradcam import YOLOV5GradCAM
import time
import torch
import torch.nn.functional as F
import gc

# PYTORCH_CUDA_ALLOC_CONFを設定
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# グローバル変数として定義（エラー回避のため）
global img_size
img_size = 640

class YOLOV5GradCAM:
    def __init__(self, model, layer_name, img_size=(640, 640), method="gradcam"):
        self.model = model
        self.gradients = dict()
        self.activations = dict()
        self.method = method
        self.cls_names = []

        def backward_hook(module, grad_input, grad_output):
            self.gradients["value"] = grad_output[0]
            return None

        def forward_hook(module, input, output):
            self.activations["value"] = output
            return None

        target_layer = find_yolo_layer(self.model, layer_name)
        target_layer.register_forward_hook(forward_hook)
        target_layer.register_backward_hook(backward_hook)

        # 初期化時のメモリ消費を抑えるため、ダミーのフォワードパスを削除
        # device = "cuda" if next(self.model.model.parameters()).is_cuda else "cpu"
        # self.model(torch.zeros(1, 3, *img_size, device=device))

    def forward(self, input_img, class_idx=True):
        saliency_maps = []
        b, c, h, w = input_img.size()

        preds, logits = self.model(input_img)

        # topkの計算は勾配不要なのでno_grad()を適用
        with torch.no_grad():
            _, top3_indices = torch.topk(logits[0], k=3)

        if top3_indices.numel() > 0:
            preds[1][0] = top3_indices.tolist()[0]
            preds[2][0] = [names[i] for i in preds[1][0]]
            self.cls_names = preds[2][0]
        else:
            self.cls_names = []

        if self.method == "eigencam":
            saliency_map = self._eigencam()
            saliency_maps.append(saliency_map)
        else:
            for cls, cls_name in zip(preds[1][0], preds[2][0]):
                if class_idx:
                    score = logits[0][0][cls]
                else:
                    score = logits[0][0].max()

                self.model.zero_grad()
                score.backward(retain_graph=True)
                gradients = self.gradients["value"]
                activations = self.activations["value"]
                b, k, u, v = gradients.size()

                if self.method == "gradcam":
                    weights = self._gradcam_weights(gradients, b, k)
                elif self.method == "gradcampp":
                    weights = self._gradcampp_weights(gradients, activations, score, b, k, u, v)

                # saliency_mapの計算は勾配不要なのでno_grad()を適用
                with torch.no_grad():
                    saliency_map = (weights * activations).sum(1, keepdim=True)
                    saliency_map = F.relu(saliency_map)
                    saliency_map = F.interpolate(saliency_map, size=(h, w), mode="bilinear", align_corners=False)
                    saliency_map_min, saliency_map_max = saliency_map.min(), saliency_map.max()
                    # ゼロ除算を避けるためにepsilonを追加
                    saliency_map = (saliency_map - saliency_map_min).div(saliency_map_max - saliency_map_min + 1e-8).data
                saliency_maps.append(saliency_map)

        return saliency_maps, logits, preds, self.cls_names

    def _gradcam_weights(self, gradients, b, k):
        alpha = gradients.view(b, k, -1).mean(2)
        weights = alpha.view(b, k, 1, 1)
        return weights

    def _gradcampp_weights(self, gradients, activations, score, b, k, u, v):
        alpha_num = gradients.pow(2)
        alpha_denom = gradients.pow(2).mul(2) + \
            activations.mul(gradients.pow(3)).view(b, k, u*v).sum(-1, keepdim=True).view(b, k, 1, 1)
        alpha_denom = torch.where(alpha_denom != 0.0, alpha_denom, torch.ones_like(alpha_denom))
        alpha = alpha_num.div(alpha_denom + 1e-7)

        relu_grad = F.relu(score.exp() * gradients)
        weights = (relu_grad * alpha).view(b, k, u*v).sum(-1).view(b, k, 1, 1)

        return weights

    def _eigencam(self):
        activations = self.activations["value"]
        b, k, u, v = activations.size()
        activations_reshaped = activations.view(b, k, -1)
        # Compute covariance matrix
        cov = activations_reshaped @ activations_reshaped.transpose(-1, -2)
        cov /= (u * v)
        # Compute eigenvectors
        eigenvalues, eigenvectors = torch.linalg.eigh(cov)
        # Select the eigenvector corresponding to the largest eigenvalue
        leading_eigenvector = eigenvectors[..., -1].unsqueeze(-1)
        # Compute EigenCAM
        eigen_cam = (activations_reshaped.transpose(-1, -2) @ leading_eigenvector).view(b, u, v)
        eigen_cam = F.relu(eigen_cam)
        # Normalize
        eigen_cam_min, eigen_cam_max = eigen_cam.min(), eigen_cam.max()
        eigen_cam = (eigen_cam - eigen_cam_min) / (eigen_cam_max - eigen_cam_min + 1e-8)
        # Resize to match the input image size
        eigen_cam = F.interpolate(eigen_cam.unsqueeze(1), size=(640, 640), mode="bilinear", align_corners=False)
        return eigen_cam

    def __call__(self, input_img):
        return self.forward(input_img)

def find_yolo_layer(model, layer_name):
   """Find yolov5 layer to calculate GradCAM and GradCAM++

   Args:
       model: yolov5 model.
       layer_name (str): the name of layer with its hierarchical information.

   Return:
       target_layer: found layer
   """
   hierarchy = layer_name.split("_")
   target_layer = model.model._modules[hierarchy[0]]

   for h in hierarchy[1:]:
       target_layer = target_layer._modules[h]
   return target_layer

def get_aoi(bbox, masks, threshold):
   total_intersect_pixels = 0  # 共通部分のピクセル数のカウンタを初期化

   for mask in masks:
       # マスクの前処理
       mask = mask.squeeze().mul(255).add_(0.5).clamp_(0, 255).detach().cpu().numpy().astype(np.uint8)
       # RuntimeWarning: invalid value encountered in cast 対策
       mask = np.nan_to_num(mask, nan=0.0).astype(np.uint8)

       mask[mask < threshold * 255] = 0
       mask[mask >= threshold * 255] = 255
       binary_mask = mask >= threshold * 255  # 閾値を超える部分を二値化

       # bboxの範囲内のマスク部分を取得
       x1, y1, x2, y2 = bbox
       mask_bbox = binary_mask[y1:y2, x1:x2]

       # 閾値を超える共通部分のピクセル数をカウント
       intersect_pixels = np.sum(mask_bbox)
       total_intersect_pixels += intersect_pixels

       # mask_bbox のピクセル数を取得
       mask_bbox_area = (y2 - y1) * (x2 - x1)

       # AOI を計算
       AOI = total_intersect_pixels / mask_bbox_area if mask_bbox_area > 0 else 0

   return AOI

def calculate_aoi(folder_path, csv_path, threshold, model, target_layers, start_index=0, end_index=None):
    df = pd.read_csv(csv_path)
    df = df.loc[:, ~df.columns.str.contains("^Unnamed")]

    # 全レイヤーの設定
    all_layers_cols = [
        ("model_17_cv3_conv", f"AOI_{threshold}_layermodel_17_cv3_conv"),
        ("model_20_cv3_conv", f"AOI_{threshold}_layermodel_20_cv3_conv"),
        ("model_23_cv3_conv", f"AOI_{threshold}_layermodel_23_cv3_conv"),
        ("model_24_m_0", f"AOI_{threshold}_layer24_m_0"),
        ("model_24_m_1", f"AOI_{threshold}_layer24_m_1"),
        ("model_24_m_2", f"AOI_{threshold}_layer24_m_2")
    ]

    # 列が存在しない場合は作成
    for layer_name, col_name in all_layers_cols:
        if col_name not in df.columns:
            df[col_name] = None

    if end_index is None:
        end_index = len(df)

    # 実際に処理された画像の数をカウント
    processed_count = 0
    not_found_count = 0
    error_count = 0

    for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing images"):
        if index < start_index or index >= end_index:
            continue

        img_basename = row["image_basename"]

        if pd.isna(img_basename):
            print(f"Skipping row {index} due to NaN image_basename")
            continue

        # 拡張子を試す
        img_found = False
        img_path = None

        for ext in [".jpg", ".jpeg", ".png", ".JPG", ".JPEG", ".PNG", ""]:
            test_path = os.path.join(folder_path, f"{img_basename}{ext}")
            if os.path.exists(test_path):
                img_path = test_path
                img_found = True
                break

        if not img_found:
            test_path = os.path.join(folder_path, img_basename)
            if os.path.exists(test_path):
                img_path = test_path
                img_found = True

        if img_found:
            img = cv2.imread(img_path)

            if img is None:
                print(f"Failed to read image: {img_path}")
                continue

            try:
                torch_img = model.preprocessing(img[..., ::-1])

                # レイヤーごとに処理
                for layer_name, col_name in all_layers_cols:
                    try:
                        # レイヤーごとにYOLOV5GradCAMインスタンスを生成・破棄
                        saliency_method = YOLOV5GradCAM(model=model, layer_name=layer_name, img_size=input_size, method="gradcampp")
                        masks, logits, [boxes, _, _, _], cls_names = saliency_method(torch_img)

                        if len(masks) > 0 and len(boxes) > 0:
                            mask = masks[0][0]
                            bbox = boxes[0][0]

                            aoi = get_aoi(bbox, [mask], threshold)
                            df.at[index, col_name] = aoi

                        # メモリ解放
                        del saliency_method
                        if device == "cuda":
                            torch.cuda.empty_cache()
                        gc.collect()

                    except Exception as e:
                        error_count += 1
                        if error_count <= 5:
                            print(f"Error processing image {img_basename} at layer {layer_name}: {str(e)}")

                processed_count += 1
                # 進捗表示
                if processed_count % 50 == 0:
                    print(f"Processed {processed_count} images...")

            except Exception as e:
                error_count += 1
                if error_count <= 5:
                    print(f"Error preprocessing image {img_basename}: {str(e)}")
        else:
            not_found_count += 1
            if not_found_count <= 5:
                print(f"Image not found: {img_basename}")

    print(f"\n処理完了: {processed_count}個の画像を処理")
    print(f"見つからなかった画像: {not_found_count}個")
    print(f"エラーが発生した画像: {error_count}個")

    df.to_csv(csv_path, index=False)
    print(f"結果を保存しました: {csv_path}")

# メイン実行部分
folder_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/Ueno_Mix1039/Images"
csv_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/Ueno_Mix1039/Ueno_Mix1039_gradcam++.csv"

threshold = 0.5

# モデルとsaliency_methodの定義
model_path = "/gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forcresco/weights/eye_nii_2202_onecaseoneimage2_doctorcompare_yolov5s_epoch200_batch16_89.8p/last.pt"
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"使用デバイス: {device}")
if device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")

input_size = (640, 640)
names = ["infection","normal","non-infection","scar","tumor","deposit","APAC","lens opacity","bullous"]

model = YOLOV5TorchObjectDetector(model_path, device, img_size=input_size, names=names)

# 全レイヤーのtarget_layers（convに修正が必要な場合）
target_layers = [
    "model_17_cv3_conv",  # actからconvに変更
    "model_20_cv3_conv",  # actからconvに変更
    "model_23_cv3_conv",  # actからconvに変更
    "model_24_m_0",
    "model_24_m_1",
    "model_24_m_2"
]

# 実行モード選択
USE_BATCH_MODE = False  # バッチ処理モードを使用するかどうか
BATCH_SIZE = 20       # バッチサイズ

if USE_BATCH_MODE:
    print("=== バッチ処理モードで実行します ===")

    # CSVファイルの総行数を確認
    df_check = pd.read_csv(csv_path)
    total_images = len(df_check)
    print(f"総画像数: {total_images}")

    # calculate_aoi関数内でレイヤーごとにインスタンスを生成するため、ここではsaliency_methodsを生成しない
    saliency_methods = [] # 空のリストとして定義

    print("\n解析を開始します...")
    # バッチ処理実行
    for batch_start in range(0, total_images, BATCH_SIZE):
        batch_end = min(batch_start + BATCH_SIZE, total_images)
        print(f"\n=== バッチ {batch_start//BATCH_SIZE + 1}/{(total_images-1)//BATCH_SIZE + 1}: "
              f"画像 {batch_start+1}-{batch_end}/{total_images} ===")

        calculate_aoi(folder_path, csv_path, threshold, model, target_layers,
                     start_index=batch_start, end_index=batch_end)

        # メモリ解放
        if device == "cuda":
            torch.cuda.empty_cache()
        gc.collect()

        print(f"バッチ {batch_start//BATCH_SIZE + 1} 完了")

    print("\n=== 全バッチ処理完了 ===")

else:
    print("=== 通常モードで実行します ===")

    # 処理範囲の設定
    start_index = 30
    end_index = 100  # Noneで全データ処理

    # calculate_aoi関数内でレイヤーごとにインスタンスを生成するため、ここではsaliency_methodsを生成しない
    saliency_methods = [] # 空のリストとして定義

    print("\n解析を開始します...")

    calculate_aoi(folder_path, csv_path, threshold, model, target_layers, start_index, end_index)





In [ ]:
# show_result
import pandas as pd

# CSVファイルのパス
csv_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/maehara_slit_total_updated.csv"

# DataFrameとしてCSVファイルを読み込む
df = pd.read_csv(csv_path)

# DataFrameの最初の数行を表示する
print(df.head())


#**注目点色塗り**

In [ ]:
#注目点に色を塗る（反転あり）
import os
import time
import numpy as np
import cv2
import torch
import torch.nn.functional as F
from google.colab.patches import cv2_imshow
from deep_utils import Box, split_extension
import gc

# パラメータ
model_path = "/gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forcresco/weights/eye_nii_2202_onecaseoneimage2_doctorcompare_yolov5s_epoch200_batch16_89.8p/last.pt"
img_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/スマホ_serial/99.jpg"
output_dir = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/GradCam_img_slit'
img_size = 640
target_layer = 'model_23_cv3_act'
method = 'gradcam'
device = 'cpu'
names = ["infection","normal","non-infection","scar","tumor","deposit","APAC","lens opacity","bullous"]

def get_res_img(bbox, masks, res_img):
    for mask in masks:
        mask = mask.squeeze().mul(255).add_(0.5).clamp_(0, 255).detach().cpu().numpy().astype(np.uint8)
        heatmap = cv2.applyColorMap(mask, cv2.COLORMAP_JET)
        n_heatmat = (Box.fill_outer_box(heatmap, bbox) / 255).astype(np.float32)
        res_img = res_img / 255
        res_img = cv2.add(res_img, n_heatmat)
        res_img = (res_img / res_img.max())
    return res_img, n_heatmat

def put_text_box(bbox, cls_name, res_img):
    x1, y1, x2, y2 = bbox
    cv2.imwrite('temp.jpg', (res_img * 255).astype(np.uint8))
    res_img = cv2.imread('temp.jpg')
    res_img = Box.put_box(res_img, bbox)

    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 2.0
    color = (0, 255, 0)
    thickness = 2

    text_size, _ = cv2.getTextSize(cls_name, font, font_scale, thickness)
    text_x = 10
    text_y = text_size[1] + 10

    cv2.putText(res_img, cls_name, (text_x, text_y), font, font_scale, color, thickness)

    return res_img

def find_yolo_layer(model, layer_name):
    hierarchy = layer_name.split('_')
    target_layer = model.model._modules[hierarchy[0]]

    for h in hierarchy[1:]:
        target_layer = target_layer._modules[h]
    return target_layer

class YOLOV5GradCAM:
    def __init__(self, model, layer_name, img_size=(640, 640)):
        self.model = model
        self.gradients = dict()
        self.activations = dict()
        self.cls_names = []

        def backward_hook(module, grad_input, grad_output):
            self.gradients['value'] = grad_output[0]
            return None

        def forward_hook(module, input, output):
            self.activations['value'] = output
            return None

        target_layer = find_yolo_layer(self.model, layer_name)
        target_layer.register_forward_hook(forward_hook)
        target_layer.register_backward_hook(backward_hook)

        device = 'cuda' if next(self.model.model.parameters()).is_cuda else 'cpu'
        self.model(torch.zeros(1, 3, *img_size, device=device))

    def forward(self, input_img, class_idx=True):
        saliency_maps = []
        b, c, h, w = input_img.size()
        preds, logits = self.model(input_img)
        _, top1_idx = torch.topk(logits[0], k=1)

        if top1_idx.numel() > 0:
            preds[1][0] = top1_idx.tolist()[0]
            preds[2][0] = [names[i] for i in preds[1][0]]
            self.cls_names = preds[2][0]
        else:
            self.cls_names = []

        cls, cls_name = preds[1][0][0], preds[2][0][0]
        if class_idx:
            score = logits[0][0][cls]
        else:
            score = logits[0][0].max()
        self.model.zero_grad()
        score.backward(retain_graph=True)
        gradients = self.gradients['value']
        activations = self.activations['value']
        b, k, u, v = gradients.size()
        alpha = gradients.view(b, k, -1).mean(2)
        weights = alpha.view(b, k, 1, 1)
        saliency_map = (weights * activations).sum(1, keepdim=True)
        saliency_map = F.relu(saliency_map)
        saliency_map = F.interpolate(saliency_map, size=(h, w), mode='bilinear', align_corners=False)
        saliency_map_min, saliency_map_max = saliency_map.min(), saliency_map.max()
        saliency_map = (saliency_map - saliency_map_min).div(saliency_map_max - saliency_map_min).data
        saliency_maps.append(saliency_map)

        return saliency_maps, logits, preds, self.cls_names

    def __call__(self, input_img):
        return self.forward(input_img)

def main(img_path, threshold=0.5):
    input_size = (img_size, img_size)
    img = cv2.imread(img_path)
    model = YOLOV5TorchObjectDetector(model_path, device, img_size=input_size, names=names)
    torch_img = model.preprocessing(img[..., ::-1])
    if method == 'gradcam':
        saliency_method = YOLOV5GradCAM(model=model, layer_name=target_layer, img_size=input_size)
    masks, logits, [boxes, _, _, _], cls_names = saliency_method(torch_img)
    result = torch_img.squeeze(0).mul(255).add_(0.5).clamp_(0, 255).permute(1, 2, 0).detach().cpu().numpy()
    result = result[..., ::-1]

    if masks:
        mask = masks[0].squeeze().cpu().numpy()
        if mask.max() >= threshold:
            binary_mask = np.where(mask >= threshold, 1, 0).astype(np.uint8)
            black_mask = np.where(mask >= threshold, 0, 1).astype(np.uint8)
            binary_mask = cv2.resize(binary_mask, (result.shape[1], result.shape[0]))
            black_mask = cv2.resize(black_mask, (result.shape[1], result.shape[0]))

            color = np.array([0,0,0], dtype=np.uint8) #マスクの色：白は[0,0,0]
            masked_res_img = result * binary_mask[..., np.newaxis] + (1 - binary_mask[..., np.newaxis]) * color
            masked_res_img = (masked_res_img / masked_res_img.max()) * 255

            black_masked_res_img = result * black_mask[..., np.newaxis] + (1 - black_mask[..., np.newaxis]) * color
            black_masked_res_img = (black_masked_res_img / black_masked_res_img.max()) * 255

            cv2_imshow(masked_res_img.astype(np.uint8))
            cv2_imshow(black_masked_res_img.astype(np.uint8))

if __name__ == '__main__':
    target_layer = 'model_23_cv3_conv'
    img_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/フォトスリット_serial/8.jpg"
    output_dir = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/GradCam_img_slit'
    threshold = 0.1  # ここで閾値を指定
    if os.path.isdir(img_path):
        folder_main(img_path, threshold)
    else:
        main(img_path, threshold)


In [ ]:
# 注目点をblurする（反転あり）
import os
import time
import numpy as np
import cv2
import torch
import torch.nn.functional as F
from google.colab.patches import cv2_imshow
from deep_utils import Box, split_extension
import gc

# パラメータ
model_path = "/gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forcresco/weights/eye_nii_2202_onecaseoneimage2_doctorcompare_yolov5s_epoch200_batch16_89.8p/last.pt"
img_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/スマホ_serial/99.jpg"
output_dir = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/GradCam_img_slit'
img_size = 640
target_layer = 'model_23_cv3_act'
method = 'gradcam'
device = 'cpu'
names = ["infection","normal","non-infection","scar","tumor","deposit","APAC","lens opacity","bullous"]

def get_res_img(bbox, masks, res_img):
    for mask in masks:
        mask = mask.squeeze().mul(255).add_(0.5).clamp_(0, 255).detach().cpu().numpy().astype(np.uint8)
        heatmap = cv2.applyColorMap(mask, cv2.COLORMAP_JET)
        n_heatmat = (Box.fill_outer_box(heatmap, bbox) / 255).astype(np.float32)
        res_img = res_img / 255
        res_img = cv2.add(res_img, n_heatmat)
        res_img = (res_img / res_img.max())
    return res_img, n_heatmat

def put_text_box(bbox, cls_name, res_img):
    x1, y1, x2, y2 = bbox
    cv2.imwrite('temp.jpg', (res_img * 255).astype(np.uint8))
    res_img = cv2.imread('temp.jpg')
    res_img = Box.put_box(res_img, bbox)

    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 2.0
    color = (0, 255, 0)
    thickness = 2

    text_size, _ = cv2.getTextSize(cls_name, font, font_scale, thickness)
    text_x = 10
    text_y = text_size[1] + 10

    cv2.putText(res_img, cls_name, (text_x, text_y), font, font_scale, color, thickness)

    return res_img

def find_yolo_layer(model, layer_name):
    hierarchy = layer_name.split('_')
    target_layer = model.model._modules[hierarchy[0]]

    for h in hierarchy[1:]:
        target_layer = target_layer._modules[h]
    return target_layer

class YOLOV5GradCAM:
    def __init__(self, model, layer_name, img_size=(640, 640)):
        self.model = model
        self.gradients = dict()
        self.activations = dict()
        self.cls_names = []

        def backward_hook(module, grad_input, grad_output):
            self.gradients['value'] = grad_output[0]
            return None

        def forward_hook(module, input, output):
            self.activations['value'] = output
            return None

        target_layer = find_yolo_layer(self.model, layer_name)
        target_layer.register_forward_hook(forward_hook)
        target_layer.register_backward_hook(backward_hook)

        device = 'cuda' if next(self.model.model.parameters()).is_cuda else 'cpu'
        self.model(torch.zeros(1, 3, *img_size, device=device))

    def forward(self, input_img, class_idx=True):
        saliency_maps = []
        b, c, h, w = input_img.size()
        preds, logits = self.model(input_img)
        _, top1_idx = torch.topk(logits[0], k=1)

        if top1_idx.numel() > 0:
            preds[1][0] = top1_idx.tolist()[0]
            preds[2][0] = [names[i] for i in preds[1][0]]
            self.cls_names = preds[2][0]
        else:
            self.cls_names = []

        cls, cls_name = preds[1][0][0], preds[2][0][0]
        if class_idx:
            score = logits[0][0][cls]
        else:
            score = logits[0][0].max()
        self.model.zero_grad()
        score.backward(retain_graph=True)
        gradients = self.gradients['value']
        activations = self.activations['value']
        b, k, u, v = gradients.size()
        alpha = gradients.view(b, k, -1).mean(2)
        weights = alpha.view(b, k, 1, 1)
        saliency_map = (weights * activations).sum(1, keepdim=True)
        saliency_map = F.relu(saliency_map)
        saliency_map = F.interpolate(saliency_map, size=(h, w), mode='bilinear', align_corners=False)
        saliency_map_min, saliency_map_max = saliency_map.min(), saliency_map.max()
        saliency_map = (saliency_map - saliency_map_min).div(saliency_map_max - saliency_map_min).data
        saliency_maps.append(saliency_map)

        return saliency_maps, logits, preds, self.cls_names

    def __call__(self, input_img):
        return self.forward(input_img)

def main(img_path, threshold=0.5):
    input_size = (img_size, img_size)
    img = cv2.imread(img_path)
    model = YOLOV5TorchObjectDetector(model_path, device, img_size=input_size, names=names)
    torch_img = model.preprocessing(img[..., ::-1])
    if method == 'gradcam':
        saliency_method = YOLOV5GradCAM(model=model, layer_name=target_layer, img_size=input_size)
    masks, logits, [boxes, _, _, _], cls_names = saliency_method(torch_img)
    result = torch_img.squeeze(0).mul(255).add_(0.5).clamp_(0, 255).permute(1, 2, 0).detach().cpu().numpy()
    result = result[..., ::-1]

    if masks:
        mask = masks[0].squeeze().cpu().numpy()
        if mask.max() >= threshold:
            binary_mask = np.where(mask >= threshold, 1, 0).astype(np.uint8)
            black_mask = np.where(mask >= threshold, 0, 1).astype(np.uint8)
            binary_mask = cv2.resize(binary_mask, (result.shape[1], result.shape[0]))
            black_mask = cv2.resize(black_mask, (result.shape[1], result.shape[0]))

            # ぼかし効果を適用
            blurred_result = cv2.GaussianBlur(result, (91, 91), 0)
            masked_res_img = result * binary_mask[..., np.newaxis] + blurred_result * (1 - binary_mask[..., np.newaxis])
            masked_res_img = (masked_res_img / masked_res_img.max()) * 255

            black_blurred_result = cv2.GaussianBlur(result, (91, 91), 0)
            black_masked_res_img = result * black_mask[..., np.newaxis] + black_blurred_result * (1 - black_mask[..., np.newaxis])
            black_masked_res_img = (black_masked_res_img / black_masked_res_img.max()) * 255

            cv2_imshow(masked_res_img.astype(np.uint8))
            cv2_imshow(black_masked_res_img.astype(np.uint8))

if __name__ == '__main__':
    target_layer = 'model_23_cv3_conv'
    img_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/フォトスリット_serial/41.jpg"
    output_dir = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/GradCam_img_slit'
    threshold = 0.3  # ここで閾値を指定
    if os.path.isdir(img_path):
        folder_main(img_path, threshold)
    else:
        main(img_path, threshold)


#**Analyze results**

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the CSV files
file_path1 = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/maehara_slit_total_updated.csv'
file_path2 = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/maehara_sumaho_total_updated.csv'

df_slit = pd.read_csv(file_path1)
df_sumaho = pd.read_csv(file_path2)

# Define class names as shown in the image
class_names = ["Normal", "Infectious keratitis", "Non-infection keratitis", "Scar", "Tumor", "Deposit", "APAC", "Lens opacity", "Bullous keratopathy"]

# Create confusion matrices for Slit and Sumaho datasets
conf_matrix_slit = pd.crosstab(df_slit['class_num'], df_slit['top1'])
conf_matrix_sumaho = pd.crosstab(df_sumaho['class_num'], df_sumaho['top1'])

# Rename the index and columns for better readability
conf_matrix_slit.index = class_names
conf_matrix_slit.columns = class_names
conf_matrix_sumaho.index = class_names
conf_matrix_sumaho.columns = class_names

# Create a figure with higher DPI
plt.figure(figsize=(15, 6), dpi=350)  # 横幅を少し広げました

# Slit Lamp Data Confusion Matrix
plt.subplot(1, 2, 1)
sns.heatmap(conf_matrix_slit, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix for Slit Lamp Data')
plt.xlabel('Predicted Class')
plt.ylabel('Actual Class')
plt.xticks(rotation=45, ha='right')  # 横軸のラベルを45度傾ける

# Smartphone Data Confusion Matrix
plt.subplot(1, 2, 2)
sns.heatmap(conf_matrix_sumaho, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix for Smartphone Data')
plt.xlabel('Predicted Class')
plt.ylabel('Actual Class')
plt.xticks(rotation=45, ha='right')  # 横軸のラベルを45度傾ける

plt.tight_layout()

# Save the figure at 350 DPI
plt.savefig('confusion_matrices_350dpi.png', dpi=350, bbox_inches='tight')
plt.show()

In [ ]:
# Calculate accuracy for Slit and Sumaho datasets
accuracy_slit = (conf_matrix_slit.values.diagonal().sum() / conf_matrix_slit.values.sum()) * 100
accuracy_sumaho = (conf_matrix_sumaho.values.diagonal().sum() / conf_matrix_sumaho.values.sum()) * 100

accuracy_slit, accuracy_sumaho


In [ ]:
#@title スマホvsスリット (GradCAM++)
import pandas as pd
import numpy as np

# ファイルパスを指定
file_path1 = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/maehara_slit_total_updated.csv'
file_path2 = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/maehara_sumaho_total_updated.csv'

# CSVファイルを読み込む
df_slit = pd.read_csv(file_path1)
df_sumaho = pd.read_csv(file_path2)

# クラス名を定義
class_names = ["infection", "normal", "non-infection", "scar", "tumor", "deposit", "APAC", "lens opacity", "bullous"]

# 列名のリストを作成
columns_of_interest = ['AOI_0.5_layer17', 'AOI_0.5_layer20', 'AOI_0.5_layer23', 'AOI_0.5_layer24_m_0', 'AOI_0.5_layer24_m_1', 'AOI_0.5_layer24_m_2']

# 両データセットの統計を計算して表示
for df, device_name in [(df_slit, "Slit Lamp"), (df_sumaho, "Smartphone")]:
    print(f"\n============= {device_name} データの統計値 =============")

    # 各レイヤーの詳細な統計量を計算
    for column in columns_of_interest:
        stats = df[column].describe()
        percentiles = df[column].quantile([0.1, 0.25, 0.75, 0.9])

        print(f"\n------- {column} の統計値 -------")
        print(f"データ数: {int(stats['count'])}")
        print(f"平均値: {stats['mean']:.6f}")
        print(f"標準偏差: {stats['std']:.6f}")
        print(f"最小値: {stats['min']:.6f}")
        print(f"10パーセンタイル: {percentiles[0.1]:.6f}")
        print(f"25パーセンタイル: {stats['25%']:.6f}")
        print(f"中央値: {stats['50%']:.6f}")
        print(f"75パーセンタイル: {stats['75%']:.6f}")
        print(f"90パーセンタイル: {percentiles[0.9]:.6f}")
        print(f"最大値: {stats['max']:.6f}")

        # 歪度と尖度も計算
        skewness = df[column].skew()
        kurtosis = df[column].kurtosis()
        print(f"歪度: {skewness:.6f}")
        print(f"尖度: {kurtosis:.6f}")

    # 3つのレイヤーの合計値の統計を計算
    layer_sum = df[columns_of_interest].sum(axis=1)
    sum_stats = layer_sum.describe()
    sum_percentiles = layer_sum.quantile([0.1, 0.9])
    sum_skewness = layer_sum.skew()
    sum_kurtosis = layer_sum.kurtosis()

    print(f"\n------- 3レイヤーの合計値の統計 -------")
    print(f"データ数: {int(sum_stats['count'])}")
    print(f"平均値: {sum_stats['mean']:.6f}")
    print(f"標準偏差: {sum_stats['std']:.6f}")
    print(f"最小値: {sum_stats['min']:.6f}")
    print(f"10パーセンタイル: {sum_percentiles[0.1]:.6f}")
    print(f"25パーセンタイル: {sum_stats['25%']:.6f}")
    print(f"中央値: {sum_stats['50%']:.6f}")
    print(f"75パーセンタイル: {sum_stats['75%']:.6f}")
    print(f"90パーセンタイル: {sum_percentiles[0.9]:.6f}")
    print(f"最大値: {sum_stats['max']:.6f}")

    # 相関係数の計算
    print(f"\n------- レイヤー間の相関係数 -------")
    correlation_matrix = df[columns_of_interest].corr()
    print(correlation_matrix.round(6))

    print("\n" + "="*50)  # セパレータの表示

In [ ]:
import pandas as pd

# ファイルパスを指定
file_path1 = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/maehara_slit_total_updated.csv'
file_path2 = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/maehara_sumaho_total_updated.csv'

# CSVファイルを読み込む
df_slit = pd.read_csv(file_path1)
df_sumaho = pd.read_csv(file_path2)

# クラス名を定義
class_names = ["infection", "normal", "non-infection", "scar", "tumor", "deposit", "APAC", "lens opacity", "bullous"]

# 列名のリストを作成
columns_of_interest = ['AOI_0.5_layer24_m_0', 'AOI_0.5_layer24_m_1', 'AOI_0.5_layer24_m_2']

# AOIの平均と標準偏差を計算
mean_sd_aoi_slit = df_slit.groupby('top1')[columns_of_interest].agg(['mean', 'std']).reindex(range(len(class_names))).fillna(0)
mean_sd_aoi_sumaho = df_sumaho.groupby('top1')[columns_of_interest].agg(['mean', 'std']).reindex(range(len(class_names))).fillna(0)

# インデックスにクラス名を設定
mean_sd_aoi_slit.index = class_names
mean_sd_aoi_sumaho.index = class_names

# 結果を表示
print("Mean ± SD AOI for Slit Lamp Data")
print(mean_sd_aoi_slit)
print("\nMean ± SD AOI for Smartphone Data")
print(mean_sd_aoi_sumaho)


In [ ]:
#@title スリット（GradCAM vs GradCAM++）
import pandas as pd
import numpy as np

# ファイルパスを指定
file_path1 = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/maehara_slit_total_gradcam.csv'
file_path2 = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/maehara_slit_total_gradcam++.csv'

# CSVファイルを読み込む
df_slit = pd.read_csv(file_path1)
df_sumaho = pd.read_csv(file_path2)

# クラス名を定義
class_names = ["infection", "normal", "non-infection", "scar", "tumor", "deposit", "APAC", "lens opacity", "bullous"]

# 列名のリストを作成
columns_of_interest = ['AOI_0.5_layer17', 'AOI_0.5_layer20', 'AOI_0.5_layer23', 'AOI_0.5_layer24_m_0', 'AOI_0.5_layer24_m_1', 'AOI_0.5_layer24_m_2']

# 両データセットの統計を計算して表示
for df, device_name in [(df_slit, "GradCAM"), (df_sumaho, "GradCAM++")]:
    print(f"\n============= {device_name} データの統計値 =============")

    # 各レイヤーの詳細な統計量を計算
    for column in columns_of_interest:
        stats = df[column].describe()
        percentiles = df[column].quantile([0.1, 0.25, 0.75, 0.9])

        print(f"\n------- {column} の統計値 -------")
        print(f"データ数: {int(stats['count'])}")
        print(f"平均値: {stats['mean']:.6f}")
        print(f"標準偏差: {stats['std']:.6f}")
        print(f"最小値: {stats['min']:.6f}")
        print(f"10パーセンタイル: {percentiles[0.1]:.6f}")
        print(f"25パーセンタイル: {stats['25%']:.6f}")
        print(f"中央値: {stats['50%']:.6f}")
        print(f"75パーセンタイル: {stats['75%']:.6f}")
        print(f"90パーセンタイル: {percentiles[0.9]:.6f}")
        print(f"最大値: {stats['max']:.6f}")

        # 歪度と尖度も計算
        skewness = df[column].skew()
        kurtosis = df[column].kurtosis()
        print(f"歪度: {skewness:.6f}")
        print(f"尖度: {kurtosis:.6f}")

    # 3つのレイヤーの合計値の統計を計算
    layer_sum = df[columns_of_interest].sum(axis=1)
    sum_stats = layer_sum.describe()
    sum_percentiles = layer_sum.quantile([0.1, 0.9])
    sum_skewness = layer_sum.skew()
    sum_kurtosis = layer_sum.kurtosis()

    print(f"\n------- 3レイヤーの合計値の統計 -------")
    print(f"データ数: {int(sum_stats['count'])}")
    print(f"平均値: {sum_stats['mean']:.6f}")
    print(f"標準偏差: {sum_stats['std']:.6f}")
    print(f"最小値: {sum_stats['min']:.6f}")
    print(f"10パーセンタイル: {sum_percentiles[0.1]:.6f}")
    print(f"25パーセンタイル: {sum_stats['25%']:.6f}")
    print(f"中央値: {sum_stats['50%']:.6f}")
    print(f"75パーセンタイル: {sum_stats['75%']:.6f}")
    print(f"90パーセンタイル: {sum_percentiles[0.9]:.6f}")
    print(f"最大値: {sum_stats['max']:.6f}")

    # 相関係数の計算
    print(f"\n------- レイヤー間の相関係数 -------")
    correlation_matrix = df[columns_of_interest].corr()
    print(correlation_matrix.round(6))

    print("\n" + "="*50)  # セパレータの表示

In [ ]:
import pandas as pd

# ファイルパスを指定
file_path1 = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/maehara_slit_total_gradcam.csv'
file_path2 = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/maehara_slit_total_gradcam++.csv'

# CSVファイルを読み込む
df_slit = pd.read_csv(file_path1)
df_sumaho = pd.read_csv(file_path2)

# クラス名を定義
class_names = ["infection", "normal", "non-infection", "scar", "tumor", "deposit", "APAC", "lens opacity", "bullous"]

# 列名のリストを作成
columns_of_interest = ['AOI_0.5_layer17', 'AOI_0.5_layer20', 'AOI_0.5_layer23', 'AOI_0.5_layer24_m_0', 'AOI_0.5_layer24_m_1', 'AOI_0.5_layer24_m_2']

# AOIの平均と標準偏差を計算
mean_sd_aoi_gradcam = df_slit.groupby('top1')[columns_of_interest].agg(['mean', 'std']).reindex(range(len(class_names))).fillna(0)
mean_sd_aoi_gradcampp = df_sumaho.groupby('top1')[columns_of_interest].agg(['mean', 'std']).reindex(range(len(class_names))).fillna(0)

# インデックスにクラス名を設定
mean_sd_aoi_gradcam.index = class_names
mean_sd_aoi_gradcampp.index = class_names

# 結果を表示
print("Mean ± SD AOI for Slit Lamp Data")
print(mean_sd_aoi_gradcam)
print("\nMean ± SD AOI for Smartphone Data")
print(mean_sd_aoi_gradcampp)

##**Layerごとに解析**

In [ ]:
#@title スリットvsスマホ

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# フォントサイズとスタイルの設定
plt.rcParams.update({
    'font.size': 12,
    'axes.labelsize': 16,
    'axes.titlesize': 16,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
    'legend.fontsize': 12,
    'figure.titlesize': 18
})

####################
layer = "23"
####################
layer_name = f"AOI_0.5_layer{layer}"

# ファイルパスを指定
file_path1 = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/maehara_slit_total_updated.csv'
file_path2 = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/maehara_sumaho_total_updated.csv'

# CSVファイルを読み込む
df_slit = pd.read_csv(file_path1)
df_sumaho = pd.read_csv(file_path2)

# クラス名を定義
class_names = ["infection", "normal", "non-infection", "scar", "tumor", "deposit", "APAC", "lens opacity", "bullous"]

# クラス名を数字に対応させる
df_slit['class_name'] = df_slit['top1'].map(dict(enumerate(class_names)))
df_sumaho['class_name'] = df_sumaho['top1'].map(dict(enumerate(class_names)))

# データフレームを整形してSeabornでプロットできるようにする
df_slit['Type'] = 'Slit'
df_sumaho['Type'] = 'Sumaho'

# 必要なカラムだけを選択し、欠損値を削除して結合
df_combined = pd.concat([
    df_slit[['class_name', layer_name, 'Type']].dropna(),
    df_sumaho[['class_name', layer_name, 'Type']].dropna()
]).reset_index(drop=True)

# 合算データを作成
df_total = pd.concat([df_slit, df_sumaho])
df_total = df_total[['class_name', layer_name]].dropna()

# 青とオレンジのカラーパレットを定義
colors = ['#1f77b4', '#ff7f0e']

# グラフ1: スリットランプとスマートフォンのデータ
plt.figure(figsize=(12, 8))
sns.boxplot(x='class_name', y=layer_name, hue='Type', data=df_combined, order=class_names, palette=colors)
plt.xlabel('Class', fontsize=14, labelpad=10)
plt.ylabel(layer_name, fontsize=14, labelpad=10)
plt.title('Box Plot of AOI_0.5 for Slit Lamp and Smartphone Data', fontsize=16, pad=20)
plt.xticks(rotation=45, ha='right')
legend = plt.legend(title='Type', bbox_to_anchor=(1.05, 1), loc='upper left')
legend.get_title().set_fontsize(14)
plt.tight_layout()
plt.show()

# グラフ2: 合算データ
plt.figure(figsize=(12, 8))
sns.boxplot(x='class_name', y=layer_name, data=df_total, order=class_names, color='#1f77b4')
plt.xlabel('Class', fontsize=14, labelpad=10)
plt.ylabel(layer_name, fontsize=14, labelpad=10)
plt.title('Box Plot of AOI_0.5 for Combined Data', fontsize=16, pad=20)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# グラフ3: slitとsumahoの全体比較
plt.figure(figsize=(8, 6))
sns.boxplot(x='Type', y=layer_name, data=df_combined, palette=colors)
plt.xlabel('Type', fontsize=14, labelpad=10)
plt.ylabel(layer_name, fontsize=14, labelpad=10)
plt.title('Comparison of AOI_0.5 between Slit Lamp and Smartphone', fontsize=16, pad=20)
plt.tight_layout()
plt.show()

In [ ]:
#@title GradCAM vs GradCAM++ (スリット)

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# フォントサイズとスタイルの設定
plt.rcParams.update({
    'font.size': 12,
    'axes.labelsize': 16,
    'axes.titlesize': 16,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
    'legend.fontsize': 12,
    'figure.titlesize': 18
})

####################
layer = "23"
####################
layer_name = f"AOI_0.5_layer{layer}"

# ファイルパスを指定
file_path1 = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/maehara_slit_total_gradcam.csv'
file_path2 = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/maehara_slit_total_gradcam++.csv'

# CSVファイルを読み込む
df_slit = pd.read_csv(file_path1)
df_sumaho = pd.read_csv(file_path2)

# クラス名を定義
class_names = ["infection", "normal", "non-infection", "scar", "tumor", "deposit", "APAC", "lens opacity", "bullous"]

# クラス名を数字に対応させる
df_slit['class_name'] = df_slit['top1'].map(dict(enumerate(class_names)))
df_sumaho['class_name'] = df_sumaho['top1'].map(dict(enumerate(class_names)))

# データフレームを整形してSeabornでプロットできるようにする
df_slit['Type'] = 'GradCAM'
df_sumaho['Type'] = 'GradCAM++'

# 必要なカラムだけを選択し、欠損値を削除して結合
df_combined = pd.concat([
    df_slit[['class_name', layer_name, 'Type']].dropna(),
    df_sumaho[['class_name', layer_name, 'Type']].dropna()
]).reset_index(drop=True)

# 合算データを作成
df_total = pd.concat([df_slit, df_sumaho])
df_total = df_total[['class_name', layer_name]].dropna()

# 青とオレンジのカラーパレットを定義
colors = ['#1f77b4', '#ff7f0e']

# グラフ1: スリットランプとスマートフォンのデータ
plt.figure(figsize=(12, 8))
sns.boxplot(x='class_name', y=layer_name, hue='Type', data=df_combined, order=class_names, palette=colors)
plt.xlabel('Class', fontsize=14, labelpad=10)
plt.ylabel(layer_name, fontsize=14, labelpad=10)
plt.title('Box Plot of AOI_0.5 for Slit Lamp Data', fontsize=16, pad=20)
plt.xticks(rotation=45, ha='right')
legend = plt.legend(title='Type', bbox_to_anchor=(1.05, 1), loc='upper left')
legend.get_title().set_fontsize(14)
plt.tight_layout()
plt.savefig('gradcam_overall_comparison.png', dpi=350, bbox_inches='tight')
plt.show()

# グラフ2: 合算データ
plt.figure(figsize=(12, 8))
sns.boxplot(x='class_name', y=layer_name, data=df_total, order=class_names, color='#1f77b4')
plt.xlabel('Class', fontsize=14, labelpad=10)
plt.ylabel(layer_name, fontsize=14, labelpad=10)
plt.title('Box Plot of AOI_0.5 for Combined Data', fontsize=16, pad=20)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# グラフ3: slitとsumahoの全体比較
plt.figure(figsize=(8, 6))
sns.boxplot(x='Type', y=layer_name, data=df_combined, palette=colors)
plt.xlabel('Type', fontsize=14, labelpad=10)
plt.ylabel(layer_name, fontsize=14, labelpad=10)
plt.title('Comparison of AOI_0.5 between GradCAM and GradCAM++', fontsize=16, pad=20)
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# フォントサイズとスタイルの設定
plt.rcParams.update({
    'font.size': 14,
    'axes.labelsize': 18,
    'axes.titlesize': 20,
    'xtick.labelsize': 14,
    'ytick.labelsize': 14,
    'legend.fontsize': 14,
    'figure.titlesize': 22
})

# ファイルパスを指定
file_path = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/maehara_slit_total_updated.csv'

# CSVファイルを読み込む
df_slit = pd.read_csv(file_path)

# クラス名を定義（指定された順序）
class_names = ["Normal", "Infectious keratitis", "Non-infection keratitis",
               "Scar", "Tumor", "Deposit", "APAC", "Lens opacity", "Bullous keratopathy"]

# クラス名を数字に対応させる
df_slit['class_name'] = df_slit['top1'].map(dict(enumerate(class_names)))

# レイヤーごとのプロットと統計量計算
for layer in [17, 20, 23]:
    layer_name = f"AOI_0.5_layer{layer}"

    # グラフの作成（縦横比3:4）
    plt.figure(figsize=(12, 9))

    # boxplotのスタイルをカスタマイズ（先ほどと同じ灰色のスタイル）
    sns.boxplot(x='class_name', y=layer_name, data=df_slit, order=class_names,
                color='#CCCCCC',
                boxprops={'facecolor':'#CCCCCC', 'edgecolor':'black', 'linewidth': 1.5},
                medianprops={'color':'black', 'linewidth':2.5},
                flierprops={'marker':'o', 'markerfacecolor':'white', 'markeredgecolor':'black', 'markersize':6},
                whiskerprops={'color':'black', 'linewidth':1.5},
                capprops={'color':'black', 'linewidth':1.5})

    plt.xlabel('Class', fontsize=18, labelpad=10)
    plt.ylabel(layer_name, fontsize=18, labelpad=10)
    plt.title(f'Box Plot of {layer_name} for Slit Lamp Data', fontsize=20, pad=20)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()

    # グラフを保存（高解像度）
    plt.savefig(f'boxplot_{layer_name}.png', dpi=350, bbox_inches='tight')
    plt.show()

    # 基本統計量の計算と表示
    stats = df_slit.groupby('class_name')[layer_name].describe()
    # クラスの順序を指定して並び替え
    stats = stats.reindex(class_names)
    print(f"\n{layer_name}の基本統計量:")
    print(stats)
    print("\n" + "="*80 + "\n")

In [ ]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns

# # フォントサイズとスタイルの設定
# plt.rcParams.update({
#     'font.size': 12,
#     'axes.labelsize': 16,
#     'axes.titlesize': 16,
#     'xtick.labelsize': 12,
#     'ytick.labelsize': 12,
#     'legend.fontsize': 12,
#     'figure.titlesize': 18
# })

# # ファイルパスを指定
# file_path1 = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/maehara_slit_total_updated.csv'
# file_path2 = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/maehara_sumaho_total_updated.csv'

# # CSVファイルを読み込む
# df_slit = pd.read_csv(file_path1)
# df_sumaho = pd.read_csv(file_path2)

# # クラス名を定義
# class_names = ["infection", "normal", "non-infection", "scar", "tumor", "deposit", "APAC", "lens opacity", "bullous"]

# # クラス名を数字に対応させる
# df_slit['class_name'] = df_slit['top1'].map(dict(enumerate(class_names)))
# df_sumaho['class_name'] = df_sumaho['top1'].map(dict(enumerate(class_names)))

# # データフレームを整形してSeabornでプロットできるようにする
# df_slit['Type'] = 'Slit'
# df_sumaho['Type'] = 'Sumaho'

# # 比較するレイヤーのリスト
# layers_group1 = ["24_m_0", "24_m_1", "24_m_2"]
# layers_group2 = ["17", "20", "23"]

# def create_boxplots(layers, group_name):
#     for layer in layers:
#         layer_name = f"AOI_0.5_layer{layer}"

#         # 必要なカラムだけを選択し、欠損値を削除して結合
#         df_combined = pd.concat([
#             df_slit[['class_name', layer_name, 'Type']].dropna(),
#             df_sumaho[['class_name', layer_name, 'Type']].dropna()
#         ]).reset_index(drop=True)

#         # グラフ: スリットランプとスマートフォンのデータ
#         plt.figure(figsize=(15, 10))
#         sns.boxplot(x='class_name', y=layer_name, hue='Type', data=df_combined, order=class_names, palette=['#333333', '#999999'])
#         plt.xlabel('Class', fontsize=14, labelpad=10)
#         plt.ylabel(layer_name, fontsize=14, labelpad=10)
#         plt.title(f'Box Plot of {layer_name} for Slit Lamp and Smartphone Data', fontsize=16, pad=20)
#         plt.xticks(rotation=45, ha='right')
#         legend = plt.legend(title='Type', bbox_to_anchor=(1.05, 1), loc='upper left')
#         legend.get_title().set_fontsize(14)
#         plt.tight_layout()
#         plt.show()

#     # レイヤー間の比較（幅を半分に）
#     plt.figure(figsize=(7.5, 10))  # 幅を半分に変更
#     df_melted = pd.melt(df_slit, id_vars=['class_name'], value_vars=[f'AOI_0.5_layer{layer}' for layer in layers], var_name='Layer', value_name='Value')
#     sns.boxplot(x='Layer', y='Value', data=df_melted, palette='Blues')
#     plt.xlabel('Layer', fontsize=14, labelpad=10)
#     plt.ylabel('AOI_0.5 Value', fontsize=14, labelpad=10)
#     plt.title(f'Comparison of AOI_0.5 Values\nAcross Different Layers\n({group_name})', fontsize=16, pad=20)
#     plt.xticks(rotation=45, ha='right')
#     plt.tight_layout()
#     plt.show()

# # グループ1のグラフを作成
# create_boxplots(layers_group1, "Group 1: 24_m_0, 24_m_1, 24_m_2")

# # グループ2のグラフを作成
# create_boxplots(layers_group2, "Group 2: 17, 20, 23")

In [ ]:
#レイヤー毎のAOI値（17, 20, 23）(24-0, 24-1, 24-2)

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# フォントサイズとスタイルの設定をさらに大きく
plt.rcParams.update({
    'font.size': 18,          # 16 → 18
    'axes.labelsize': 24,     # 22 → 24
    'axes.titlesize': 26,     # 24 → 26
    'xtick.labelsize': 18,    # 16 → 18
    'ytick.labelsize': 18,    # 16 → 18
    'legend.fontsize': 18,    # 16 → 18
    'figure.titlesize': 28    # 26 → 28
})

# ファイルパスを指定
file_path = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/maehara_slit_total_updated.csv'

# CSVファイルを読み込む
df = pd.read_csv(file_path)

# 2つのグループに分けるレイヤーのリスト
layers_group1 = ["AOI_0.5_layer17", "AOI_0.5_layer20", "AOI_0.5_layer23"]
layers_group2 = ["AOI_0.5_layer24_m_0", "AOI_0.5_layer24_m_1", "AOI_0.5_layer24_m_2"]

# レイヤー名を変更するマッピング
layer_mapping = {
    "AOI_0.5_layer17": "Layer 17",
    "AOI_0.5_layer20": "Layer 20",
    "AOI_0.5_layer23": "Layer 23",
    "AOI_0.5_layer24_m_0": "Layer 24_0",
    "AOI_0.5_layer24_m_1": "Layer 24_1",
    "AOI_0.5_layer24_m_2": "Layer 24_2"
}

# グラフ1（3:4のアスペクト比）
plt.figure(figsize=(9, 12))  # 3:4の比率

df_melted_1 = pd.melt(df[layers_group1], var_name='Layer', value_name='AOI Value')
df_melted_1['Layer'] = df_melted_1['Layer'].map(layer_mapping)

sns.boxplot(x='Layer', y='AOI Value', data=df_melted_1,
            color='#CCCCCC',
            boxprops={'facecolor':'#CCCCCC', 'edgecolor':'black', 'linewidth': 1.5},
            medianprops={'color':'black', 'linewidth':2.5},
            flierprops={'marker':'o', 'markerfacecolor':'white', 'markeredgecolor':'black', 'markersize':6},
            whiskerprops={'color':'black', 'linewidth':1.5},
            capprops={'color':'black', 'linewidth':1.5})

plt.xlabel('Layer', fontsize=24, labelpad=15)
plt.ylabel('AOI Value', fontsize=24, labelpad=15)
plt.title('Comparison of AOI Values\nLayers 17, 20, and 23', fontsize=26, pad=25)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

# グラフを保存（350dpi）
plt.savefig('boxplot_aoi_comparison_group1.png', dpi=350, bbox_inches='tight')
plt.show()

# グラフ2（3:4のアスペクト比）
plt.figure(figsize=(9, 12))  # 3:4の比率

df_melted_2 = pd.melt(df[layers_group2], var_name='Layer', value_name='AOI Value')
df_melted_2['Layer'] = df_melted_2['Layer'].map(layer_mapping)

sns.boxplot(x='Layer', y='AOI Value', data=df_melted_2,
            color='#CCCCCC',
            boxprops={'facecolor':'#CCCCCC', 'edgecolor':'black', 'linewidth': 1.5},
            medianprops={'color':'black', 'linewidth':2.5},
            flierprops={'marker':'o', 'markerfacecolor':'white', 'markeredgecolor':'black', 'markersize':6},
            whiskerprops={'color':'black', 'linewidth':1.5},
            capprops={'color':'black', 'linewidth':1.5})

plt.xlabel('Layer', fontsize=24, labelpad=15)
plt.ylabel('AOI Value', fontsize=24, labelpad=15)
plt.title('Comparison of AOI Values\nLayer 24 Outputs', fontsize=26, pad=25)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

# グラフを保存（350dpi）
plt.savefig('boxplot_aoi_comparison_group2.png', dpi=350, bbox_inches='tight')
plt.show()

# 基本統計量の計算と表示
print("\n基本統計量 - グループ1 (Layers 17, 20, 23):")
print(df[layers_group1].describe())
print("\n基本統計量 - グループ2 (Layer 24 Outputs):")
print(df[layers_group2].describe())

In [ ]:
!pip install scikit_posthocs --q

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scikit_posthocs import posthoc_dunn

# フォントサイズとスタイルの設定
plt.rcParams.update({
    'font.size': 18,
    'axes.labelsize': 24,
    'axes.titlesize': 26,
    'xtick.labelsize': 18,
    'ytick.labelsize': 18,
    'legend.fontsize': 18,
    'figure.titlesize': 28
})

# ファイルパスを指定
file_path = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/maehara_slit_total_updated.csv'

# CSVファイルを読み込む
df = pd.read_csv(file_path)

# すべてのレイヤーのリスト
layers = ["AOI_0.5_layer17", "AOI_0.5_layer20", "AOI_0.5_layer23",
          "AOI_0.5_layer24_m_0", "AOI_0.5_layer24_m_1", "AOI_0.5_layer24_m_2"]

# レイヤー名を変更するマッピング
layer_mapping = {
    "AOI_0.5_layer17": "Layer 17",
    "AOI_0.5_layer20": "Layer 20",
    "AOI_0.5_layer23": "Layer 23",
    "AOI_0.5_layer24_m_0": "Layer 24_0",
    "AOI_0.5_layer24_m_1": "Layer 24_1",
    "AOI_0.5_layer24_m_2": "Layer 24_2"
}

# データを整形
df_melted = pd.melt(df[layers], var_name='Layer', value_name='AOI Value')
df_melted['Layer'] = df_melted['Layer'].map(layer_mapping)

# グラフ作成（3:4のアスペクト比）
plt.figure(figsize=(15, 12))

# boxplotのスタイルをカスタマイズ
sns.boxplot(x='Layer', y='AOI Value', data=df_melted,
            color='#CCCCCC',
            boxprops={'facecolor':'#CCCCCC', 'edgecolor':'black', 'linewidth': 1.5},
            medianprops={'color':'black', 'linewidth':2.5},
            flierprops={'marker':'o', 'markerfacecolor':'white', 'markeredgecolor':'black', 'markersize':6},
            whiskerprops={'color':'black', 'linewidth':1.5},
            capprops={'color':'black', 'linewidth':1.5})

plt.xlabel('Layer', fontsize=24, labelpad=15)
plt.ylabel('AOI Value', fontsize=24, labelpad=15)
plt.title('Comparison of AOI Values Across Different Layers', fontsize=26, pad=25)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

# グラフを保存（350dpi）
plt.savefig('boxplot_aoi_comparison_all.png', dpi=350, bbox_inches='tight')
plt.show()

# 基本統計量の計算と表示
print("\n基本統計量:")
print(df[layers].describe())

# Kruskal-Wallis 検定を実施
h_statistic, p_value = stats.kruskal(*[df[layer] for layer in layers])

print('\nKruskal-Wallis test results:')
print(f'H-statistic: {h_statistic:.3f}')
print(f'p-value: {p_value:.3e}')

# Dunn's testによる多重比較（Bonferroni補正）
dunn = posthoc_dunn(df_melted, val_col='AOI Value', group_col='Layer', p_adjust='bonferroni')

print('\nDunn\'s test results (p-values):')
print(dunn.round(4))  # p値を4桁に丸める

# 有意差のあるペアを抽出（p < 0.05）
significant_pairs = []
for i in dunn.index:
    for j in dunn.columns:
        if i < j:  # 重複を避けるため、上三角行列のみを見る
            if dunn.loc[i, j] < 0.05:
                significant_pairs.append((i, j, dunn.loc[i, j]))

print('\n有意差のあるペア (p < 0.05):')
for pair in significant_pairs:
    print(f'{pair[0]} vs {pair[1]}: p = {pair[2]:.4f}')

In [ ]:
!pip install scikit_posthocs --q

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scikit_posthocs import posthoc_dunn

# フォントサイズとスタイルの設定
plt.rcParams.update({
    'font.size': 22,  # フォントサイズを大きめに設定
    'axes.labelsize': 28,  # 軸ラベルのフォントサイズ
    'axes.titlesize': 30,  # タイトルのフォントサイズ
    'xtick.labelsize': 24,  # x軸の目盛りラベルのフォントサイズ
    'ytick.labelsize': 24,  # y軸の目盛りラベルのフォントサイズ
    'legend.fontsize': 24,  # 凡例のフォントサイズ
    'figure.titlesize': 32  # フィギュアタイトルのフォントサイズ
})

# ファイルパスを指定
file_path1 = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/maehara_slit_total_gradcam.csv'
file_path2 = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/maehara_slit_total_gradcam++.csv'

# CSVファイルを読み込む
df1 = pd.read_csv(file_path1)
df2 = pd.read_csv(file_path2)

# 使用するレイヤーのリスト
layers = ["AOI_0.5_layer17", "AOI_0.5_layer20", "AOI_0.5_layer23",
          "AOI_0.5_layer24_m_0", "AOI_0.5_layer24_m_1", "AOI_0.5_layer24_m_2"]

# レイヤー名を変更するマッピング
layer_mapping = {
    "AOI_0.5_layer17": "Layer 17",
    "AOI_0.5_layer20": "Layer 20",
    "AOI_0.5_layer23": "Layer 23",
    "AOI_0.5_layer24_m_0": "Layer 24_0",
    "AOI_0.5_layer24_m_1": "Layer 24_1",
    "AOI_0.5_layer24_m_2": "Layer 24_2"
}

# データを整形
df1_melted = pd.melt(df1[layers], var_name='Layer', value_name='AOI Value')
df1_melted['Layer'] = df1_melted['Layer'].map(layer_mapping)
df1_melted['Method'] = 'Grad-CAM'

df2_melted = pd.melt(df2[layers], var_name='Layer', value_name='AOI Value')
df2_melted['Layer'] = df2_melted['Layer'].map(layer_mapping)
df2_melted['Method'] = 'Grad-CAM++'

# データを結合し、インデックスを振り直す
df_combined = pd.concat([df1_melted, df2_melted], ignore_index=True)
# あるいは
# df_combined = pd.concat([df1_melted, df2_melted])
# df_combined = df_combined.reset_index(drop=True)

# グラフ作成（4:3のアスペクト比）
plt.figure(figsize=(16, 12))  # 4:3の比率になるように調整

# boxplotのスタイルをカスタマイズ
sns.boxplot(x='Layer', y='AOI Value', hue='Method', data=df_combined,
            palette=["#1f77b4", "#ff7f0e"],
            boxprops={'edgecolor':'black', 'linewidth': 1.5},
            medianprops={'color':'black', 'linewidth':2.5},
            flierprops={'marker':'o', 'markerfacecolor':'white', 'markeredgecolor':'black', 'markersize':6},
            whiskerprops={'color':'black', 'linewidth':1.5},
            capprops={'color':'black', 'linewidth':1.5})

plt.xlabel('Layer', fontsize=28, labelpad=15)  # 軸ラベルのフォントサイズをさらに大きく
plt.ylabel('AOI Value', fontsize=28, labelpad=15)  # 軸ラベルのフォントサイズをさらに大きく
plt.title('Comparison of AOI Values Across Different Layers and Methods', fontsize=30, pad=25)  # タイトルのフォントサイズをさらに大きく
plt.xticks(rotation=45, ha='right')
plt.legend(title='Type')
plt.tight_layout()

# グラフを保存（350dpi）
plt.savefig('boxplot_aoi_comparison_all_combined.png', dpi=350, bbox_inches='tight')
plt.show()

# 基本統計量の計算と表示 (Grad-CAM)
print("\n基本統計量 (Grad-CAM):")
print(df1[layers].describe())

# 基本統計量の計算と表示 (Grad-CAM++)
print("\n基本統計量 (Grad-CAM++):")
print(df2[layers].describe())

# Kruskal-Wallis 検定を実施 (Grad-CAM)
h_statistic1, p_value1 = stats.kruskal(*[df1[layer] for layer in layers])

print('\nKruskal-Wallis test results (Grad-CAM):')
print(f'H-statistic: {h_statistic1:.3f}')
print(f'p-value: {p_value1:.3e}')

# Kruskal-Wallis 検定を実施 (Grad-CAM++)
h_statistic2, p_value2 = stats.kruskal(*[df2[layer] for layer in layers])

print('\nKruskal-Wallis test results (Grad-CAM++):')
print(f'H-statistic: {h_statistic2:.3f}')
print(f'p-value: {p_value2:.3e}')

# Dunn's testによる多重比較（Bonferroni補正）(Grad-CAM)
dunn1 = posthoc_dunn(df1_melted, val_col='AOI Value', group_col='Layer', p_adjust='bonferroni')

print('\nDunn\'s test results (Grad-CAM) (p-values):')
print(dunn1.round(4))  # p値を4桁に丸める

# Dunn's testによる多重比較（Bonferroni補正）(Grad-CAM++)
dunn2 = posthoc_dunn(df2_melted, val_col='AOI Value', group_col='Layer', p_adjust='bonferroni')

print('\nDunn\'s test results (Grad-CAM++) (p-values):')
print(dunn2.round(4))  # p値を4桁に丸める

# 有意差のあるペアを抽出（p < 0.05）(Grad-CAM)
significant_pairs1 = []
for i in dunn1.index:
    for j in dunn1.columns:
        if i < j:  # 重複を避けるため、上三角行列のみを見る
            if dunn1.loc[i, j] < 0.05:
                significant_pairs1.append((i, j, dunn1.loc[i, j]))

print('\n有意差のあるペア (Grad-CAM) (p < 0.05):')
for pair in significant_pairs1:
    print(f'{pair[0]} vs {pair[1]}: p = {pair[2]:.4f}')

# 有意差のあるペアを抽出（p < 0.05）(Grad-CAM++)
significant_pairs2 = []
for i in dunn2.index:
    for j in dunn2.columns:
        if i < j:  # 重複を避けるため、上三角行列のみを見る
            if dunn2.loc[i, j] < 0.05:
                significant_pairs2.append((i, j, dunn2.loc[i, j]))

print('\n有意差のあるペア (Grad-CAM++) (p < 0.05):')
for pair in significant_pairs2:
    print(f'{pair[0]} vs {pair[1]}: p = {pair[2]:.4f}')

In [ ]:
####
#### スリット vs スマホの比較
####

import scipy.stats as stats

# 各クラスごとに対応のあるt検定を行う
results = []
for class_name in class_names:
    slit_data = df_slit[df_slit['class_name'] == class_name][layer_name].dropna().reset_index(drop=True)
    sumaho_data = df_sumaho[df_sumaho['class_name'] == class_name][layer_name].dropna().reset_index(drop=True)

    # 対応のあるデータを取るため、最小の長さに合わせる
    min_length = min(len(slit_data), len(sumaho_data))
    slit_data = slit_data[:min_length]
    sumaho_data = sumaho_data[:min_length]

    t_stat, p_value = stats.ttest_rel(slit_data, sumaho_data)

    # スリットランプデータの統計値
    slit_mean = slit_data.mean()
    slit_std = slit_data.std()

    # スマートフォンデータの統計値
    sumaho_mean = sumaho_data.mean()
    sumaho_std = sumaho_data.std()

    results.append({
        'class_name': class_name,
        't_stat': t_stat,
        'p_value': p_value,
        'slit_mean': slit_mean,
        'slit_std': slit_std,
        'sumaho_mean': sumaho_mean,
        'sumaho_std': sumaho_std
    })

# 全クラスまとめた対応のあるt検定
all_slit_data = df_slit[layer_name].dropna().reset_index(drop=True)
all_sumaho_data = df_sumaho[layer_name].dropna().reset_index(drop=True)

# 対応のあるデータを取るため、最小の長さに合わせる
min_length_all = min(len(all_slit_data), len(all_sumaho_data))
all_slit_data = all_slit_data[:min_length_all]
all_sumaho_data = all_sumaho_data[:min_length_all]

t_stat_all, p_value_all = stats.ttest_rel(all_slit_data, all_sumaho_data)

# 全クラスまとめた統計値
all_slit_mean = all_slit_data.mean()
all_slit_std = all_slit_data.std()
all_sumaho_mean = all_sumaho_data.mean()
all_sumaho_std = all_sumaho_data.std()

# 結果を追加
results.append({
    'class_name': 'All Classes',
    't_stat': t_stat_all,
    'p_value': p_value_all,
    'slit_mean': all_slit_mean,
    'slit_std': all_slit_std,
    'sumaho_mean': all_sumaho_mean,
    'sumaho_std': all_sumaho_std
})

# 結果をデータフレームに変換して表示
df_results = pd.DataFrame(results)
df_results['p_value'] = df_results['p_value'].map(lambda x: f'{x:.3f}')

df_results

In [ ]:
# #クラス毎の差（スマホ＋スリット）
# from statsmodels.stats.multicomp import pairwise_tukeyhsd

# # ANOVAを実行
# anova_result = stats.f_oneway(
#     df_combined[df_combined['class_name'] == 'infection'][layer_name],
#     df_combined[df_combined['class_name'] == 'normal'][layer_name],
#     df_combined[df_combined['class_name'] == 'non-infection'][layer_name],
#     df_combined[df_combined['class_name'] == 'scar'][layer_name],
#     df_combined[df_combined['class_name'] == 'tumor'][layer_name],
#     df_combined[df_combined['class_name'] == 'deposit'][layer_name],
#     df_combined[df_combined['class_name'] == 'APAC'][layer_name],
#     df_combined[df_combined['class_name'] == 'lens opacity'][layer_name],
#     df_combined[df_combined['class_name'] == 'bullous'][layer_name]
# )

# print(f"ANOVA result: F={anova_result.statistic}, p={anova_result.pvalue}")

# # 事後検定（TukeyのHSD検定）を実行
# tukey_result = pairwise_tukeyhsd(df_combined[layer_name], df_combined['class_name'])

# # 結果を表示
# print(tukey_result)

In [ ]:
import pandas as pd
from scipy import stats
import numpy as np
from statsmodels.stats.multitest import multipletests
import warnings
warnings.filterwarnings('ignore')

# ファイルパスを指定
file_path = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/maehara_slit_total_updated.csv'

# CSVファイルを読み込む
df_slit = pd.read_csv(file_path)

# クラス名を定義（指定された順序）
class_names = ["Normal", "Infectious keratitis", "Non-infection keratitis",
               "Scar", "Tumor", "Deposit", "APAC", "Lens opacity", "Bullous keratopathy"]

# クラス名を数字に対応させる
df_slit['class_name'] = df_slit['top1'].map(dict(enumerate(class_names)))

# 解析するレイヤーのリスト
layers = [17, 20, 23, "24_m_0", "24_m_1", "24_m_2"]

def check_all_values_same(groups):
    """全ての値が同じかチェック"""
    all_values = np.concatenate([group.values for group in groups])
    return np.all(all_values == all_values[0]), all_values[0]

def dunn_test(groups, group_names):
    """
    Dunn's testの実装
    """
    # 全データをランク付け
    all_data = np.concatenate(groups)
    ranks = stats.rankdata(all_data)

    # グループごとの平均ランクを計算
    start = 0
    mean_ranks = []
    ns = []
    for group in groups:
        n = len(group)
        group_ranks = ranks[start:start + n]
        mean_ranks.append(np.mean(group_ranks))
        ns.append(n)
        start += n

    # 全ペアの組み合わせでDunn's testを実行
    N = len(ranks)
    k = len(groups)
    comparisons = []
    p_values = []

    for i in range(k):
        for j in range(i + 1, k):
            # 平均ランクの差
            diff = abs(mean_ranks[i] - mean_ranks[j])

            # 標準誤差
            se = np.sqrt((N * (N + 1) / 12) * (1/ns[i] + 1/ns[j]))

            # z統計量
            z = diff / se

            # p値（両側検定）
            p = 2 * (1 - stats.norm.cdf(abs(z)))

            comparisons.append((group_names[i], group_names[j]))
            p_values.append(p)

    # Bonferroni補正
    rejected, p_corrected, _, _ = multipletests(p_values, method='bonferroni')

    # 有意な結果を返す
    significant_results = []
    for (name1, name2), p_corr, is_rej in zip(comparisons, p_corrected, rejected):
        if is_rej:  # p < 0.05 after correction
            significant_results.append({
                'group1': name1,
                'group2': name2,
                'p_value': p_corr
            })

    return significant_results

for layer in layers:
    # レイヤー名を設定
    if isinstance(layer, int):
        layer_name = f"AOI_0.5_layer{layer}"
    else:
        layer_name = f"AOI_0.5_layer{layer}"

    print(f"\n{'='*80}")
    print(f"\nAnalysis for {layer_name}")
    print('='*80)

    # 欠損値を除外してデータを準備
    data_for_analysis = df_slit[[layer_name, 'class_name']].dropna()

    if len(data_for_analysis) > 0:
        # クラスごとのデータを収集（欠損値を除外）
        class_groups = []
        valid_class_names = []

        for class_name in class_names:
            group_data = df_slit[df_slit['class_name'] == class_name][layer_name].dropna()
            if len(group_data) > 0:
                class_groups.append(group_data)
                valid_class_names.append(class_name)

        if len(class_groups) > 1:  # 少なくとも2つのグループが必要
            # 全ての値が同じかチェック
            is_all_same, same_value = check_all_values_same(class_groups)

            if is_all_same:
                print(f"\n全てのクラスで同じ値: {same_value:.4f}")
                print("統計的検定は不要です。")
                continue

            # Kruskal-Wallis H-testを実行
            h_statistic, p_value = stats.kruskal(*class_groups)

            print(f"\nKruskal-Wallis H-test結果:")
            print(f"H-statistic: {h_statistic:.4f}")
            print(f"p-value: {p_value:.4e}")

            if p_value < 0.05:
                # Dunn's testを実行
                significant_pairs = dunn_test(class_groups, valid_class_names)

                if significant_pairs:
                    print("\n有意差のあるペア (Bonferroni補正後 p < 0.05):")
                    for result in significant_pairs:
                        print(f"{result['group1']:25} vs {result['group2']:25}: "
                              f"p = {result['p_value']:.4e}")
        else:
            print(f"\n警告: {layer_name}の解析に十分なデータがありません。")
    else:
        print(f"\n警告: {layer_name}に有効なデータがありません。")

In [ ]:
import pandas as pd
from scipy import stats
import numpy as np
from scipy.stats import mannwhitneyu
import warnings
warnings.filterwarnings('ignore')

# ファイルパスを指定
file_path = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/maehara_slit_total_updated.csv'

# CSVファイルを読み込む
df_slit = pd.read_csv(file_path)

# クラス名を定義（指定された順序）
class_names = ["Normal", "Infectious keratitis", "Non-infection keratitis",
               "Scar", "Tumor", "Deposit", "APAC", "Lens opacity", "Bullous keratopathy"]

# クラス名を数字に対応させる
df_slit['class_name'] = df_slit['top1'].map(dict(enumerate(class_names)))

# 解析するレイヤーのリスト
layers = [17, 20, 23, "24_m_0", "24_m_1", "24_m_2"]

def check_all_values_same(groups):
    """全ての値が同じかチェック"""
    all_values = np.concatenate([group.values for group in groups])
    return np.all(all_values == all_values[0]), all_values[0]

def perform_pairwise_mannwhitney(groups, group_names, alpha=0.05):
    """ペアワイズMann-Whitney U検定を実行"""
    n_groups = len(groups)
    results = []

    for i in range(n_groups):
        for j in range(i+1, n_groups):
            try:
                stat, p_value = mannwhitneyu(groups[i], groups[j], alternative='two-sided')
                n1, n2 = len(groups[i]), len(groups[j])
                effect_size = 1 - (2 * stat) / (n1 * n2)  # Common language effect size

                if p_value < alpha:
                    results.append({
                        'group1': group_names[i],
                        'group2': group_names[j],
                        'p_value': p_value,
                        'effect_size': effect_size
                    })
            except ValueError:
                continue

    return results

for layer in layers:
    # レイヤー名を設定
    if isinstance(layer, int):
        layer_name = f"AOI_0.5_layer{layer}"
    else:
        layer_name = f"AOI_0.5_layer{layer}"

    print(f"\n{'='*80}")
    print(f"\nAnalysis for {layer_name}")
    print('='*80)

    # 欠損値を除外してデータを準備
    data_for_analysis = df_slit[[layer_name, 'class_name']].dropna()

    if len(data_for_analysis) > 0:
        # クラスごとのデータを収集（欠損値を除外）
        class_groups = []
        valid_class_names = []

        for class_name in class_names:
            group_data = df_slit[df_slit['class_name'] == class_name][layer_name].dropna()
            if len(group_data) > 0:
                class_groups.append(group_data)
                valid_class_names.append(class_name)

        if len(class_groups) > 1:  # 少なくとも2つのグループが必要
            # 全ての値が同じかチェック
            is_all_same, same_value = check_all_values_same(class_groups)

            if is_all_same:
                print(f"\n全てのクラスで同じ値: {same_value:.4f}")
                print("統計的検定は不要です。")
                continue

            # Kruskal-Wallis H-testを実行
            h_statistic, p_value = stats.kruskal(*class_groups)

            print(f"\nKruskal-Wallis H-test結果:")
            print(f"H-statistic: {h_statistic:.4f}")
            print(f"p-value: {p_value:.4e}")

            if p_value < 0.05:
                # ペアワイズMann-Whitney U検定を実行
                significant_pairs = perform_pairwise_mannwhitney(
                    class_groups,
                    valid_class_names
                )

                if significant_pairs:
                    print("\n有意差のあるペア (p < 0.05):")
                    for result in significant_pairs:
                        print(f"{result['group1']:25} vs {result['group2']:25}: "
                              f"p = {result['p_value']:.4e}, effect size = {result['effect_size']:.4f}")
        else:
            print(f"\n警告: {layer_name}の解析に十分なデータがありません。")
    else:
        print(f"\n警告: {layer_name}に有効なデータがありません。")

In [ ]:
import pandas as pd
from scipy import stats
import numpy as np
from scipy.stats import mannwhitneyu

# ファイルパスを指定
file_path = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/maehara_slit_total_updated.csv'

# CSVファイルを読み込む
df = pd.read_csv(file_path)

# 解析するレイヤーのリスト
layers = ["AOI_0.5_layer17", "AOI_0.5_layer20", "AOI_0.5_layer23",
          "AOI_0.5_layer24_m_0", "AOI_0.5_layer24_m_1", "AOI_0.5_layer24_m_2"]

# データを長形式に変換
df_melted = pd.melt(df[layers], var_name='Layer', value_name='AOI Value')

# レイヤー名を簡略化
layer_mapping = {
    "AOI_0.5_layer17": "Layer 17",
    "AOI_0.5_layer20": "Layer 20",
    "AOI_0.5_layer23": "Layer 23",
    "AOI_0.5_layer24_m_0": "Layer 24_0",
    "AOI_0.5_layer24_m_1": "Layer 24_1",
    "AOI_0.5_layer24_m_2": "Layer 24_2"
}
df_melted['Layer'] = df_melted['Layer'].map(layer_mapping)

# 欠損値を除外
df_melted = df_melted.dropna()

# Kruskal-Wallis H-testを実行
groups = [group['AOI Value'].values for name, group in df_melted.groupby('Layer')]
h_statistic, p_value = stats.kruskal(*groups)

print("Kruskal-Wallis H-test結果:")
print(f"H-statistic: {h_statistic:.4f}")
print(f"p-value: {p_value:.4e}")

# レイヤーのペアワイズ比較（Mann-Whitney U test）を実行
layer_names = sorted(df_melted['Layer'].unique())
significant_pairs = []

print("\n有意差のあるペア (p < 0.05):")
for i, layer1 in enumerate(layer_names):
    for layer2 in layer_names[i+1:]:
        group1 = df_melted[df_melted['Layer'] == layer1]['AOI Value']
        group2 = df_melted[df_melted['Layer'] == layer2]['AOI Value']

        try:
            stat, p_value = mannwhitneyu(group1, group2, alternative='two-sided')
            n1, n2 = len(group1), len(group2)
            effect_size = 1 - (2 * stat) / (n1 * n2)  # Common language effect size

            if p_value < 0.05:  # Bonferroni補正を適用する場合は 0.05/len(pairs) を使用
                print(f"{layer1:10} vs {layer2:10}: p = {p_value:.4e}, effect size = {effect_size:.4f}")
                significant_pairs.append((layer1, layer2, p_value, effect_size))
        except ValueError:
            continue

# 各レイヤーの基本統計量（中央値と四分位数を含む）
print("\n各レイヤーの基本統計量:")
stats_summary = df_melted.groupby('Layer')['AOI Value'].agg([
    ('count', 'count'),
    ('median', 'median'),
    ('mean', 'mean'),
    ('std', 'std'),
    ('Q1', lambda x: x.quantile(0.25)),
    ('Q3', lambda x: x.quantile(0.75))
]).round(4)

print(stats_summary)

# マトリックス形式で有意差を表示
print("\n有意差マトリックス (★: p < 0.05)")
matrix = pd.DataFrame(index=layer_names, columns=layer_names)
np.fill_diagonal(matrix.values, '-')
matrix = matrix.fillna('　')

for pair in significant_pairs:
    matrix.loc[pair[0], pair[1]] = "★"
    matrix.loc[pair[1], pair[0]] = "★"

print(matrix)

###**ANOVA_heatmap**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# ファイルパスを設定
file_path1 = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/maehara_slit_total_updated.csv'
file_path2 = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/maehara_sumaho_total_updated.csv'

# CSVファイルを読み込む
df_slit = pd.read_csv(file_path1)
df_sumaho = pd.read_csv(file_path2)

# クラス名を定義
class_names = ["infection", "normal", "non-infection", "scar", "tumor", "deposit", "APAC", "lens opacity", "bullous"]

# クラス名を数字に対応させる
df_slit['class_name'] = df_slit['top1'].map(dict(enumerate(class_names)))
df_sumaho['class_name'] = df_sumaho['top1'].map(dict(enumerate(class_names)))

# データフレームを整形
df_slit['Type'] = 'Slit'
df_sumaho['Type'] = 'Sumaho'

# 必要なカラムだけを選択し、欠損値を削除して結合
df_combined = pd.concat([
    df_slit[['class_name', layer_name, 'Type']].dropna(),
    df_sumaho[['class_name', layer_name, 'Type']].dropna()
]).reset_index(drop=True)

# 事後検定（TukeyのHSD検定）を実行
tukey_result = pairwise_tukeyhsd(endog=df_combined[layer_name], groups=df_combined['class_name'])

# 結果をDataFrameに変換
tukey_result_df = pd.DataFrame(data=tukey_result._results_table.data[1:], columns=tukey_result._results_table.data[0])

# マトリックス図を作成するためにクラス名をインデックスとして使用
matrix = pd.DataFrame(np.zeros((len(class_names), len(class_names))), index=class_names, columns=class_names)

# マトリックスにp値を入力
for i in range(len(tukey_result_df)):
    group1 = tukey_result_df.loc[i, 'group1']
    group2 = tukey_result_df.loc[i, 'group2']
    p_value = float(tukey_result_df.loc[i, 'p-adj'])
    matrix.loc[group1, group2] = p_value
    matrix.loc[group2, group1] = p_value

# 同じクラス間のセルをNaNに設定
np.fill_diagonal(matrix.values, np.nan)

# ヒートマップを描画
plt.figure(figsize=(12, 10))
sns.heatmap(matrix, annot=True, fmt=".3f", cmap="coolwarm", cbar_kws={'label': 'p-value'})
plt.title('Pairwise Comparison P-Values (Tukey HSD)')
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from matplotlib.colors import ListedColormap

# ファイルパスを設定
file_path1 = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/maehara_slit_total_updated.csv'
file_path2 = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/maehara_sumaho_total_updated.csv'

# CSVファイルを読み込む
df_slit = pd.read_csv(file_path1)
df_sumaho = pd.read_csv(file_path2)

# クラス名を定義
class_names = ["infection", "normal", "non-infection", "scar", "tumor", "deposit", "APAC", "lens opacity", "bullous"]

# クラス名を数字に対応させる
df_slit['class_name'] = df_slit['top1'].map(dict(enumerate(class_names)))
df_sumaho['class_name'] = df_sumaho['top1'].map(dict(enumerate(class_names)))

# データフレームを整形
df_slit['Type'] = 'Slit'
df_sumaho['Type'] = 'Sumaho'

# 必要なカラムだけを選択し、欠損値を削除して結合
df_combined = pd.concat([
    df_slit[['class_name', layer_name, 'Type']].dropna(),
    df_sumaho[['class_name', layer_name, 'Type']].dropna()
]).reset_index(drop=True)

# 事後検定（TukeyのHSD検定）を実行
tukey_result = pairwise_tukeyhsd(endog=df_combined[layer_name], groups=df_combined['class_name'])

# 結果をDataFrameに変換
tukey_result_df = pd.DataFrame(data=tukey_result._results_table.data[1:], columns=tukey_result._results_table.data[0])

# マトリックス図を作成するためにクラス名をインデックスとして使用
matrix = pd.DataFrame(np.zeros((len(class_names), len(class_names))), index=class_names, columns=class_names)

# マトリックスにp値を入力
for i in range(len(tukey_result_df)):
    group1 = tukey_result_df.loc[i, 'group1']
    group2 = tukey_result_df.loc[i, 'group2']
    p_value = float(tukey_result_df.loc[i, 'p-adj'])
    matrix.loc[group1, group2] = p_value
    matrix.loc[group2, group1] = p_value

# 同じクラス間のセルをNaNに設定
np.fill_diagonal(matrix.values, np.nan)

# カスタムカラーマップを作成（オレンジ、グレー、白）
colors = ['#FFA500', '#D3D3D3', 'white']  # オレンジ、グレー、白
custom_cmap = ListedColormap(colors)

# p値に基づいてデータを作成（NaNは2、p<0.05は0、それ以外は1）
color_data = np.where(np.isnan(matrix), 2, np.where(matrix < 0.05, 0, 1))

# ヒートマップを描画
plt.figure(figsize=(12, 10))
sns.heatmap(color_data,
            annot=matrix,  # 元のp値を表示
            fmt=".3f",
            cmap=custom_cmap,
            cbar=False,  # カラーバーを非表示に
            xticklabels=class_names,  # x軸のラベルをクラス名に設定
            yticklabels=class_names)  # y軸のラベルをクラス名に設定

# カスタムカラーバーを追加
sm = plt.cm.ScalarMappable(cmap=ListedColormap(colors[:2]), norm=plt.Normalize(vmin=0, vmax=1))
sm.set_array([])
cbar = plt.colorbar(sm, label='p-value', ticks=[0.25, 0.75])
cbar.set_ticklabels(['p < 0.05', 'p ≥ 0.05'])

plt.title('Pairwise Comparison P-Values (Tukey HSD)')
plt.xticks(rotation=45, ha='right')  # x軸のラベルを45度回転
plt.yticks(rotation=0)  # y軸のラベルを水平に
plt.tight_layout()  # レイアウトを調整
plt.show()

###**Tukey_sumaho/slit別**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import scipy.stats as stats

# CSVファイルを読み込む
df_slit = pd.read_csv(file_path1)
df_sumaho = pd.read_csv(file_path2)

# クラス名を定義
class_names = ["infection", "normal", "non-infection", "scar", "tumor", "deposit", "APAC", "lens opacity", "bullous"]

# クラス名を数字に対応させる
df_slit['class_name'] = df_slit['top1'].map(dict(enumerate(class_names)))
df_sumaho['class_name'] = df_sumaho['top1'].map(dict(enumerate(class_names)))

# データフレームを整形
df_slit['Type'] = 'Slit'
df_sumaho['Type'] = 'Sumaho'

def perform_anova_and_tukey(df, title):
    # 欠損値を削除
    df = df.dropna(subset=[layer_name])

    # ANOVAを実行
    anova_result = stats.f_oneway(
        df[df['class_name'] == 'infection'][layer_name],
        df[df['class_name'] == 'normal'][layer_name],
        df[df['class_name'] == 'non-infection'][layer_name],
        df[df['class_name'] == 'scar'][layer_name],
        df[df['class_name'] == 'tumor'][layer_name],
        df[df['class_name'] == 'deposit'][layer_name],
        df[df['class_name'] == 'APAC'][layer_name],
        df[df['class_name'] == 'lens opacity'][layer_name],
        df[df['class_name'] == 'bullous'][layer_name]
    )

    print(f"{title} ANOVA result: F={anova_result.statistic}, p={anova_result.pvalue}")

    # 事後検定（TukeyのHSD検定）を実行
    tukey_result = pairwise_tukeyhsd(endog=df[layer_name], groups=df['class_name'])

    # 結果をDataFrameに変換
    tukey_result_df = pd.DataFrame(data=tukey_result._results_table.data[1:], columns=tukey_result._results_table.data[0])

    # マトリックス図を作成するためにクラス名をインデックスとして使用
    matrix = pd.DataFrame(np.zeros((len(class_names), len(class_names))), index=class_names, columns=class_names)

    # マトリックスにp値を入力
    for i in range(len(tukey_result_df)):
        group1 = tukey_result_df.loc[i, 'group1']
        group2 = tukey_result_df.loc[i, 'group2']
        p_value = float(tukey_result_df.loc[i, 'p-adj'])
        matrix.loc[group1, group2] = p_value
        matrix.loc[group2, group1] = p_value

    # 同じクラス間のセルをNaNに設定
    np.fill_diagonal(matrix.values, np.nan)

    # ヒートマップを描画
    plt.figure(figsize=(12, 10))
    sns.heatmap(matrix, annot=True, fmt=".3f", cmap="coolwarm", cbar_kws={'label': 'p-value'}, mask=matrix.isnull())
    plt.title(f'Pairwise Comparison P-Values (Tukey HSD) - {title}')
    plt.show()

# スリットランプデータの解析
perform_anova_and_tukey(df_slit, "Slit Lamp Data")

# スマートフォンデータの解析
perform_anova_and_tukey(df_sumaho, "Smartphone Data")

In [ ]:
import pandas as pd
import scipy.stats as stats

# CSVファイルを読み込む
df_slit = pd.read_csv(file_path1)
df_sumaho = pd.read_csv(file_path2)

# クラス名を定義
class_names = ["infection", "normal", "non-infection", "scar", "tumor", "deposit", "APAC", "lens opacity", "bullous"]

# クラス名を数字に対応させる
df_slit['class_name'] = df_slit['top1'].map(dict(enumerate(class_names)))
df_sumaho['class_name'] = df_sumaho['top1'].map(dict(enumerate(class_names)))

# データフレームを整形
df_slit['Type'] = 'Slit'
df_sumaho['Type'] = 'Sumaho'

# 両方のデータを結合
df_combined = pd.concat([df_slit, df_sumaho]).reset_index(drop=True)

# "scar" + "non-infection" グループとその他のクラスに分類
df_combined['group'] = df_combined['class_name'].apply(lambda x: 'scar + non-infection' if x in ['scar', 'non-infection'] else 'others')

# グループごとにデータを抽出
scar_non_infection_group = df_combined[df_combined['group'] == 'scar + non-infection'][layer_name].dropna()
others_group = df_combined[df_combined['group'] == 'others'][layer_name].dropna()

# t検定を実行
t_stat, p_value = stats.ttest_ind(scar_non_infection_group, others_group)

# 結果を表示
print(f"t-statistic: {t_stat}, p-value: {p_value}")

# 統計値を計算
scar_non_infection_mean = scar_non_infection_group.mean()
scar_non_infection_std = scar_non_infection_group.std()
others_mean = others_group.mean()
others_std = others_group.std()

print(f"scar + non-infection mean: {scar_non_infection_mean}, std: {scar_non_infection_std}")
print(f"others mean: {others_mean}, std: {others_std}")
